## Import Libraries

In [ ]:
import folium
import gurobipy as gp
import gurobipy as gp
import math
import matplotlib.pyplot as plt
import matplotlib.pyplot as plt
import matplotlib.pyplot as plt
import numpy as np
import numpy as np
import numpy as np
import pandas as pd
import pandas as pd
import pandas as pd
import requests
import seaborn as sns
from branca.colormap import linear
from folium.plugins import HeatMap
from gurobipy import GRB
from gurobipy import GRB
from pmdarima import auto_arima
from scipy import stats
from scipy import stats
from scipy.cluster.hierarchy import linkage, dendrogram, fcluster
from scipy.cluster.hierarchy import linkage, dendrogram, fcluster
from scipy.stats import zscore
from scipy.stats import zscore
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.graphics.tsaplots import plot_pacf
from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.tsa.stattools import acf

## Read Data

In [ ]:
states_cases=pd.read_excel("Data 08.22.2024.xlsx",sheet_name='State Cases')
county_vac=pd.read_excel("Data 08.22.2024.xlsx",sheet_name='Vaccine')
county_cases=pd.read_excel("Data 08.22.2024.xlsx",sheet_name='County Cases')
county_demand=pd.read_excel("Updated Data-7.3.24.xlsx",sheet_name='Demand')
county_pharmacy=pd.read_excel("Updated Data-7.3.24.xlsx",sheet_name='Pharmacy')
#testing=pd.read_excel("C:/Users/moham/OneDrive - Virginia Tech/Kimiya Research/Paper 1/Data/Data 08.22.2024.xlsx",sheet_name='COVID-19_Diagnostic_Laboratory_')
allocation=pd.read_excel("Data 08.22.2024.xlsx",sheet_name='Allocation')
# counties_test=pd.DataFrame.from_dict(counties_test,orient='Index',columns=['GEO_ID','Pop'])
#2#1# Data Processing: Define supply availability
"""
Data for supply is manually imported
"""
Pf_M=[54.924,47.658,52.728,47.658,47.658,48.438,48.438,46.332,45.747,51.948,56.745,60.606,61.659,62.595,60.255,62.088,52.962,53.6835,56.94,56.94,1076.0025]
Pf_MW=[52.65,37.5375,47.8725,37.3425,37.5375,38.9025,38.9025,38.5125,38.9025,38.9025,61.542,70.551,76.635,82.836,84.006,102.843,84.006,85.761,91.962,91.962,1239.1665]
Pf_N=[13.1625,9.4575,11.895,9.36,9.4575,9.6525,9.6525,9.6525,9.6525,9.6525,15.327,17.55,19.071,20.475,21.06,25.506,21.06,21.294,22.932,22.932,308.802]
Pf_S=[75.855,54.015,68.9325,53.9175,54.015,55.965,55.965,55.77,55.965,55.965,88.452,101.673,111.033,119.34,121.212,148.59,121.212,123.903,133.146,133.146,1788.072]
Pf_SW=[33.345,23.79,30.3225,23.6925,23.79,24.6675,24.6675,24.57,24.6675,24.6675,39.078,44.694,48.789,52.533,53.469,65.52,53.469,54.639,58.617,58.617,787.605]
Pf_W=[59.67,39.975,51.1875,42.6075,41.2425,41.73,41.73,44.1675,41.73,41.73,65.871,80.847,82.368,88.452,89.973,119.691,89.973,91.962,98.631,98.631,1352.169]
Mo_M=[22.064,74.014,40.344,40.28,39.84,40.73,40.73,45.1,47.39,48.69,48.69,48.69,48.69,48.69,48.69,48.69,48.53,46.98,45.37,45.37,917.572]
Mo_MW=[0,106.92,36.97,36.82,36.97,38.8,38.8,51.2,55.97,60.49,60.49,60.49,60.49,60.49,60.49,60.49,63.2,63.2,68.17,68.17,1088.62]
Mo_N=[0,26.4,9.14,9.1,9.14,9.6,9.6,12.65,13.83,14.95,14.95,14.95,14.95,14.95,14.95,14.95,15.61,15.61,16.84,16.84,269.01]
Mo_S=[0,154.65,53.45,53.22,53.45,56.15,56.15,74.05,80.94,87.46,87.46,87.46,87.46,87.46,87.46,87.46,91.37,91.37,98.56,98.56,1574.14]
Mo_SW=[0,68.22,23.58,23.49,23.58,24.78,24.78,32.67,35.72,38.6,38.6,38.6,38.6,38.6,38.6,38.6,40.33,40.33,43.5,43.5,694.68]
Mo_W=[0,117.45,39.69,42.07,39.69,41.68,41.68,58.2,60.09,64.93,64.93,69.86,64.93,64.93,64.93,70.18,67.85,67.85,73.16,73.16,1187.26]
Jo_M=[0,0,0,0,0,0,0,0,0,0,0,40.9700,0,5.6600,5.6600,28.7800,73.3000,8.6800,0,0,0]
Jo_MW=[0,0,0,0,0,0,0,0,0,0,0,50.9100,0,6.4400,7.0700,35.8200,91.0400,10.8000,0,0,0]
Jo_N=[0,0,0,0,0,0,0,0,0,0,0,12.5900,0,1.7500,1.7500,8.8500,22.4800,2.6900,0,0,0]
Jo_S=[0,0,0,0,0,0,0,0,0,0,0,73.6100,0,10.1800,10.1800,51.7700,131.6700,15.5900,0,0,0]
Jo_SW=[0,0,0,0,0,0,0,0,0,0,0,32.4700,0,4.4900,4.4900,22.8200,58.1000,6.8700,0,0,0]
Jo_W=[0,0,0,0,0,0,0,0,0,0,0,55.5300,0,7.6800,7.6800,39.0300,99.3100,11.7800,0,0,0]
Pf_MANE=[]
Mo_MANE=[]
Jo_MANE=[]
for r in range(0,len(Pf_M)):
    Pf_MANE.append(Pf_M[r]+Pf_N[r])
    Mo_MANE.append(Mo_M[r]+Mo_N[r])
    Jo_MANE.append(Mo_M[r]+Mo_N[r])

"""
Distances are defined for calculating the cost
"""
distance={'AL':625,'AK':3050,'AZ':1850,'AR':725,'CA':2150,'CO':1150,'CT':725,'DE':675,'FL':1250,'GA':750,'HI':4250,'ID':1650,'IL':115,'IN':145,'IA':350,'KS':725,'KY':350,'LA':875,'ME':1025,'MD':625,'MA':825,'MI':50,'MN':400,'MS':775,'MO':500,'MT':1450,'NE':675,'NV':1750,'NH':875,'NJ':675,'NM':1450,'NY':660,'NC':575,'ND':750,'OH':275,'OK':875,'OR':2150,'PA':525,'RI':825,'SC':725,'SD':650,'TN':425,'TX':1250,'UT':1450,'VT':825,'VA':625,'WA':2150,'WV':425,'WI':50,'WY':1250,'DC':590}

"""
Vaccine administration data
"""
Pfizer_dose1={}
Moderna_dose1={}
Janssen={}
states = ['AL', 'AK', 'AZ', 'AR', 'CA', 'CO', 'CT', 'DE', 'DC', 'FL', 'GA', 'HI', 'ID', 'IL', 'IN', 'IA', 'KS', 'KY',
          'LA', 'ME', 'MD', 'MA', 'MI', 'MN', 'MS', 'MO', 'MT', 'NE', 'NV', 'NH', 'NJ', 'NM', 'NY', 'NC', 'ND', 'OH',
          'OK', 'OR', 'PA', 'RI', 'SC', 'SD', 'TN', 'TX', 'UT', 'VT', 'VA', 'WA', 'WV', 'WI', 'WY']
for s in states:
    for t in range(len(list(allocation['Week of Allocations*'].unique()))):
        Pfizer_dose1[t,s]=list(allocation.loc[(allocation['State']==s)&(allocation['Week of Allocations*']==list(allocation['Week of Allocations*'].unique())[t])]['1st Dose Allocations Pfizer'])[0]
        Moderna_dose1[t,s]=list(allocation.loc[(allocation['State']==s)&(allocation['Week of Allocations*']==list(allocation['Week of Allocations*'].unique())[t])]['1st Dose Allocations Moderna'])[0]
        Janssen[t,s]=list(allocation.loc[(allocation['State']==s)&(allocation['Week of Allocations*']==list(allocation['Week of Allocations*'].unique())[t])]['1st Dose Allocations Janssen'])[0]



## Data Preprocessing

In [ ]:
#2#2# Data Processing: Define Lists and Dictionaries
Vac_TX=[0,205463, 610667,870895,943542,567960,1497157,1711009,2027595,1849865,955496,792447,783461,583813,453895,351140]
#DEFINING REGIONS
New_England=['CT','ME','MA','NH','RI','VT']
Middle_Atlantic=['DE','MD','NJ','NY','PA','DC']
South=['AL','AR','FL','GA','KY','LA','MS','MO','NC','SC','TN','VA','WV']
Midwest=['IL','IN','IA','KS','MI','MN','NE','ND','OH','SD','WI']
Southwest=['AZ','NM','OK','TX']
West=['AK','CA','CO','HI','ID','MT','NV','OR','UT','WA','WY']
columns1=(states_cases.columns)
columns2=(county_vac.columns)
columns3=(county_cases.columns)
dates1=list(states_cases["end_date*"].unique())
dates2=list(county_vac["Date*"].unique())
dates3=list(county_cases["date*"].unique())
states1=list(states_cases["state"].unique())
states2=list(county_vac["Recip_State"].unique())
states3=list(county_cases["State_abr"].unique())
counties0={}
counties1={}
counties2={}
counties20={}
counties3={}
state_vaccine0={}
state_vaccine={}
state_sum_county0={}
state_sum_county={}
counties20['UNK']=list(county_vac.loc[county_vac["Recip_State"]=='UNK']["FIPS"].unique())
#COUNTY VACCINE
for s2 in states2:
    counties2[s2]=list(county_vac.loc[county_vac["Recip_State"]==s2]["FIPS"].unique())
#CALCULATE STATE VACCINES BASED ON COUNTY VALUES
for s2 in states2:
    for t2 in range(0,len(dates2)):
        state_vaccine0[t2,s2]=list(county_vac.loc[(county_vac["Recip_State"]==s2)&(county_vac["Date*"]==dates2[t2])]['Administered_Dose1_Recip_filled'])
        state_vaccine[t2,s2]=sum(a for a in state_vaccine0[t2,s2])
        if s2=='TX':
            state_vaccine[t2,s2]=sum(Vac_TX[t] for t in range(0,t2+1))
#COUNTY INFECTIONS
for s3 in states3:
    counties3[s3]=list(county_cases.loc[county_cases["State_abr"]==s3]["fips"].unique())
#STATE CASES BASED ON COUNTY CASES
for s2 in states3:
    for t2 in range(0,len(dates3)):
        state_sum_county0[t2,s2]=list(county_cases.loc[(county_cases["State_abr"]==s2)&(county_cases["date*"]==dates3[t2])]['cases'])
        state_sum_county[t2,s2]=sum(a for a in state_sum_county0[t2,s2])
#DROP UNNECESSARY STATES
indexdmh=county_vac[(county_vac["Recip_State"]=='MH')].index
county_vac.drop(indexdmh , inplace=True)
#UNKNOWN COUNTIES DROPPED
county_vac_unk=county_vac[(county_vac["FIPS"]=='UNK')]
county_cases_unk=county_cases[(county_cases["fips"]=='UNK')]
indexunk1=county_cases[(county_cases["fips"]=='UNK')].index
indexunk2=county_vac[(county_vac["FIPS"]=='UNK')].index
state_vaccine0_unk={}
state_vaccine_unk={}
for s2 in states2:
    for t2 in range(0,len(dates2)):
        state_vaccine0_unk[t2,s2]=list(county_vac_unk.loc[(county_vac_unk["Recip_State"]==s2)&(county_vac_unk["Date*"]==dates2[t2])]['Administered_Dose1_Recip_filled'])
        state_vaccine_unk[t2,s2]=sum(a for a in state_vaccine0_unk[t2,s2])
for s in states2:
    print(sum(state_vaccine_unk[a,s] for a in  range(0,len(dates2))))
county_cases.drop(indexunk1 , inplace=True)
county_vac.drop(indexunk2, inplace=True)

#REDEFINE THE COLUMNS AND DATES
columns1=(states_cases.columns)
columns2=(county_vac.columns)
columns3=(county_cases.columns)
# dates0=list(agg_data["Date*"].unique())
dates1=list(states_cases["end_date*"].unique())
dates2=list(county_vac["Date*"].unique())
dates3=list(county_cases["date*"].unique())
# states0=list(agg_data["Recip_State"].unique())
states1=list(states_cases["state"].unique())
states2=list(county_vac["Recip_State"].unique())
states3=list(county_cases["State_abr"].unique())
counties0={}
counties1={}
counties2={}
counties20={}
counties3={}
counties20['UNK']=list(county_vac.loc[county_vac["Recip_State"]=='UNK']["FIPS"].unique())

for s2 in states2:
    counties2[s2]=list(county_vac.loc[county_vac["Recip_State"]==s2]["FIPS"].unique())
for s3 in states3:
    counties3[s3]=list(county_cases.loc[county_cases["State_abr"]==s3]["fips"].unique())
vaccination={}
for s in states2:
    for c in counties2[s]:
        vaccination[s,c]=list(county_vac.loc[(county_vac["Recip_State"]==s) & (county_vac["FIPS"]==c)]["Administered_Dose1_Recip_filled"])
# #INCONSISTENT DATA
popplus12={}
for s in states2:
    for c in counties2[s]:
        popplus12[s,c]=list(county_vac.loc[(county_vac["Recip_State"]==s) & (county_vac["FIPS"]==c)]["Census2019_12PlusPop"].unique())[0]
        # print(popplus12[s,c],s,c)
popplus12_s={}
for s in states2:
    popplus12_s[s]=sum(popplus12[s,c] for c in counties2[s])
diff23={}
diff32={}
states3.remove('MP')
for s2 in states2:
    for s3 in states3:
        diff23[s2]=[]
        diff32[s3]=[]
        for i in range(0,len(counties2[s2])):
            if counties2[s2][i] not in counties3[s2]:
                diff23[s2].append(counties2[s2][i])
        for j in range(0,len(counties3[s3])):
            if counties3[s3][j] not in counties2[s3]:
                diff32[s3].append(counties3[s3][j])
for drop in diff23['NY']:
    indexdiff32=county_vac[(county_vac["Recip_State"]=='NY') & ((county_vac["FIPS"]== drop))].index
    county_vac.drop(indexdiff32 , inplace=True)
for drop in diff23['AK']:
    indexdiff321=county_vac[(county_vac["Recip_State"]=='AK') & ((county_vac["FIPS"]== drop))].index
    county_vac.drop(indexdiff321 , inplace=True)
for drop in diff32['AK']:
    indexdiff3210=county_cases[(county_cases["State_abr"]=='AK') & ((county_cases["fips"]== drop))].index
    county_cases.drop(indexdiff3210 , inplace=True)
states1=list(states_cases["state"].unique())
states2=list(county_vac["Recip_State"].unique())
states3=list(county_cases["State_abr"].unique())
states_cases0={}
for s in states2:
    states_cases0[s]=list(states_cases.loc[states_cases["state"]==s]['tot_cases'])

"""
Infections are aggregated
"""
states = ['AL', 'AK', 'AZ', 'AR', 'CA', 'CO', 'CT', 'DE', 'DC', 'FL', 'GA', 'HI', 'ID', 'IL', 'IN', 'IA', 'KS', 'KY',
          'LA', 'ME', 'MD', 'MA', 'MI', 'MN', 'MS', 'MO', 'MT', 'NE', 'NV', 'NH', 'NJ', 'NM', 'NY', 'NC', 'ND', 'OH',
          'OK', 'OR', 'PA', 'RI', 'SC', 'SD', 'TN', 'TX', 'UT', 'VT', 'VA', 'WA', 'WV', 'WI', 'WY']
infections={}
for s in states:
    for c in counties2[s]:
        infections[s,c]=[]
        for t in dates2:
        # if c!=NaN or c!=0:
            #if(len(list(county_cases.loc[(county_cases["State_abr"]==s) & (county_cases["fips"]==c)&(county_cases["date*"]==t)]["cases"])))==0:
                #print(s,c,t)
            infections[s,c].append(list(county_cases.loc[(county_cases["State_abr"]==s) & (county_cases["fips"]==c)&(county_cases["date*"]==t)]["cases"])[0])
for s in states:
    for c in counties2[s]:
        infections[s,c].insert(0,(states_cases0[s][22])/(states_cases0[s][23])*infections[s,c][0])
"""
Checking If supply and administration data are consistent
"""
for s in states:
    for t in range(1,11):
        if state_vaccine[t,s]-sum(Pfizer_dose1[2*a,s]+Moderna_dose1[2*a,s]+Janssen[2*a,s]+Pfizer_dose1[2*a+1,s]+Moderna_dose1[2*a+1,s]+Janssen[2*a+1,s] for a in range(0,t))>=0:
            print(s,t,state_vaccine[t,s]-sum(Pfizer_dose1[2*a,s]+Moderna_dose1[2*a,s]+Janssen[2*a,s]+Pfizer_dose1[2*a+1,s]+Moderna_dose1[2*a+1,s]+Janssen[2*a+1,s] for a in range(0,t)),'under')
        if state_vaccine[t,s]-sum(Pfizer_dose1[2*a,s]+Moderna_dose1[2*a,s]+Janssen[2*a,s]+Pfizer_dose1[2*a+1,s]+Moderna_dose1[2*a+1,s]+Janssen[2*a+1,s] for a in range(0,t))<0:
            print(s,t,-state_vaccine[t,s]+sum(Pfizer_dose1[2*a,s]+Moderna_dose1[2*a,s]+Janssen[2*a,s]+Pfizer_dose1[2*a+1,s]+Moderna_dose1[2*a+1,s]+Janssen[2*a+1,s] for a in range(0,t)),'over')
"""
Access Data from US News
"""
Access=[12,2,7,11,25,16,47,40,35,5,6,49,30,19,21,41,31,28,29,36,43,48,34,32,1,8,20,24,0,44,38,9,37,27,18,23,3,22,42,46,14,13,15,10,17,45,35,39,26,33,4]
access1={}
i=0
access={}
for s in states2:
    access1[s]=50-Access[i]
    i+=1
access_log={}
for s in states2:
    access_log[s]=np.log(access1[s])
for s in states2:
    access_log[s]=access_log[s]/max(access_log.values())


## Defining Parameters

In [ ]:
reinf_rate=0.01
reinf_period=6
mu=14/(75*365)
gamma = 1
gamma1=1
gamma =1
T=12
vac_eff=0.2
def createList(r1, r2):
    return [item for item in range(r1, r2 + 1)]
time=['12/2/2020*','12/16/2020*', '12/30/2020*', '1/27/2021*', '2/10/2021*', '2/24/2021*', '3/10/2021*', '3/24/2021*', '4/7/2021*', '4/21/2021*', '5/5/2021*', '5/19/2021*']
underreported={}
for s in states2:
    for c in counties2[s]:
        for t in range(2,T+1):
            if t>=7:
                underreported[s,c,t]=1.5
            else:
                underreported[s,c,t]=1.5
wills={}
for s in states:
    wills[s]=(1-coefs[s][0])*state_vaccine[T-1,s]/popplus12_s[s]
    if (wills[s]>=1):
        wills[s]=1
    if s=='TX':
        wills[s]=sum(Vac_TX[t] for t in range(0,11))/popplus12_s[s]
# print(wills['TX'])

state_will_time={}
for t in range(1,T):
    for s in states:
        state_will_time[t,s]=state_vaccine[t-1,s]/popplus12_s[s]
        if s=='TX':
            state_will_time[t,s]=sum(Vac_TX[a-2] for a in range(0,t-1))/popplus12_s[s]
demand={}
for s in states:
    for t in range(2,T):
        demand[t,s]=Vac[t,s]/popplus12_s[s]/(1-coefs[s][0])
will_final={}
for s in states2:
    for c in counties2[s]:
        will_final[s,c]=vaccination[s,c][T-1]/popplus12[s,c]
        if (will_final[s,c]>=1):
            will_final[s,c]=1
#
for s in ['TX']:
    for c in counties2[s]:
        will_final[s, c] = 0.2
# (1-coef_c[s,c][0])*
hesit={}
for s in states2:
    for c in counties2[s]:
        hesit[c]=county_demand.loc[county_demand['FIPS Code']==c]['Estimated hesitant or unsure'].values[0]
        print(hesit[c],will_final[s,c])
state_will={}
for s in states2:
    state_will[s]=sum((1-hesit[c])*popplus12[s,c] for c in counties2[s])/sum(popplus12[s,c] for c in counties2[s])
county_cap={}
difference_result = []
counties_test={}
sum_n={}
average_n={}
number=[]
for s in states:
    for item in counties2[s]:
      if item not in list(county_pharmacy['GEO_ID']):
        difference_result.append(item)
        counties_test[item]=[new_SVI[item], popplus12[s,item][0]]
print(counties_test)

staff=5
shift=8
workdays=12
vacperhour=6
for s in states:
    for c in counties2[s]:
        county_cap[c]=1*(county_pharmacy.loc[county_pharmacy['GEO_ID']==c]['Number'].values[0])*staff*shift*workdays*vacperhour/10000
print(county_cap)

## Calculating Effective Infection Rate

In [ ]:
states2=states
coef_c={}
inf_will={}
for s in states2:
    for c in counties2[s]:
        inf_will[s,c]=0.4
        coef_c[s,c]=[inf_will[s,c]*infections[s,c][1]/popplus12[s,c]]
        if coef_c[s,c][0]>=1:
            print(coef_c[s,c][0])
        for t in range(0,len(dates2)):
            if vaccination[s,c][t] >= popplus12[s,c]:
                print(s,c)
                vaccination[s,c][t]=popplus12[s,c]
S00={}
V00={}
I00={}
beta_c={}
beta_c1={}
Vac_c={}
# inf={}
# inf1={}
epidemic_condition={}
for s in states2:
    for c in counties2[s]:
        epidemic_condition[s,c]=np.ceil(0.00001*popplus12[s,c])
for s in states2:
    for c in counties2[s]:
        if (infections[s,c][1]-infections[s,c][0])<=0:
            if infections[s,c][0]==0:
                hzero=1
            else:
                hzero=0
            S00[2,s,c] = popplus12[s,c]-(1-reinf_rate)*underreported[s,c,t]*infections[s,c][0]- hzero*epidemic_condition[s,c]
            I00[2,s,c]= reinf_rate* underreported[s,c,t]*infections[s,c][0]+hzero*epidemic_condition[s,c]
        else:
            S00[2,s,c] = popplus12[s,c]-(1-reinf_rate)*underreported[s,c,t]*(infections[s,c][1])
            I00[2,s,c]= underreported[s,c,t]*(infections[s,c][1]-infections[s,c][0])

        V00[2,s,c]=0
        Vac_c[2,s,c]=(1-coef_c[s,c][0])*(vaccination[s,c][1]-vaccination[s,c][0])
        Vac_c[1,s,c]=0
for t in range(2,T+1):
    for s in states2:
        for c in counties2[s]:
            # if t>=3:
            #     epidemic_condition=underreported[s,c,t]*(infections[s,c][t-1]-infections[s,c][t-2])
            # else:
            epidemic_condition[s,c]=np.ceil(0.00001*popplus12[s,c])
            if infections[s,c][t]-infections[s,c][t-1]<=0:
                I00[t+1,s,c] = (1-mu-gamma)*I00[t,s,c] + epidemic_condition[s,c]
            else:
                I00[t+1,s,c] = (1-mu-gamma)*I00[t,s,c] + underreported[s,c,t]*(infections[s,c][t]-infections[s,c][t-1])
            # I00[t+1,s,c] = infections[s,c][t+8+1]
            if (vaccination[s,c][t-1]-vaccination[s,c][t-2])<=0:
                Vac_c[t,s,c]=0
            else:
                Vac_c[t,s,c]= (1-coef_c[s,c][0])*(vaccination[s,c][t-1]-vaccination[s,c][t-2])
            if s=='TX':
                Vac_c[t,s,c]= (1-coef_c[s,c][0])*Vac_TX[t-2]*popplus12[s,c]/popplus12_s[s]

x={}
reinfection={}
for t in range(2,T+1):
    for s in states2:
        for c in counties2[s]:
            x[t,s,c]=[]
            if t>=7:
                reinfection[t,s,c]=(reinf_rate)*1.5*(infections[s,c][t-6]-infections[s,c][t-7])
            else:
                reinfection[t,s,c]=0
            if ((infections[s,c][t]-infections[s,c][t-1])<=0):
                epidemic_condition[s,c]=np.ceil(0.00001*popplus12[s,c])
                a = np.array([[vac_eff*I00[t,s,c]*V00[t,s,c], 1, 0], [(I00[t,s,c]*S00[t,s,c] +vac_eff*I00[t,s,c]*V00[t,s,c]),0, 0],[(I00[t,s,c]*S00[t,s,c]),0, 1]])
                b = np.array([((1-mu)*V00[t,s,c] +Vac_c[t-gamma1,s,c]) , 1*epidemic_condition[s,c],(1-mu)*S00[t,s,c] +mu - Vac_c[t-gamma1,s,c]+reinfection[t,s,c]])
            else:
                a = np.array([[vac_eff*I00[t,s,c]*V00[t,s,c], 1, 0], [(I00[t,s,c]*S00[t,s,c] +vac_eff*I00[t,s,c]*V00[t,s,c]),0, 0],[(I00[t,s,c]*S00[t,s,c]),0, 1]])
                b = np.array([((1-mu)*V00[t,s,c] +Vac_c[t-gamma1,s,c]) , underreported[s,c,t]*(infections[s,c][t]-infections[s,c][t-1]),(1-mu)*S00[t,s,c] +mu- Vac_c[t-gamma1,s,c]+reinfection[t,s,c]])
            x[t,s,c] = list(np.linalg.solve(a, b))
            [beta_c[t,s,c],V00[t+1,s,c],S00[t+1,s,c]]=x[t,s,c]
            if S00[t+1,s,c]<0:
                S00[t+1,s,c]=0
will={}
for s in states2:
    for c in counties2[s]:
        will[s,c]=[(1-coef_c[s,c][0])*vaccination[s,c][T-2]/popplus12[s,c]]
        if (will[s,c][0]>=1):
            will[s,c][0]=1
o=0
for s in states2:
    for c in counties2[s]:
        for t in range(2,T+1):
            if S00[t,s,c] <0:
                print(S00[t,s,c],I00[t,s,c],beta_c[t,s,c],beta_c[t-1,s,c]*popplus12[s,c],s,c,t,infections[s,c][t],infections[s,c][t-1],infections[s,c][t-2],popplus12[s,c],vaccination[s,c][t])
                o+=1
beta_c0={(c):beta_c[2,s,c] for s in states2 for c in counties2[s]}
# print(beta_c0)
#Initial periods effective infection rate
S01={}
V01={}
I01={}
R01={}
beta01={}
beta101={}
T=24
underreported={}
for s in states:
    for t in range(-1,T+1):
        if t<=15:
            underreported[s,t]=2
        else:
            underreported[s,t]=1.2
        if s=='TX':
            underreported[s,t]=2.5

#####State Reinfections
reinf0={}
for t in range(-1,T+1):
    for s in states:
        if t>=10:
            reinf0[t,s]=reinf_rate*underreported[s,t]*(states_cases0[s][t-reinf_period]-states_cases0[s][t-reinf_period-1])
        else:
            reinf0[t,s]=0
for s in states:
    S01[-1,s] = popplus12_s[s]
    I01[-1,s]= 0
    R01[-1,s]=0
    states_cases0[s][-1]=0
for t in range(-1,T+1):
    for s in states:
        # for c in counties2[s]:
        if((states_cases0[s][t+1]-states_cases0[s][t])>0):
            I01[t+1,s] = (1-mu-gamma)*I01[t,s] +underreported[s,t]*(states_cases0[s][t+1]-states_cases0[s][t])
            R01[t+1,s]=(1-gamma)*I01[t,s]-(-mu)*R01[t,s]
            S01[t+1,s]=(1-mu)*S01[t,s] +mu-underreported[s,t]*(states_cases0[s][t+1]-states_cases0[s][t])+reinf0[t,s]
        else:
            I01[t+1,s]=0
            S01[t+1,s]=S01[t,s]
            R01[t+1,s]=0
S0101={}
I0101={}
for s in states:
    S0101[0,s]=S01[23,s]
    I0101[0,s]=I01[23,s]
for t in range(1,T+1):
    for s in states:
        if((states_cases0[s][t+1]-states_cases0[s][t])>0):
            print((states_cases0[s][t+1]-states_cases0[s][t])/S01[t,s]/I01[t,s],s,t,I01[t,s],states_cases0[s][t+1],states_cases0[s][t])
beta01={}
for t in range(1,24):
    for s in states:
        if I01[t,s] ==0:
            beta01[t,s]=0.00
        else:
            beta01[t,s]=underreported[s,t]*(states_cases0[s][t+1]-states_cases0[s][t])/S01[t,s]/I01[t,s]

#PRINT BASIC REPRODUCTIVE NUMBER IF YOU WANT
# for s in states:
#     print(s,popplus12_s[s]*beta01[4,s])

T = len(dates2)
T=14
inf_will=0.4
# underreported={}
for s in states2:
    for t in range(1,T+1):
        underreported[s,t]=1.5
coefs={}
for s in states:
    coefs[s]=[inf_will*underreported[s,T]*states_cases0[s][23]/popplus12_s[s]]
    if coefs[s][0]>=1:
        print(coefs[s])
states_cases0={}
for s in states:
    states_cases0[s]=list(states_cases.loc[states_cases["state"]==s]['tot_cases'])
T = len(time)+1
S0={}
V0={}
I0={}
beta={}
beta1={}
Vac={}
infs={}
inf1s={}

for s in states:
    S0[2,s] = S01[23,s]
    I0[2,s]= I01[23,s]
    V0[2,s]=0
    Vac[2,s]=(1-coefs[s][0])*(state_vaccine[1,s]-state_vaccine[0,s])
    Vac[1,s]=0

epidemic_condition=4 #can be anything
for t in range(2,T+1):
    for s in states:
        if states_cases0[s][t+22]-states_cases0[s][t+21]<=0:
            I0[t+1,s] = (1-mu-gamma)*I0[t,s] + epidemic_condition
        else:
            I0[t+1,s] = (1-mu-gamma)*I0[t,s] +underreported[s,t]*(states_cases0[s][t+22]-states_cases0[s][t+21])
        Vac[t,s]= (1-coefs[s][0])*(state_vaccine[t-1,s]-state_vaccine[t-2,s])
        if s == 'TX':
            Vac[t,s]=Vac_TX[t-2]
        Vac[1,s]=0
x={}
reinfection={}
for t in range(2,T+1):
    for s in states:
        x[t,s]=[]
        if t<=reinf_period+2:
            reinfection[t,s]=reinf_rate*underreported[s,t+21-reinf_period]*(states_cases0[s][t+22-reinf_period]-states_cases0[s][t+21-reinf_period])
        else:
            reinfection[t,s]=reinf_rate*beta[t-reinf_period-1,s]*S0[t-reinf_period-1,s]*I0[t-reinf_period-1,s]
        reinfection[t,s]=reinf_rate*underreported[s,t]*(states_cases0[s][t+22-reinf_period]-states_cases0[s][t+21-reinf_period])
        a = np.array([[effec*I0[t,s]*V0[t,s], 1, 0], [(I0[t,s]*S0[t,s] +effec*I0[t,s]*V0[t,s]),0, 0],[(I0[t,s]*S0[t,s]),0, 1]])
        b = np.array([((1-mu)*V0[t,s] +Vac[t-gamma1,s]) , underreported[s,t]*(states_cases0[s][t+22]-states_cases0[s][t+21]),(1-mu)*S0[t,s] +mu+reinf0[t,s] - Vac[t-gamma1,s]+reinfection[t,s]])
        x[t,s] = list(np.linalg.solve(a, b))
        [beta[t,s],V0[t+1,s],S0[t+1,s]]=x[t,s]
print(beta)

"""
Check if the rates are correct
for t in range(2,T):
    for s in states:
        if S0[t,s]<0:
            print(t,s)
"""


## Parameter Calibration

In [ ]:
##################################################Clustering
new_SVI={}
# for f in list(county_vac['FIPS'].unique()):
for c in list(county_vac['FIPS'].unique()):
    if c in beta_c0.keys():
    # new_SVI[f]=county_demand.loc[county_demand['FIPS Code']==f]['Social Vulnerability Index (SVI)'].values[0]
        new_SVI[c]=county_vac.loc[county_vac['FIPS']==c]['SVI'].values[0]
county_demand.columns
# print(new_SVI)
SVI={}
for f in list(county_vac['FIPS'].unique()):
    # SVI[f]=list(county_vac['SVI_CTGY'].loc[county_vac['FIPS']==f])[0]
    if list(county_vac['SVI_CTGY'].loc[county_vac['FIPS']==f])[0]=='A':
        SVI[f]=1
    if list(county_vac['SVI_CTGY'].loc[county_vac['FIPS']==f])[0]=='B':
        SVI[f]=2
    if list(county_vac['SVI_CTGY'].loc[county_vac['FIPS']==f])[0]=='C':
        SVI[f]=3
    if list(county_vac['SVI_CTGY'].loc[county_vac['FIPS']==f])[0]=='D':
        SVI[f]=4
print(SVI)

for s in states:
    for f in counties2[s]:
        if (math.isnan(new_SVI[f])== True) or (new_SVI[f]==0):
            new_SVI[f]=SVI[f]*25/100
county_pop=[]
for c in list(county_vac['FIPS'].unique()):
    if c in beta_c0.keys():
        county_pop.append(list(county_vac['Census2019'].loc[county_vac['FIPS']==c])[0])
data=pd.DataFrame( columns=['FIPS','Beta','SVI','Population'])
data['FIPS']=list(beta_c0.keys())
data['Population']=county_pop
data['Beta']=beta_c0.values()
data['SVI']=new_SVI.values()
print(data)
list1=[]
list2=[]
list3=[]
colors = [
    'Aqua', 'Aquamarine', 'Blue', 'BlueViolet', 'Brown', 'CadetBlue', 'Chartreuse', 'Chocolate',
    'Coral', 'Crimson', 'Cyan', 'DarkBlue', 'DarkCyan', 'DarkGoldenRod', 'DarkGray', 'DarkGreen',
    'DarkKhaki', 'DarkMagenta', 'DarkOliveGreen', 'DarkOrange', 'DarkOrchid', 'DarkRed',
    'DarkSalmon', 'DarkSeaGreen', 'DarkSlateBlue', 'DarkSlateGray', 'DarkTurquoise', 'DarkViolet',
    'DeepPink', 'DeepSkyBlue', 'DimGray', 'DodgerBlue', 'FireBrick', 'ForestGreen', 'Fuchsia',
    'Gold', 'GoldenRod', 'Gray', 'IndianRed', 'Indigo', 'LawnGreen', 'LightCoral', 'Lime',
    'Magenta', 'Maroon', 'MediumBlue', 'MediumOrchid', 'MediumPurple', 'MediumSeaGreen',
    'MediumSlateBlue', 'MediumSpringGreen'
]

d={}
%matplotlib inline

for s in states:
    list1=[]
    list2=[]
    list3=[]
    dataset=data.loc[data.isin(counties2[s])['FIPS']==True]
    dataset = dataset.apply(zscore)
    list1=list(dataset['SVI'])
    list2=list(dataset['Beta'])
    list3=list(dataset['Population'])
    for c in range(0,len(counties2[s])):
        list1[c]-=min(list(dataset['SVI']))
        list2[c]-=min(list(dataset['Beta']))
        list3[c]-=min(list(dataset['Population']))
    #Check the head after scaling
    # print(dataset)
    labelList = list(dataset['FIPS'].unique())
    for c in range(0,len(counties2[s])):
        if s=='DC':
            d['DC', 11001]=1
        else:
            d[s,counties2[s][c]]=(((list1[c]/max(list1))**2+(list2[c]/max(list2))**2+(list3[c]/max(list2))**2)**(0.5))/sum(((list1[c]/max(list1))**2+(list2[c]/max(list2))**2+(list3[c]/max(list2))**2)**(0.5) for c in  range(0,len(counties2[s])))
print(d)


# Sample dictionary
ordered_counties={}
data1={}
weights={}
plt.figure(facecolor='none',figsize=(10, 12))
# fig(figsize=(6, 10))
tick=[]
num=1
for s in states:
    # data={}

    data1[s]=[]
    ordered_counties[s]=[]
    for c in counties2[s]:
        data1[s].append(-np.log(d[s,c]))
        ordered_counties[s].append(-np.log(d[s,c]))
        weights[s,c]=-np.log(d[s,c])
    x = range(num*1000,num*1000+len(counties2[s]))
    y = data1[s]
    print(data1[s])
    # Create a scatter plot

    plt.scatter(y, x,label=f'{s}',c=colors[num-1],linewidth=1)
    plt.grid(False)
    plt.ylabel('US Counties',fontsize=20)
    plt.xlabel('Knapsack Weights',fontsize=20)

    plt.title('Equitable Infection-based Knapsack Weights',fontsize=20)
    plt.gcf().set_facecolor('none')
    # Move the legend outside the plot
    plt.legend(loc='upper left', bbox_to_anchor=(1.0, 1),fontsize=14,ncol=2)

    # Adjust layout to make space for the legend
    plt.tight_layout(rect=[0, 0, 1, 0.85])

    # Show the plot
    #
    # data1.clear()
    tick.append(num*1000)
    num+=1
    # cmap='viridis')
# plt.scatter(1.2,1000)

# for s in states:
#     tick.append(num*1000)
plt.yticks(ticks=tick,labels=states,fontsize=12
           )
plt.xticks(fontsize=12)
plt.gca().spines['top'].set_color('black')
plt.gca().spines['right'].set_color('black')
plt.gca().spines['left'].set_color('black')
plt.gca().spines['bottom'].set_color('black')
plt.grid(axis='y', linestyle='--', alpha=0.5,color='black')  # Customize style and transparency

print(ordered_counties)
# plt.legend(loc=1,fontsize=6)
plt.show()

access=access1
beta_0=[beta[2,s] for s in states2]
states_pop=[]
for s in states:
    states_pop.append(popplus12_s[s])
data_s=pd.DataFrame( columns=['State','Beta_s','Access','Population_s'])

data_s['State']=states
data_s['Population_s']=states_pop
data_s['Beta_s']=beta_0
data_s['Access']=access.values()
d_s=[]

list10=[]
list20=[]
list30=[]
dataset0=data_s.drop('State',axis=1)
dataset0 = dataset0.apply(zscore)
dataset0.insert(0,'State',states)

list10=list(dataset0['Access'])
list20=list(dataset0['Beta_s'])
list30=list(dataset0['Population_s'])
for s in range(0,len(states)):
    list10[s]-=min(list(dataset0['Access']))
    list20[s]-=min(list(dataset0['Beta_s']))
    list30[s]-=min(list(dataset0['Population_s']))
#Check the head after scaling
for s in range(0,len(states)):
    d_s.append((((list10[s]/max(list10))**2+(list20[s]/max(list20))**2+(list30[s]/max(list30))**2)**(0.5))/sum(((list10[s]/max(list10))**2+(list20[s]/max(list20))**2+(list30[s]/max(list30))**2)**(0.5) for s in  range(0,len(states))))
    #d_s.append(((list10[s]**2+list20[s]**2)**(0.5))/sum((list10[s]**2+list20[s]**2)**(0.5) for s in  range(0,len(states))))
print(d_s)
        # d.loc[d["FIPS"]==counties2[s][c]]["Distance"]=(list1[c]**2+list2[c]**2+list3[c]**2)**(0.5)
# print(d['DC', 11001])
# print(counties2['DC'])

# Sample dictionary
ordered_states={}
data1={}
# weights={}

# data={}
data10=[]
t1=2
T2=11
for s in range(len(states)):
    ordered_states[states[s]]=[]
    data10.append(d_s[s])
    ordered_states[states[s]].append(d_s[s])
print(data10)
x = states
y = data10
# print(data1[s]),
# Create a scatter plot
fig = plt.figure(facecolor='white')
plt.figure(figsize=(9, 12))
n=0
for s in states:
    plt.scatter(y[n], x[n],c=colors[n],label=f'{s}',linewidth=3)
    n+=1
plt.gca().spines['top'].set_color('black')
plt.gca().spines['right'].set_color('black')
plt.gca().spines['left'].set_color('black')
plt.gca().spines['bottom'].set_color('black')
# Add labels and title
plt.grid(False)
plt.ylabel('States',fontsize=20)
# ax.spines['bottom'].set_color('black')  # Bottom x-axis
# ax.spines['left'].set_color('black')    # Left y-axis
# ax.spines['top'].set_color('black')     # Top axis (optional)
# ax.spines['right'].set_color('black')
# plt.gcf().set_axis_off()
plt.rcParams['axes.facecolor'] = 'none'
# plt.box()
# plt.xlim(0,0.06)
# plt.ylim()
plt.yticks(fontsize=12)
plt.xticks(fontsize=12)
plt.xlabel('Infection-based Healthcare Access Weights',fontsize=20)
plt.title('Infection-based Healthcare Access Weights',fontsize=20)
plt.gcf().set_facecolor('none')
# plt.legend(loc='center left', bbox_to_anchor=(1.0, 0.5),fontsize=10)
plt.grid(axis='y', linestyle='--', alpha=0.5,color='black')  # Customize style and transparency

# Show the plot
plt.show()
print(ordered_states['DC'])

access={}
i=0
for s in states:
    access[s]=d_s[i]
    i+=1
print(sum(access[s] for s in states))
print(access)

## Model Validation

In [ ]:
actual_beta = {}
for s in states:
    for t in range(2, 13):
        actual_beta[t, s] = beta[t, s]
T = 12
# model0.reset()
Pi={}
Pi[0,1]=0
Pi[0,2]=0
Pi[0,3]=0
states=states2
wastage_MANE=0.19
wastage_S=0.08
wastage_SW=0.075
wastage_W=0
wastage_MW=0
wastage=0.00
pfizer_sub=[]
moderna_sub=[]
johnson_sub=[]
for t in range(0,21):
    pfizer_sub.append(0)
    moderna_sub.append(0)
    johnson_sub.append(0)
    pfizer_sub[t]+=Pf_S[t]
    moderna_sub[t]+=Mo_S[t]
    johnson_sub[t]+=Jo_S[t]
    pfizer_sub[t]+=Pf_M[t]
    moderna_sub[t]+=Mo_M[t]
    johnson_sub[t]+=Jo_M[t]
    pfizer_sub[t]+=Pf_SW[t]
    moderna_sub[t]+=Mo_SW[t]
    johnson_sub[t]+=Jo_SW[t]
    pfizer_sub[t]+=Pf_MW[t]
    moderna_sub[t]+=Mo_MW[t]
    johnson_sub[t]+=Jo_MW[t]
    pfizer_sub[t]+=Pf_N[t]
    moderna_sub[t]+=Mo_N[t]
    johnson_sub[t]+=Jo_N[t]
    pfizer_sub[t]+=Pf_W[t]
    moderna_sub[t]+=Mo_W[t]
    johnson_sub[t]+=Jo_W[t]
for t in range(1,T-1):
    t1=2*t-2
    t2=2*t-1
    Pi[t,1]=1*(pfizer_sub[2*t-2]+pfizer_sub[2*t-1])
    Pi[t,2]=moderna_sub[2*t-2]+moderna_sub[2*t-1]
    Pi[t,3]=johnson_sub[2*t-2]+johnson_sub[2*t-1]
Pi[10,1]+=1*(pfizer_sub[19])
Pi[10,2]+=moderna_sub[19]
Pi[10,3]+=johnson_sub[19]
# for t in range(0,)
states200=states
T1=2
T2=12


tau={}
Scale = 10000
for s in states:
    i=0
    tau[s]=0.01*(popplus12_s[s])/Scale
    i+=1
T1=2
T2=T
M = 100000000000/Scale
supply=1000000/Scale
cost_param=pd.DataFrame({"w":[1,1,1,1],
            "g":[1,1,1,1],
            "c":[20,20,15,5]})
B=[750000,1000000,1200000]
Demand={}
for t in range(2,T):
    for s in states:
        Demand[t,s]=demand[t,s]*popplus12_s[s]
suppliers = createList(1, 3)
mu=14/75/365/10000
gamma = 1
gamma1=1
sol_time={}
sol_time0={}
Vac_avl={}
for s in states:
    Vac_avl[1,s]=0
dic={}
for t in time:
    for s in states:
        dic[t,s]=None
results1=pd.DataFrame.from_dict(dic, orient='index')
dic1={}
for t in time:
    for s in states:
        for c in counties2[s]:
            dic1[t,s]=None
results2=pd.DataFrame.from_dict(dic1, orient='index')

global lower_bound
effec=0.2
Inventory={}
for s in states:
    for c in counties2[s]:
        Inventory[0,s,c]=0
model0 = gp.Model()
lb={}
for t in range(T1-1,T2):
    lb[t]= model0.addVar( lb=0, vtype=GRB.CONTINUOUS)
S={}
V={}
I={}
U_I={}
U_D={}
Vac={}
inf={}
Results={}
reinf={}
for s in states:
    for t in range(T1,T2+2):
        S[t,s] = model0.addVar( lb=0, vtype=GRB.CONTINUOUS)
        V[t,s] = model0.addVar( lb=0,  vtype=GRB.CONTINUOUS)
        I[t,s] = model0.addVar( lb=0, vtype=GRB.CONTINUOUS)
        reinf[t,s] = model0.addVar( lb=0, vtype=GRB.CONTINUOUS)
    for t in range(T1-1,T2):
        if (T2-1)>=t>=2:
            Vac[t,s] = model0.addVar(lb=(state_vaccine[t-1,s]-state_vaccine[t-2,s])/10000, ub=(state_vaccine[t-1,s]-state_vaccine[t-2,s])/10000,vtype=GRB.CONTINUOUS)
        if t==1:
            Vac[t,s]=model0.addVar(lb=0, ub=0, vtype=GRB.CONTINUOUS)
        if t==T2:
            Vac[t,s]=model0.addVar(lb=0, ub=0, vtype=GRB.CONTINUOUS)
        U_I[t,s] = model0.addVar(vtype=GRB.BINARY)
        U_D[t,s] = model0.addVar(vtype=GRB.BINARY)
cost={}
for s in states:
    cost[s] = model0.addVar(lb=0, vtype=GRB.CONTINUOUS)
Gini_Coef = {}
G1 = {}
W1 = {}
W2 = {}
G2 = {}
# Y = {}
X_of = {}
Vac0 = {}
capacity=10000/Scale
Vac_all={}
G1_supply={}
lb0={}
loc_cap={}
for s in states:
    Gini_Coef[s] = model0.addVar( lb=0, ub=1, vtype=GRB.CONTINUOUS)
    for t in range(T1-2,T2):
        lb0[t,s]=model0.addVar(lb=0,vtype=GRB.CONTINUOUS)
        W1[t,s] = model0.addVar( lb=0, vtype=GRB.CONTINUOUS)
        for i in suppliers:
            G1[t,i,s] = model0.addVar( lb=0, ub=10000, vtype=GRB.CONTINUOUS)
x={(t): model0.addVar(lb=0,ub=1, vtype=GRB.CONTINUOUS)for t in range(1,T+1)}
W2 = {(t, s, c): model0.addVar(lb=0, vtype=GRB.CONTINUOUS) for t in range(T1-2,T2) for s in states for c in counties2[s]}
Vac1 = {(t, s): model0.addVar(lb=0,ub=popplus12_s[s]/10000, vtype=GRB.CONTINUOUS) for t in range(T1-1,T2) for s in states}
G2 = {(t, s, c): model0.addVar(lb=0, vtype=GRB.CONTINUOUS) for t in range(T1-1,T2) for s in states for c in counties2[s]}
# Y = {(t, s, c): model0.addVar(lb=0, ub=1, vtype=GRB.BINARY) for t in range(2,T+1) for s in states for c in counties2[s]}
X_o = {(t, s): model0.addVar(lb=0, vtype=GRB.BINARY)  for t in  range(T1-1,T2) for s in states}
Vac0 = {(t, s,c): model0.addVar(lb=0, ub=popplus12[s,c]/10000, vtype=GRB.CONTINUOUS) for t in range(T1-1,T2) for s in states for c in counties2[s]}
ub=model0.addVar(lb=0,ub=0.1)
y=model0.addVar(vtype=GRB.BINARY)

############################################################################Constraints
for s in states:
    model0.addConstr(S[T1, s] == S0101[0,s]/Scale)
    model0.addConstr(V[T1, s] == 0)
    model0.addConstr(I[T1, s] == I0101[0,s]/Scale)
    # model0.addConstr(Vac[1, s] == 0)
    for t in range(T1,T2+1):

        model0.addConstr(S[t+1,s] == reinf[t,s]+(1-mu)*S[t,s] +mu - Scale*beta[t,s]*I[t,s]*S[t,s]-(1-coefs[s][0])*Vac[t-gamma1,s])
        model0.addConstr(V[t+1,s] == V[t,s] +(1-coefs[s][0])*Vac[t-gamma1,s] -effec*Scale*beta[t,s]*V[t,s]*I[t,s] -mu*V[t,s])
        model0.addConstr(I[t+1,s] == I[t,s] + Scale*beta[t,s]*S[t,s] *I[t,s]+effec*Scale*beta[t,s]*V[t,s]*I[t,s]-gamma*I[t,s]-mu*I[t,s])
        if t>=9:
            model0.addConstr(reinf[t,s]==(reinf_rate)*(10000*beta[t-6,s]*S[t-7,s] *I[t-7,s]+effec*10000*beta[t-7,s]*V[t-7,s]*I[t-7,s]-mu*I[t-7,s]))
        else:
            model0.addConstr(reinf[t,s]==(reinf_rate)*1.5*(states_cases0[s][t+22-6]-states_cases0[s][t+21-6])/10000)
for s in states:
    for c in counties2[s]:
        model0.addConstr(sum(Vac0[t,s,c] for t in range(1,T))<=popplus12[s,c])

M1=10000       #################################################################Supply Chain Part
for t in range(T1-1,T2):
    for s in states:
        model0.addConstr(X_o[t,s]<= U_I[t,s]+U_D[t,s])
for t in range(T1-1,T2):
    for s in states:
        model0.addConstr(Vac[t,s]==(1-wastage)*(sum(Vac0[t,s,c] for c in counties2[s]))+(1-wastage)*X_o[t,s]*capacity)
for s in states:
    for t in range(T1-1,T2-1):
        # model.addConstr(gp.quicksum(G3[t,h_s,j] for h_s in statehubs)*1/21==gp.quicksum(G4[t,j,l]for l in localcenters[j]))
        model0.addConstr(sum(G1[t,i,s] for i in suppliers)+W1[t-1,s] == sum(G2[t,s,c] for c in counties2[s] )+X_o[t,s]*capacity+W1[t,s])
    model0.addConstr(W1[T1-1,s]==0)
for s in states:
    for c in counties2[s]:
        model0.addConstr(sum(Vac0[t,s,c] for t in range(T1-1,T2))<=popplus12[s,c]/10000)
for t in range(T1-1,T2):
    for s in states:
        for c in counties2[s]:
            # model0.addConstr(Vac0[t,s,c]<=county_cap[c])
            # model0.addConstr(lb0[t,s]<=Vac0[t,s,c])
            model0.addConstr (G2[t,s,c]  + W2[t-1,s,c]== Vac0[t,s,c]+ W2[t,s,c] )
            model0.addConstr(W2[T1-1,s,c]==0)
#########################################################################################################################Objective Function
model0.setObjective(-0*gp.quicksum(gp.quicksum((Scale*beta[t,s] * S[t, s] * I[t, s] + effec*Scale*beta[t,s] *  V[t, s] * I[t, s]) for s in states)for t in range(T1, T2+1)), GRB.MAXIMIZE)
model0.Params.TIME_LIMIT = 5*3600
model0.optimize()
# model0.setParam(GRB.Param.Heuristics, 0)
inf={(t,s): beta[t,s] * S[t, s].x * I[t, s].x + effec*beta[t,s] *  V[t, s].x * I[t, s].x for t in range(T1, T2) for s in states  }
U_I={(t, s): U_I[t, s].x for t in range(T1-1,T2-1+1)  for s in states  }
Vac_i={(t-1, s): Vac[t-1, s].x for t in range(T1,T2+1)  for s in states }
vac_r={}
for s in states:
    for t in range(T1,T2):
        if sum(Vac[t-1, s].x for s in states)==0:
            vac_r[t-1,s]=0
        else:
            vac_r[t-1,s]=(Vac[t-1, s]).x/sum(Vac[t-1, s].x for s in states)
S1={(t, s): S[t, s].x for t in range(T1,T2+1)  for s in states }
I1={(t, s): I[t, s].x for t in range(T1,T2+1)  for s in states }
V1={(t, s): V[t, s].x for t in range(T1,T2+1) for s in states }
Vac_df=pd.DataFrame({"States":states})
sol_time[T1]=model0.Runtime
# model0.reset()
U_I0={}
for s in states:
    for t in range(T1,T2):
        U_I0[t,s]=U_I[t,s]
        for i in range(T1,T2-2):
            if U_I[i,s]==1:
                U_I0[i+1,s]=0
                U_I0[i+2,s]=0

if model0.Status == GRB.OPTIMAL:
    for s in states:
        for t in  range(T1-1,T2+1-1):
            for c in counties2[s]:
                Vac_all[t,s,c] = Vac0[t,s,c].x
elif model0.Status == GRB.INFEASIBLE:
    for s in states:
        for t in  range(T1-1,T2+1-1):
            for c in counties2[s]:
                Vac_all[t,s,c] = "nan"
if model0.Status == GRB.OPTIMAL:
    for s in states:
        for t in  range(T1-1,T2+1-1):
            Vac_avl[t,s] = sum(Vac0[t,s,c].x for c in counties2[s])
elif model0.Status == GRB.INFEASIBLE:
    for s in states:
        for t in  range(T1-1,T2+1-1):
            Vac_avl[t,s] = "nan"
if model0.Status == GRB.OPTIMAL:
    for s in states:
        for t in  range(T1-1,T2-1):
            X_of[t,s] = X_o[t,s].x
elif model0.Status == GRB.INFEASIBLE:
    for s in states:
        for t in  range(T1-1,T2-1):
           X_of[t,s] = "nan"
if model0.Status == GRB.OPTIMAL:
    cost[T1]= sum(sum(cost_param["w"][0]*W1[t,s].x for s in states )+sum(cost_param["w"][1]*W2[t,s,c].x for s in states for c in counties2[s])+sum(cost_param["g"][1]*G2[t,s,c].x for s in states for c in counties2[s] )+sum(cost_param["c"][1]*G1[t,1,s].x+cost_param["c"][2]*G1[t,2,s].x for s in states )+sum(cost_param["c"][0]*X_o[t,s].x for s in states)for t in  range(T1-1,T2-1))
elif model0.Status == GRB.INFEASIBLE:
    cost[T1]= "nan"
for s in states:
    for t in range(T1-1,T2-1):
        for c in counties2[s]:
            if model0.Status == GRB.OPTIMAL:
                Inventory[t,s,c]= W2[t,s,c].x
            elif model0.Status == GRB.INFEASIBLE:
                Inventory[t,s,c]="nan"
#Vaccination
for t in range(T1-1,T2-1):
    if model0.Status == GRB.OPTIMAL:
        results1[f"t/s"]=Vac[t,s]
    elif model0.Status == GRB.INFEASIBLE:
        results1[f"t/s"]="nan"
#Supply from suppliers
for t in range(T1-1,T2-1):
    for i in suppliers:
        if model0.Status == GRB.OPTIMAL:
            G1_supply[t,i,s]= G1[t,i,s].x
        elif model0.Status == GRB.INFEASIBLE:
            G1_supply[t,i,s]="nan"
#Vaccination
for t in range(T1-1,T2-1):
    if model0.Status == GRB.OPTIMAL:
        results1[f"t/s"]=Vac[t,s]
    elif model0.Status == GRB.INFEASIBLE:
        results1[f"t/s"]="nan"
sol_time0[T1]=model0.Runtime
actual_values={}
optimized_values={}
for s in states:
    actual_values[s]=[]
    optimized_values[s]=[]
    for t in range(2,13):
        optimized_values[s].append((Scale*beta[t,s] * S[t, s].x * I[t, s].x + effec*Scale*beta[t,s] *  V[t, s].x * I[t, s].x)*10000/100000)
        actual_values[s].append(1.5*(states_cases0[s][t+22]-states_cases0[s][t+21])/100000)
        print(s,(optimized_values[s][t-2]-actual_values[s][t-2])/actual_values[s][t-2]*100)

print(sum(Scale*beta[t,s] * S[t, s].x * I[t, s].x + effec*Scale*beta[t,s] *  V[t, s].x * I[t, s].x for s in states for t in range(T1, 13))*10000,sum(1.5*(states_cases0[s][t+22]-states_cases0[s][t+21]) for s in states for t in range(2,13)))

# Simulated data for demonstration
# Plot 1: Total Actual vs. Optimized Results
fig, ax = plt.subplots(figsize=(8, 8))

actual_time = []
optimized_time = []

for t in range(11):
    actual_time.append(sum(actual_values[s][t] for s in states))
    optimized_time.append(sum(optimized_values[s][t] for s in states))

ax.plot(actual_time, optimized_time, linewidth=2.0, label="Total Results")
ax.set_facecolor("white")
ax.set(xlim=(1000000/100000, 5000000/100000),
       ylim=(1000000/100000, 5000000/100000))
ax.set_title('Comparison of Total Actual vs. Optimized Results', fontsize=20)
ax.set_xlabel('Actual Results (million cases)', fontsize=20)
ax.set_ylabel('Optimized Results (million cases)', fontsize=20)

# Customize spines
for spine in ax.spines.values():
    spine.set_color('black')
plt.tick_params(labelsize=14)
# Legend outside
# ax.legend(loc='upper left', bbox_to_anchor=(1.05, 1), fontsize=10)
# plt.tight_layout(rect=[0, 0, 0.85, 1])  # Adjust layout for legend space
plt.grid(True)
plt.show()

# Plot 2: Individual State Comparison
fig, ax = plt.subplots(figsize=(9, 8))

for s in states:
    ax.plot(actual_values[s], optimized_values[s], linewidth=1.5, label=s)

ax.set_facecolor("white")
ax.set(xlim=(0, 500000/100000), xlabel="Actual Results (million cases)",
       ylim=(0, 500000/100000), ylabel="Optimized Results (million cases)")
ax.set_title('Comparison of State Actual vs. Optimized Results', fontsize=20)
ax.set_xlabel('Actual Results (million cases)', fontsize=20)
ax.set_ylabel('Optimized Results (million cases)', fontsize=20)

# Customize spines
for spine in ax.spines.values():
    spine.set_color('black')
plt.tick_params(labelsize=14)
# Legend outside
# ax.legend(loc='upper left', bbox_to_anchor=(1.0, 1), fontsize=8)
plt.legend(fontsize=12,ncol=2,loc="upper left", bbox_to_anchor=(1, 1))
plt.tight_layout(rect=[0, 0, 1, 1])  # Adjust layout for legend space
plt.grid(True)
plt.show()

# Sample data: Two sets of related data (e.g., before and after measurements)
before = {}
after = {}
for t in range(2,12):
    before[t]=[]
    after[t]=[]
    for s in states:
        before[t].append(actual_values[s][t-2])
        after[t].append(optimized_values[s][t-2])
# Perform paired t-test
for t in range(2,12):
    t_stat, p_value = stats.ttest_rel(before[t], after[t])
    if p_value<0.025:
        print(f"T-statistic: {t_stat}",t)
        print(f"P-value: {p_value}")
for s in states:
    t_stat, p_value = stats.ttest_rel(actual_values[s], optimized_values[s],alternative='two-sided')
    if p_value<0.025:
        print(f"T-statistic: {t_stat}",s)
        print(f"P-value: {p_value}")



## Model Validation 2

In [ ]:
historical_beta={}
for s in states:
    historical_beta[s]={}
    for t in range(5,24):
        historical_beta[s][dates1[t]]=np.log(beta01[t,s]*popplus12_s[s]*10)
df=pd.DataFrame(historical_beta)
best_m_value={}
p=0
# m_mode=[6,11,5,11,6,5,5,5,6,5,6,11,3,4,5,3,3,7,5,5,6,5,6,5,4,3,4,5,6,6,6,4,6,4,6,4,5,5,6,6,4,5,5,6,5,4,6,6,2,6,2]
m_mode=[10,15,15,16,16,8,12,16,8,8,16,16,8,6,8,4,6,8,8,10,8,16,8,8,16,8,8,8,8,16,16,8,16,10,8,8,8,10,10,8,16,16,16,16,8,10,10,10,16,16,8]
for s in states:
    best_m_value[s]=m_mode[p]
    p+=1


m=12
suggested_m = {
    "ME": 5,  # Includes significant lags and some residual patterns
    "NH": 6,  # Captures early oscillations and significant correlations
    "NJ": 8,  # Includes significant and oscillatory lags
    "NY": 8,  # Captures meaningful lags and persistent negative correlations
    "PA": 6,  # Balances capturing structure with simplicity
    "CT": 7,  # Includes extended significant lags
    "RI": 6,  # Captures early dependencies
    "VT": 5,  # Focuses on sharp declines and initial correlations
    "MA": 7,  # Captures significant lags and minor oscillations
    "MD": 6,  # Balances capturing dependencies without overfitting
    "DC": 6,  # Includes significant patterns without unnecessary lags
    "DE": 5   # Accounts for dependencies and early oscillations
}
suggested_m = {
    'ME': 2,
    'NH': 2,
    'NJ': 2,
    'NY': 2,
    'PA': 2,
    'CT': 2,
    'RI': 1,
    'VT': 6,
    'MA': 2,
    'MD': 2,
    'DC': 1,
    'DE': 2
}


def SARIMA(s):
    date_range = range(5,24)
    time_series = pd.Series(list(df[s]), index=date_range)

    # Plot the time series data
    time_series.plot(title="Time Series Data")
    plt.show()
    m=2
    if s in ['DC']:
        m=5
    if s in ['NY']:
        m=6
    if s in ['DE',  'NH', 'VT']:
        m=10
    if s in ['NJ', 'ME', 'RI']:
        m=11
    # m=6
    # Step 1: Automatically find the best SARIMA model
    model = auto_arima(time_series, seasonal=True, m=  m,  # 'm=12' for monthly seasonality
                       trace=True,  # Show the iterations
                       error_action='ignore',  # Ignore certain warnings
                       suppress_warnings=True,  # Suppress warnings
                       stepwise=True)  # Stepwise fit for fast computation

    # Print the best model summary
    print(model.summary())

    # Step 2: Fit the model
    sarima_model = SARIMAX(time_series,
                           order=model.order,
                           seasonal_order=model.seasonal_order)
    sarima_result = sarima_model.fit()

    # Print the fitted model summary
    print(sarima_result.summary())
    sigma[s]=sarima_result.scale
    # Step 3: Forecast for the next 11 periods
    forecast = sarima_result.get_forecast(steps=20)
    forecast_values[s] = forecast.predicted_mean
    confidence_intervals[s] = forecast.conf_int()

    # forecast_range[s] = max(predicted_mean) - min(predicted_mean)
    # Plot the forecast
    plt.figure(figsize=(10, 6))
    plt.plot(time_series, label='Observed')
    plt.plot(forecast_values[s], label='Forecast', color='red')
    plt.fill_between(forecast_values[s].index,
                     confidence_intervals[s].iloc[:, 0],
                     confidence_intervals[s].iloc[:, 1], color='pink', alpha=0.3)
    plt.title(f'Forecast with SARIMA Model with seasonality {m} in state {s}')
    plt.legend()
    plt.savefig(f'state{s}-SARIMA-final-{m}.png')
    plt.show()
    plot_acf(time_series, lags=18)
    plt.title(f'ACF in state {s}')
    plt.savefig(f'state{s}-ACF-{m}.png')
    plt.show()
    # Print the forecasted values
    print(forecast_values[s])
# Simulated example: Replace this with real data loading for each state.

    # Plot PACF for each state with adjusted lags
    fig, ax = plt.subplots(figsize=(6, 4))
    plot_pacf(time_series, ax=ax, method='ywmle', lags=8, zero=False)  # Adjust lags as needed
    ax.set_title(f"PACF in state {s}")
    plt.show()

plt.show()

sigma = {}
forecast_values = {}
forecast_range = {}
confidence_intervals = {}
for s in states:
    SARIMA(s)

lower_beta = {}
upper_beta = {}

for s in states:
    for t in range(3, 13):
        print(s, t,  f'the predicted mean is {round(np.exp(forecast_values[s][t + 21])*10**6 / popplus12_s[s] / 10,2)}',
                  f'the actual value is {round(actual_beta[t, s]*10**6,2)}',f'the confidence interval is',
                  round(np.exp(confidence_intervals[s]['lower y'][t + 21]) *10**6/ popplus12_s[s] / 10,2),'to',round(10**6*np.exp(confidence_intervals[s]['upper y'][t + 21]) / popplus12_s[s] / 10,2))


T1=2
beta={}
ranked_beta={}
for s in states:

    ranked_beta[s]=[]
    for t in range(6,23):
        ranked_beta[s].append(beta01[t,s])
for t in range(2,14):
    for s in states:
        if t==2:
            beta[t,s]=beta01[23,s]
        else:
            for s in states:
                for t in range(3,13):
                    beta[t,s]=(np.exp((1*confidence_intervals[s]['lower y'][t+21]+1*confidence_intervals[s]['upper y'][t+21])/2.00)/popplus12_s[s]/10)
    T1=t
    T2=t+1
est_demand={}
Scale=10000

for s in states:
    for t in range(T1-1,T2+1):
        vac_hes[t,s]=0
        # est_demand[t,s]=1/(T2-T1)*(1-hesit[t,s])*popplus12_s[s]/Scale

local_centers={}
for s in states:
    for c in counties2[s]:
        local_centers[s,c]=range(int(np.floor(3*popplus12[s,c]/10000)))
tau={}
T2=12
Scale = 10000
for s in states:
    i=0
    tau[s]=0.01*(popplus12_s[s])/Scale
    i+=1
T1=2
T2=11
M = 100000000000/Scale
supply=1000000/Scale
cost_param=pd.DataFrame({"w":[1,1,1,1],
            "g":[1,1,1,1],
            "c":[20,20,15,5]})
B=[750000,1000000,1200000]
Demand={}
for t in range(2,T):
    for s in states:
        Demand[t,s]=demand[t,s]*popplus12_s[s]
suppliers = createList(1, 3)

mu=14/75/365/10000
gamma = 1.0
gamma1=1
sol_time={}
sol_time0={}
Vac_avl={}
for s in states:
    Vac_avl[1,s]=0
dic={}
for t in time:
    for s in states:
        dic[t,s]=None
results1=pd.DataFrame.from_dict(dic, orient='index')
dic1={}
for t in time:
    for s in states:
        for c in counties2[s]:
            dic1[t,s]=None
results2=pd.DataFrame.from_dict(dic1, orient='index')
wastage=0

global lower_bound
effec=0.2
Inventory={}
for s in states:
    for c in counties2[s]:
        Inventory[0,s,c]=0
model0 = gp.Model()
lb={}
for t in range(T1-1,T2):
    lb[t]= model0.addVar( lb=0, vtype=GRB.CONTINUOUS)
S={}
V={}
I={}
U_I={}
U_D={}
Vac={}
inf={}
# beta={}
Results={}
reinf={}
Gini_Coef = {}
G1 = {}
W1 = {}
W2 = {}
cost={}
G2 = {}
# Y = {}
X_of = {}
Vac0 = {}
capacity=10000/Scale
Vac_all={}
G1_supply={}
lb0={}
loc_cap={}
for s in states:
    for t in range(T1,T2+2):
        S[t,s] = model0.addVar( lb=0, vtype=GRB.CONTINUOUS)
        V[t,s] = model0.addVar( lb=0,  vtype=GRB.CONTINUOUS)
        I[t,s] = model0.addVar( lb=0, vtype=GRB.CONTINUOUS)
        # beta[t,s] = model0.addVar( ub=np.exp(confidence_intervals[s]['lower y'][t+22])/popplus12_s[s]/10,lb=np.exp(confidence_intervals[s]['lower y'][t+22])/popplus12_s[s]/10, vtype=GRB.CONTINUOUS)
        reinf[t,s] = model0.addVar( lb=0, vtype=GRB.CONTINUOUS)
    for t in range(T1-1,T2+1):
        if (T2-1)>=t>=2:
            Vac[t,s] = model0.addVar(lb=(state_vaccine[t-1,s]-state_vaccine[t-2,s])/10000, ub=(state_vaccine[t-1,s]-state_vaccine[t-2,s])/10000,vtype=GRB.CONTINUOUS)
        if t==1:
            Vac[t,s]=model0.addVar(lb=0, ub=0, vtype=GRB.CONTINUOUS)
        if t==T2:
            Vac[t,s]=model0.addVar(lb=0, ub=0, vtype=GRB.CONTINUOUS)
        U_I[t,s] = model0.addVar(vtype=GRB.BINARY)
        U_D[t,s] = model0.addVar(vtype=GRB.BINARY)
for s in states:
    cost[s] = model0.addVar(lb=0, vtype=GRB.CONTINUOUS)
for s in states:
    Gini_Coef[s] = model0.addVar( lb=0, ub=1, vtype=GRB.CONTINUOUS)
    for t in range(T1-2,T2):
        lb0[t,s]=model0.addVar(lb=0,vtype=GRB.CONTINUOUS)
        W1[t,s] = model0.addVar( lb=0, vtype=GRB.CONTINUOUS)
        for i in suppliers:
            G1[t,i,s] = model0.addVar( lb=0, ub=10000, vtype=GRB.CONTINUOUS)
x={(t): model0.addVar(lb=0,ub=1, vtype=GRB.CONTINUOUS)for t in range(1,T+1)}
W2 = {(t, s, c): model0.addVar(lb=0, vtype=GRB.CONTINUOUS) for t in range(T1-2,T2) for s in states for c in counties2[s]}
Vac1 = {(t, s): model0.addVar(lb=0,ub=popplus12_s[s]/10000, vtype=GRB.CONTINUOUS) for t in range(T1-1,T2) for s in states}
G2 = {(t, s, c): model0.addVar(lb=0, vtype=GRB.CONTINUOUS) for t in range(T1-1,T2) for s in states for c in counties2[s]}
# Y = {(t, s, c): model0.addVar(lb=0, ub=1, vtype=GRB.BINARY) for t in range(2,T+1) for s in states for c in counties2[s]}
X_o = {(t, s): model0.addVar(lb=0, vtype=GRB.BINARY)  for t in  range(T1-1,T2) for s in states}
Vac0 = {(t, s,c): model0.addVar(lb=0, ub=popplus12[s,c]/10000, vtype=GRB.CONTINUOUS) for t in range(T1-1,T2) for s in states for c in counties2[s]}
ub=model0.addVar(lb=0,ub=0.1)
y=model0.addVar(vtype=GRB.BINARY)

############################################################################Constraints
for s in states:
    model0.addConstr(S[2, s] == S0101[0,s]/Scale)
    model0.addConstr(V[2, s] == 0)
    model0.addConstr(I[2, s] == I0101[0,s]/Scale)
    model0.addConstr(Vac[1, s] == 0)
    for t in range(2,T2+1):
        model0.addConstr(S[t+1,s] == (1-mu)*S[t,s] +mu - Scale*beta[t,s]*I[t,s]*S[t,s]-(1-coefs[s][0])*Vac[t-gamma1,s]+reinf[t,s],name=f'constraint 2a {j,t}')
        model0.addConstr(V[t+1,s] == V[t,s] +(1-coefs[s][0])*Vac[t-gamma1,s] -effec*Scale*beta[t,s]*V[t,s]*I[t,s] -mu*V[t,s],name=f'constraint 2b {j,t}')
        model0.addConstr(I[t+1,s] == I[t,s] + Scale*beta[t,s]*S[t,s] *I[t,s]+effec*Scale*beta[t,s]*V[t,s]*I[t,s]-gamma*I[t,s]-mu*I[t,s],name=f'constraint 2c {j,t}')
        if t>=7:
            model0.addConstr(reinf[t,s]==0.01*(10000*beta[t-5,s]*S[t-5,s] *I[t-5,s]+effec*10000*beta[t-5,s]*V[t-5,s]*I[t-5,s]-mu*I[t-5,s]))
        else:
            model0.addConstr(reinf[t,s]==0.015*(states_cases0[s][t+22-4]-states_cases0[s][t+21-4])/10000)
M1=10000       #################################################################Supply Chain Part

for t in range(T1-1,T2):
    for s in states:
        model0.addConstr(X_o[t,s]<= U_I[t,s]+U_D[t,s])
# for s in states:
#     model0.addConstr(sum(X_o[t,s] for t in range(T1-1,T2-1))<=4)
for t in range(T1-1,T2-3):
    for s in states:
        for k in range(0,3):
            model0.addConstr(X_o[t,s] <= (1-X_o[t+k,s]))
for t in range(T1-1,T2):
    for s in states:
        model0.addConstr(Vac[t,s]==(1-wastage)*(sum(Vac0[t,s,c] for c in counties2[s]))+(1-wastage)*X_o[t,s]*capacity)
for s in states:
    for t in range(T1-1,T2-1):
        # model.addConstr(gp.quicksum(G3[t,h_s,j] for h_s in statehubs)*1/21==gp.quicksum(G4[t,j,l]for l in localcenters[j]))
        model0.addConstr(sum(G1[t,i,s] for i in suppliers)+W1[t-1,s] == sum(G2[t,s,c] for c in counties2[s] )+X_o[t,s]*capacity+W1[t,s])
    model0.addConstr(W1[T1-1,s]==0)
for s in states:
    for c in counties2[s]:
        model0.addConstr(sum(Vac0[t,s,c] for t in range(T1-1,T2))<=popplus12[s,c]/10000)
for t in range(T1-1,T2):
    for s in states:
        for c in counties2[s]:
            # model0.addConstr(Vac0[t,s,c]<=county_cap[c])
            # model0.addConstr(lb0[t,s]<=Vac0[t,s,c])
            model0.addConstr (G2[t,s,c]  + W2[t-1,s,c]== Vac0[t,s,c]+ W2[t,s,c] )
            model0.addConstr(W2[T1-1,s,c]==0)
model0.setObjective(-0*gp.quicksum(gp.quicksum((Scale*beta[t,s] * S[t, s] * I[t, s] + effec*Scale*beta[t,s] *  V[t, s] * I[t, s]) for s in states)for t in range(T1, T2+1)), GRB.MAXIMIZE)
# Set optimization parameters
model0.Params.TIME_LIMIT = 5*3600
model0.optimize()
inf={(t,s): beta[t,s] * S[t, s].x * I[t, s].x + effec*beta[t,s] *  V[t, s].x * I[t, s].x for t in range(T1, T2) for s in states}
U_I={(t, s): U_I[t, s].x for t in range(T1-1,T2-1+1)  for s in states  }
Vac_i={(t-1, s): Vac[t-1, s].x for t in range(T1,T2+1)  for s in states }
vac_r={}
for s in states:
    for t in range(T1,T2):
        if sum(Vac[t-1, s].x for s in states)==0:
            vac_r[t-1,s]=0
        else:
            vac_r[t-1,s]=(Vac[t-1, s]).x/sum(Vac[t-1, s].x for s in states)
# for s in states:
# vac_r[2,s]=0
S1={(t, s): S[t, s].x for t in range(T1,T2+1)  for s in states }
I1={(t, s): I[t, s].x for t in range(T1,T2+1)  for s in states }
V1={(t, s): V[t, s].x for t in range(T1,T2+1) for s in states }
Vac_df=pd.DataFrame({"States":states})
# for s in states:
#     Vac_df[s]=list(Vac[s,t])
# model0.computeIIS()
# model0.write("model.ilp")
sol_time[T1]=model0.Runtime
# model0.reset()
U_I0={}
for s in states:
    for t in range(T1,T2):
        U_I0[t,s]=U_I[t,s]
        for i in range(T1,T2-2):
            if U_I[i,s]==1:
                U_I0[i+1,s]=0
                U_I0[i+2,s]=0

if model0.Status == GRB.OPTIMAL:
    for s in states:
        for t in  range(T1-1,T2+1-1):
            for c in counties2[s]:
                Vac_all[t,s,c] = Vac0[t,s,c].x
elif model0.Status == GRB.INFEASIBLE:
    for s in states:
        for t in  range(T1-1,T2+1-1):
            for c in counties2[s]:
                Vac_all[t,s,c] = "nan"
if model0.Status == GRB.OPTIMAL:
    for s in states:
        for t in  range(T1-1,T2+1-1):
            Vac_avl[t,s] = sum(Vac0[t,s,c].x for c in counties2[s])
elif model0.Status == GRB.INFEASIBLE:
    for s in states:
        for t in  range(T1-1,T2+1-1):
            Vac_avl[t,s] = "nan"
if model0.Status == GRB.OPTIMAL:
    for s in states:
        for t in  range(T1-1,T2-1):
            X_of[t,s] = X_o[t,s].x
elif model0.Status == GRB.INFEASIBLE:
    for s in states:
        for t in  range(T1-1,T2-1):
           X_of[t,s] = "nan"
if model0.Status == GRB.OPTIMAL:
    cost[T1]= sum(sum(cost_param["w"][0]*W1[t,s].x for s in states )+sum(cost_param["w"][1]*W2[t,s,c].x for s in states for c in counties2[s])+sum(cost_param["g"][1]*G2[t,s,c].x for s in states for c in counties2[s] )+sum(cost_param["c"][1]*G1[t,1,s].x+cost_param["c"][2]*G1[t,2,s].x for s in states )+sum(cost_param["c"][0]*X_o[t,s].x for s in states)for t in  range(T1-1,T2-1))
elif model0.Status == GRB.INFEASIBLE:
    cost[T1]= "nan"
for s in states:
    for t in range(T1-1,T2-1):
        for c in counties2[s]:
            if model0.Status == GRB.OPTIMAL:
                Inventory[t,s,c]= W2[t,s,c].x
            elif model0.Status == GRB.INFEASIBLE:
                Inventory[t,s,c]="nan"
#Vaccination
for t in range(T1-1,T2-1):
    if model0.Status == GRB.OPTIMAL:
        results1[f"t/s"]=Vac[t,s]
    elif model0.Status == GRB.INFEASIBLE:
        results1[f"t/s"]="nan"
#Supply from suppliers
for t in range(T1-1,T2-1):
    for i in suppliers:
        if model0.Status == GRB.OPTIMAL:
            G1_supply[t,i,s]= G1[t,i,s].x
        elif model0.Status == GRB.INFEASIBLE:
            G1_supply[t,i,s]="nan"
#Vaccination
for t in range(T1-1,T2-1):
    if model0.Status == GRB.OPTIMAL:
        results1[f"t/s"]=Vac[t,s]
    elif model0.Status == GRB.INFEASIBLE:
        results1[f"t/s"]="nan"

sol_time0[T1]=model0.Runtime
actual_values={}
optimized_values={}
for s in states:
    actual_values[s]=[]
    optimized_values[s]=[]
    for t in range(2,13):
        optimized_values[s].append((Scale*beta[t,s] * S[t, s].x * I[t, s].x + effec*Scale*beta[t,s] *  V[t, s].x * I[t, s].x)*10000)
        actual_values[s].append(1.5*(states_cases0[s][t+22]-states_cases0[s][t+21]))

before = {}
after = {}
for t in range(2,12):
    before[t]=[]
    after[t]=[]
    for s in states:
        before[t].append(actual_values[s][t-2])
        after[t].append(optimized_values[s][t-2])
# Perform paired t-test
for t in range(2,12):
    t_stat, p_value = stats.ttest_rel(before[t], after[t])
    if p_value<0.025:
        print(f"T-statistic: {t_stat}",t)
        print(f"P-value: {p_value}")
badacf=[]
for s in states:
    t_stat, p_value = stats.ttest_rel(actual_values[s], optimized_values[s],alternative='two-sided')
    if p_value<0.025:
        print(f"T-statistic: {t_stat}",s)
        print(f"P-value: {p_value}")
        badacf.append(s)


In [ ]:
#######Demand Prameters

T1=2
T=12
vac_hes={}
est_demand={}
Scale=10000
suppliers=[1,2,3]
states=states2
for s in states:
    for t in range(1,13):
        vac_hes[t,s]=0
        # est_demand[t,s]=1/13*(1-sum(hesit[c]for c in counties2[s])/len(counties2[s]))*popplus12_s[s]/Scale
        if s in West:
            est_demand[t,s]=1/9*(1-sum(hesit[c]for c in counties2[s])/len(counties2[s]))*popplus12_s[s]/Scale
        if (s in New_England):
            est_demand[t,s]=1/8*(1-sum(hesit[c]for c in counties2[s])/len(counties2[s]))*popplus12_s[s]/Scale
        if (s in Middle_Atlantic):
            est_demand[t,s]=1/10*(1-sum(hesit[c]for c in counties2[s])/len(counties2[s]))*popplus12_s[s]/Scale
        if s in Midwest:
            est_demand[t,s]=1/10*(1-sum(hesit[c]for c in counties2[s])/len(counties2[s]))*popplus12_s[s]/Scale
        if s in South:
            est_demand[t,s]=1/11*(1-sum(hesit[c]for c in counties2[s])/len(counties2[s]))*popplus12_s[s]/Scale
        if s in Southwest:
            est_demand[t,s]=1/10*(1-sum(hesit[c]for c in counties2[s])/len(counties2[s]))*popplus12_s[s]/Scale
        est_demand[t,s]=1*est_demand[t,s]
local_centers={}
for s in states:
    for c in counties2[s]:
        local_centers[s,c]=range(int(np.floor(3*popplus12[s,c]/10000)))

In [ ]:
#######Supply Prameters

Pi={}
Pi[0,1]=0
Pi[0,2]=0
Pi[0,3]=0
wastage=0.00
pfizer_sub=[]
moderna_sub=[]
johnson_sub=[]
for t in range(0,21):
    pfizer_sub.append(0)
    moderna_sub.append(0)
    johnson_sub.append(0)

    pfizer_sub[t]+=Pf_M[t]
    moderna_sub[t]+=Mo_M[t]
    johnson_sub[t]+=Jo_M[t]
    pfizer_sub[t]+=Pf_N[t]
    moderna_sub[t]+=Mo_N[t]
    johnson_sub[t]+=Jo_N[t]

    # pfizer_sub[t]+=Pf_S[t]
    # moderna_sub[t]+=Mo_S[t]
    # johnson_sub[t]+=Jo_S[t]
    #
    # pfizer_sub[t]+=Pf_SW[t]
    # moderna_sub[t]+=Mo_SW[t]
    # johnson_sub[t]+=Jo_SW[t]
    #
    # pfizer_sub[t]+=Pf_MW[t]
    # moderna_sub[t]+=Mo_MW[t]
    # johnson_sub[t]+=Jo_MW[t]
    #
    # pfizer_sub[t]+=Pf_W[t]
    # moderna_sub[t]+=Mo_W[t]
    # johnson_sub[t]+=Jo_W[t]
# # wastage=wastage_S
# # states.extend(South)
# pfizer_sub.extend(Pf_S)
# moderna_sub.extend(Mo_S)
# johnson_sub.extend(Jo_S)
# pfizer_sub.extend(Pf_M)
# moderna_sub.extend(Mo_M)
# johnson_sub.extend(Jo_M)
# pfizer_sub.extend(Pf_SW)
# moderna_sub.extend(Mo_SW)
# johnson_sub.extend(Jo_SW)
# pfizer_sub.extend(Pf_MW)
# moderna_sub.extend(Mo_MW)
# johnson_sub.extend(Jo_MW)
# pfizer_sub.extend(Pf_N)
# moderna_sub.extend(Mo_N)
# johnson_sub.extend(Jo_N)
wastage=0.0

states=[]

# states=states2
states.extend(Middle_Atlantic)
states.extend(New_England)
# pfizer_sub=Pf_MANE
# moderna_sub=Mo_MANE
# johnson_sub=Jo_MANE
# pfizer_sub.insert(0,0)
# moderna_sub.insert(0,0)
# johnson_sub.insert(0,0)
# states.extend(West)
# pfizer_sub=Pf_W
# moderna_sub=Mo_W
# johnson_sub=Jo_W

# states.extend(South)

# states.extend(Midwest)
# pfizer_sub=Pf_MW
# moderna_sub=Mo_MW
# johnson_sub=Jo_MW

# states.extend(Southwest)
# pfizer_sub=Pf_SW
# moderna_sub=Mo_SW
# johnson_sub=Jo_SW

states200=states


# for t in range(0,)


for t in range(1,T-1):
    t1=2*t-2
    t2=2*t-1
    Pi[t,1]=pfizer_sub[2*t-2]+pfizer_sub[2*t-1]
    Pi[t,2]=moderna_sub[2*t-2]+moderna_sub[2*t-1]
    Pi[t,3]=johnson_sub[2*t-2]+johnson_sub[2*t-1]
Pi[10,1]+=1*(pfizer_sub[19])
Pi[10,2]+=moderna_sub[19]
Pi[10,3]+=johnson_sub[19]
for t in range(1,T-1):
    for i in suppliers:
        Pi[t,i]=1*Pi[t,i]

In [ ]:

tau={}
for s in states:
    i=0
    tau[s]=0.0001*(popplus12_s[s])/10000
    i+=1
M = 1000000000/10000
supply=1000000/10000
cost_param=pd.DataFrame({"w":[2,2,2,5],
                "g":[20,20,20,20],
                "c":[20,19,15,11]})

B=[500000,200000,1200000]
suppliers = [1,2,3]
mu=14/75/365/10000
gamma = 1
gamma1=1

def vac_alloc(I01,S01,V01,R01,T1,T2,available_at_supplier,a,lambda21):
    global Vac_i,U_I,U_D,Results,inf,U_I0,vac_r,S1,I1,V1,lb,R1
    model0 = gp.Model()

    for t in range(T1-1,T2-1):
        lb[t]= model0.addVar( lb=0, vtype=GRB.CONTINUOUS)
    for s in states:

        for t in range(T1,T2+1):
            S[t,s] = model0.addVar( lb=0, ub=popplus12_s[s] ,vtype=GRB.CONTINUOUS)
            V[t,s] = model0.addVar( lb=0, ub=popplus12_s[s] , vtype=GRB.CONTINUOUS)
            I[t,s] = model0.addVar( lb=0, ub=popplus12_s[s] ,vtype=GRB.CONTINUOUS)
            R[t,s] = model0.addVar( lb=0, ub=popplus12_s[s] ,vtype=GRB.CONTINUOUS)
            reinf[t,s]= model0.addVar( lb=0,ub=popplus12_s[s] ,vtype=GRB.CONTINUOUS)
            tau[t,s]= model0.addVar( lb=0,ub=popplus12_s[s] ,vtype=GRB.CONTINUOUS)
        for t in range(T1-1,T2-1):
            Vac_1[t,s] = model0.addVar(lb=0, ub=upper[t,s]*popplus12_s[s]/10000,vtype=GRB.CONTINUOUS)
            Vac_2[t,s] = model0.addVar(lb=0, ub=upper[t,s]*popplus12_s[s]/10000,vtype=GRB.CONTINUOUS)
            Vac[t,s] = model0.addVar(lb=upper[t,s]*0, ub=upper[t,s]*popplus12_s[s]/10000,vtype=GRB.CONTINUOUS)
            # Vac[t,s] = model0.addVar(lb=upper[t,s]*opening_f[t-1,s], ub=upper[t,s]*popplus12_s[s]/10000,vtype=GRB.CONTINUOUS)
        for t in range(T1-1,T2-1):
            U_I[t,s] = model0.addVar(vtype=GRB.BINARY)
            U_D[t,s] = model0.addVar(vtype=GRB.BINARY)
    X_o = {(t, s): model0.addVar(lb=0,ub=1-budget_inf[T1-3],vtype=GRB.BINARY)  for t in  range(T1-1,T2-1) for s in states}
    cost={}
    for s in states:
        cost[s] = model0.addVar(lb=0, vtype=GRB.CONTINUOUS)

    for s in states:
        model0.addConstr(S[T1, s] ==S01[T1,s])
        model0.addConstr(V[T1, s] == V01[T1,s])
        model0.addConstr(I[T1, s] == I01[T1,s])
        model0.addConstr(R[T1, s] == R01[T1,s])

        for t in range(T1,T2):
            model0.addConstr(S[t+1,s] == reinf[t,s]+(1-mu)*S[t,s] +mu - 10000*beta[t,s]*I[t,s]*S[t,s]-Vac_1[t-gamma1,s])
            model0.addConstr(V[t+1,s] == V[t,s] +Vac_2[t-gamma1,s]+Vac_1[t-gamma1,s] -effec*10000*beta[t,s]*V[t,s]*I[t,s] -mu*V[t,s])
            model0.addConstr(I[t+1,s] == I[t,s] + 10000*beta[t,s]*S[t,s] *I[t,s]+effec*10000*beta[t,s]*V[t,s]*I[t,s]-gamma*I[t,s]-mu*I[t,s])
            model0.addConstr(R[t+1,s] == R[t,s] + +gamma*I[t,s]-mu*R[t,s]-Vac_2[t-gamma1,s])
            model0.addConstr(Vac[t-gamma1,s]==Vac_1[t-gamma1,s]+Vac_2[t-gamma1,s])
        if t>=(reinf_period+3):
            model0.addConstr(reinf[t,s]==(reinf_rate)*(10000*beta[t-reinf_period-1,s]*S01[t-reinf_period-1,s] *I01[t-reinf_period-1,s]-mu*I01[t-reinf_period-1,s]))
        else:
            model0.addConstr(reinf[t,s]==reinf_rate*underreported[s,t+21-reinf_period]*(states_cases0[s][t+22-reinf_period]-states_cases0[s][t+21-reinf_period])/10000)
        if t>=5:
            model0.addConstr(tau[t,s]==sum(10000*beta[ti,s]*S01[ti,s] *I01[ti,s]+effec*10000*beta[ti,s]*V01[ti,s]*I01[ti,s] for ti in range(t-3,t))/3)
        else:
            model0.addConstr(tau[t,s]==0.5*beta[2,s]*popplus12_s[s]/Scale)
    #                 model0.addConstr((Vac[t,s])<=0.4*sum(county_cap[c] for c in counties2[s]))
    for t in range(T1-1,T2-1):
        for s in states:
            model0.addConstr(Vac[t,s]<=1*est_demand[t,s])
            model0.addConstr(X_o[t,s]<=Vac[t,s])
            model0.addConstr(Vac[t,s]<=sum(county_cap[c] for c in counties2[s])+X_o[t,s])
    for s in states:
        for t in range(T1,T2):
            model0.addConstr(Vac[t-1,s]/sum(popplus12[s,c]for c in counties2[s])*10000>=lb[t-1])


    for s in states:
        for t in range(T1,T2):
            model0.addConstr((tau[t,s] - M*(1-U_I[t-1,s]))<=Scale*beta[t,s] * S[t, s] * I[t, s] + effec*Scale*beta[t,s] *  V[t, s] * I[t, s])
            model0.addConstr((tau[t,s] + M*(U_I[t-1,s]) )>=Scale*beta[t,s] * S[t, s] * I[t, s] + effec*Scale*beta[t,s] *  V[t, s] * I[t, s] )
    for s in states:
        for t in range(T1,T2):
            model0.addConstr((sum(county_cap[c] for c in counties2[s])*(T2-T1) - M*(1-U_D[t-1,s]))<=est_demand[t,s])
            model0.addConstr((sum(county_cap[c] for c in counties2[s])*(T2-T1)+ M*(U_D[t-1,s]) )>=est_demand[t,s] )
    for t in range(T1-1,T2-1):
        model0.addConstr(1/(1-wastage)*(gp.quicksum(Vac[t,s] for s in states))<= a*(Pi[t-1,1]+Pi[t-1,2]+Pi[t-1,3])+a*sum(available_at_supplier[T1-1,i] for i in suppliers))
    for t in range(T1-1,T2-1):
        for s in states:
            model0.addConstr(X_o[t,s]<= U_I[t,s]+U_D[t,s])
            if t>=2:
                model0.addConstr(X_o[t,s]>= (1-budget_inf[T1-3])* (U_I[t,s])*upper[t,s]/popplus12_s[s]*Scale)
                model0.addConstr(X_o[t,s]>=(1-budget_inf[T1-3])*U_D[t,s]*upper[t,s]/popplus12_s[s]*Scale)
    model0.setObjective(-gp.quicksum(gp.quicksum((10000*beta[t,s] * S[t, s] * I[t, s] + effec*10000*beta[t,s] *  V[t, s] * I[t, s]) for s in states)for t in range(T1+1, T2+1))+lambda21*sum(1*lb[t-1] for t in range(T1, T2)), GRB.MAXIMIZE)
    # Set optimization parameters
    model0.Params.TIME_LIMIT = 72000
    # model0.Params.MIPGap=0.06
    # model0.setParam(GRB.Param.FlowCoverCuts, 2)
    # model0.setParam(GRB.Param.NetworkCuts, 2)
    model0.optimize()
    # model0.setParam(GRB.Param.Heuristics, 0)
    inf={(t,s): beta[t,s] * S[t, s].x * I[t, s].x + effec*beta[t,s] *  V[t, s].x * I[t, s].x for t in range(T1, T2) for s in states  }
    for t in range(T1-1,T2-1):
        for s in states:
            U_I[t, s]= U_I[t, s].x
            U_D[t, s]= U_D[t, s].x
    Vac_i={(t, s): Vac[t, s].x for t in range(T1-1,T2-1)  for s in states }
    vac_r={}
    for s in states:
        for t in range(T1,T2):
            if sum(Vac[t-1, s].x for s in states)==0:
                vac_r[t-1,s]=0
            else:
                vac_r[t-1,s]=(Vac[t-1, s]).x/sum(Vac[t-1, s].x for s in states)
    S1={(t, s): S[t, s].x for t in range(T1+1,T2+1)  for s in states }
    I1={(t, s): I[t, s].x for t in range(T1+1,T2+1)  for s in states }
    V1={(t, s): V[t, s].x for t in range(T1+1,T2+1) for s in states }
    R1={(t, s): R[t, s].x for t in range(T1+1,T2+1) for s in states }
    infection_results[T1]=sum(10000*beta[t,s] * S[t, s].x * I[t, s].x + effec*10000*beta[t,s] *  V[t, s].x * I[t, s].x for s in states for t in range(T1, T2))
    state_lower_bound_results[T1]=sum(1*lb[t-1].x for t in range(T1, T2))
    Vac_df=pd.DataFrame({"States":states})
    sol_time[T1]=model0.Runtime
    if model0.Status == GRB.OPTIMAL:
        for s in states:
            for t in  range(T1-1,T2-1):
                X_of[t,s] = X_o[t,s].x
    elif model0.Status == GRB.INFEASIBLE:
        for s in states:
            for t in  range(T1-1,T2-1):
               X_of[t,s] = "nan"
    state_min[T1]=lb[T1-1].x
    return Results,Vac_i,inf,U_I,vac_r,S1,I1,V1,lb,U_D,R1, X_of


def state_alloc(Vac_i,inf,X_of,T1,T2,W1_p,W2_p,W3_p,lambda22,lambda23):
    global inventory
    model1 = gp.Model()

    for s in states:
        Gini_Coef[s] = model1.addVar( lb=0, ub=1, vtype=GRB.CONTINUOUS)
        Y_b[s] = model1.addVar(lb=0, vtype=GRB.BINARY)
        for t in range(T1-2,T2):
            lb0[t,s]=model1.addVar(lb=0,vtype=GRB.CONTINUOUS)
            W1[t,s] = model1.addVar( lb=0, vtype=GRB.CONTINUOUS)
            for i in [1,2,3]:
                G1[t,i,s] = model1.addVar( lb=0, vtype=GRB.CONTINUOUS)
    W2 = {(t, s, c): model1.addVar(lb=0, vtype=GRB.CONTINUOUS) for t in range(T1-2,T2) for s in states for c in counties2[s]}
    G2 = {(t, s, c): model1.addVar(lb=0, vtype=GRB.CONTINUOUS) for t in range(T1-1,T2) for s in states for c in counties2[s]}
    G3 = {(t, s, c,l): model1.addVar(lb=0, vtype=GRB.CONTINUOUS) for t in range(T1-1,T2) for s in states for c in counties2[s] for l in local_centers[s,c]}
    W3 = {(t, s, c,l): model1.addVar(lb=0, vtype=GRB.CONTINUOUS) for t in range(T1-2,T2) for s in states for c in counties2[s] for l in local_centers[s,c]}
    Vac0 = {(t, s,c): model1.addVar(lb=0, ub=upper0[T1-1,s,c]*popplus12[s,c]/10000, vtype=GRB.CONTINUOUS) for t in range(T1-1,T2-1) for s in states for c in counties2[s]}
    ub=model1.addVar(lb=0,ub=0.1)
    y=model1.addVar(lb=0,vtype=GRB.CONTINUOUS)
    sup_avl={(t,i): model1.addVar(lb=0, ub=Pi[t-2,i], vtype=GRB.CONTINUOUS) for t in range(T1,T2) for i in suppliers}
    for t in range(T1-1,T2-1):
        for s in states:
            for c in counties2[s]:
                model1.addConstr(Vac0[t,s,c]<=county_cap[c])

    for t in range(T1-1,T2-1):
        for s in states:
            model1.addConstr(Vac_i[t,s]>=(gp.quicksum(Vac0[t,s,c] for c in counties2[s]))+(1-wastage)*X_of[t,s])
    for t in range(T1-1,T2-1):
        for i in suppliers:
            model1.addConstr(gp.quicksum(G1[t,i,s] for s in states)+sup_avl[T1,i]==Pi[t-1,i]+available_at_supplier[T1-1,i])
    for s in states:
        for t in range(T1-1,T2-1):
            model1.addConstr(gp.quicksum(G1[t,i,s] for i in suppliers)+W1[t-1,s] == gp.quicksum(G2[t,s,c] for c in counties2[s] )+X_of[t,s]*capacity+W1[t,s])
            # model.addConstr(gp.quicksum(G3[t,h_s,j] for h_s in statehubs)*1/21==gp.quicksum(G4[t,j,l]for l in localcenters[j]))
        model1.addConstr(W1[t-1,s]==W1_p[T1-2,s])
    for t in range(T1-1,T2-1):
        for s in states:
            for c in counties2[s]:
                model1.addConstr (G2[t,s,c]  + W2[t-1,s,c]== gp.quicksum(G3[t,s,c,l] for l in local_centers[s,c])+ W2[t,s,c] )
                model1.addConstr(W2[t-1,s,c]==W2_p[T1-2,s,c])
    for t in range(T1-1,T2-1):
        for s in states:
            for c in counties2[s]:
                model1.addConstr((1-wastage)*gp.quicksum(G3[t,s,c,l]+W3[t-1,s,c,l] for l in local_centers[s,c])==Vac0[t,s,c]+gp.quicksum(W3[t,s,c,l] for l in local_centers[s,c]))
                for l in local_centers[s,c]:
                    model1.addConstr(W3[t-1,s,c,l]==W3_p[T1-2,s,c,l])
                    # model1.addConstr(W3[-1,s,c,l]==0)
    for t in range(T1-1,T2-1):
        for s in states:
            for c in counties2[s]:
                model1.addConstr(Vac0[t,s,c]/popplus12[s,c]*10000 >= lb0[t,s])
    model1.setObjective(lambda22*(sum(access[s]*(sum(weights[s,c]*(Vac0[t,s,c]) for c in counties2[s])+1*X_of[t,s])for s in states for t in range(T1-1,T2-1)))+lambda23*sum(lb0[t,s] for s in states for t in range(T1-1,T2-1)), GRB.MAXIMIZE)
    model1.Params.TIME_LIMIT = 7200
    # model1.Params.MIPGap=1
    model1.setParam(GRB.Param.FlowCoverCuts, 2)
    model1.setParam(GRB.Param.NetworkCuts, 2)
    model1.Params.ScaleFlag=2
    model1.optimize()
    model1.getObjective()
    if model1.Status == GRB.OPTIMAL:
        for s in states:
            for t in  range(T1-1,T2-1):
                for c in counties2[s]:
                    Vac_all[t,s,c] = Vac0[t,s,c].x
    elif model1.Status == GRB.INFEASIBLE:
        for s in states:
            for t in  range(T1-1,T2-1):
                for c in counties2[s]:
                    Vac_all[t,s,c] = "nan"
    if model1.Status == GRB.OPTIMAL:
        for s in states:
            for t in  range(T1-1,T2-1):
                Vac_avl[t,s] = sum(Vac0[t,s,c].x for c in counties2[s])
    elif model1.Status == GRB.INFEASIBLE:
        for s in states:
            for t in  range(T1-1,T2-1):
                Vac_avl[t,s] = "nan"
    if model1.Status == GRB.INFEASIBLE:
        cost[T1]= "nan"

    else:
        cost[T1]= sum(sum(cost_param["w"][0]*W1[t,s].x for s in states )+sum(cost_param["w"][1]*W2[t,s,c].x for s in states for c in counties2[s])+sum(cost_param["w"][2]*W3[t,s,c,l].x for s in states for c in counties2[s] for l in local_centers[s,c])+sum(distance[s]/2000*cost_param["g"][1]*G2[t,s,c].x for s in states for c in counties2[s] )+sum(cost_param["c"][1]*G1[t,1,s].x+cost_param["c"][2]*G1[t,2,s].x +cost_param["c"][3]*G1[t,3,s].x for s in states )+sum(cost_param["w"][3]*G3[t,s,c,l].x for s in states for c in counties2[s] for l in local_centers[s,c])+sum(cost_param["c"][0]*X_of[t,s] for s in states)for t in  range(T1-1,T2-1))
    for i in suppliers:
        available_at_supplier[T1,i]=sup_avl[T1,i].x
    for s in states:
        for t in range(T1-1,T2-1):
            #for c in counties2[s]:
            if model1.Status == GRB.OPTIMAL:
                #inventory[t,s]= sum(W2[t,s,c].x for c in counties2[s])
                inventory[t,s]=W1[t,s].x
            elif model1.Status == GRB.INFEASIBLE:
                inventory[t,s]="nan"
    #inventory
    if model1.Status == GRB.OPTIMAL:
        for t in range(T1-1,T2-1):
            for s in states:
                W1_p[t,s]=W1[t,s].x
                for c in counties2[s]:
                    W2_p[t,s,c]=W2[t,s,c].x
                    for l in local_centers[s,c]:
                        W3_p[t,s,c,l]=W3[t,s,c,l].x
    knapsack_results[T1]=(sum(access[s]*(sum(weights[s,c]*(Vac0[t,s,c].x) for c in counties2[s])+1*X_of[t,s])for s in states for t in range(T1-1,T2-1)))
    county_lower_bound_results[T1]=sum(lb0[t,s].x for s in states for t in range(T1-1,T2-1))

#Vaccination
    # for t in range(T1-1,T2-1):
    #     if model1.Status == GRB.OPTIMAL:
    #         results1[f"t/s"]=Vac[t,s]
    #     elif model1.Status == GRB.INFEASIBLE:
    #         results1[f"t/s"]="nan"
#Supply from suppliers
    for t in range(T1-1,T2-1):
        for s in states:
            if model1.Status == GRB.OPTIMAL:
                G1_supply[t,1,s]= G1[t,1,s].x
                G1_supply[t,2,s]= G1[t,2,s].x
                G1_supply[t,3,s]= G1[t,3,s].x
            elif model1.Status == GRB.INFEASIBLE:
                G1_supply[t,1,s]="nan"
                G1_supply[t,2,s]="nan"
                G1_supply[t,3,s]="nan"
    for t in range(T1-1,T2-1):
        for s in states:
            for c in counties2[s]:
                if model1.Status == GRB.OPTIMAL:
                    G2_supply[t,s,c]= G2[t,s,c].x
                elif model1.Status == GRB.INFEASIBLE:
                    G2_supply[t,s,c]="nan"
    sol_time0[T1]=model1.Runtime
    return inventory,available_at_supplier

def alg_impl_dec2(lambda21,lambda22,lambda23):
    global final_cost,n, supply, activate
    for s in states:
        for t in  range(-3,1):
            opening_f[t,s]=0
            opening[t,s]=0
    for s in states:
        S01[2,s]=S0101[0,s]/10000
        I01[2,s]=I0101[0,s]/10000
        R01[2,s]=0.95*2*states_cases0[s][23]/10000
        V01[2,s]=0
        Vac[1,s]=0
        upper[1,s]=0
    T2=3
    T1=2
    a=1
    for s in states:
        upper[T1-1,s]=0
        for c in counties2[s]:
            upper0[T1-1,s,c]=0
    vac_alloc(I01,S01,V01,R01,T1,T2,available_at_supplier,a,lambda21)
    state_alloc(Vac_i,inf,X_of,T1,T2,W1_p,W2_p, W3_p,lambda22,lambda23)
    final_cost+=cost[T1]
    for s in states:
        for t in  range(T1-1,T2-1):
            opening[t,s]=U_I[t,s]
            opening_f[t,s]=X_of[t,s]
            Vac_state[t,s]=Vac_i[t,s]
            for c in counties2[s]:
                Vac_coun[t,s,c] = Vac_all[t,s,c]

    while ((T2<T+1)):
        T1+=1
        T2+=1
        for s in states:
            if s=='TX':
                upper[T1-1,s]=(state_vaccine[10,s]/10000-sum(Vac_state[t,s] for t in range(1,T1-1)))/(popplus12_s[s]/10000)
                if upper[T1-1,s]<=0:
                    upper[T1-1,s]=0
            else:
                upper[T1-1,s]=(state_vaccine[10,s]/10000-sum(Vac_state[t,s] for t in range(1,T1-1)))/(popplus12_s[s]/10000)
                if upper[T1-1,s]<=0:
                    upper[T1-1,s]=0
            for c in counties2[s]:
                upper0[T1-1,s,c]=(popplus12[s,c]/10000-sum(Vac_coun[t,s,c] for t in range(1,T1-1)))/popplus12[s,c]*10000
                if s=='TX':
                    upper0[T1-1,s,c]=(popplus12[s,c]/10000-sum(Vac_coun[t,s,c] for t in range(1,T1-1)))/popplus12[s,c]*10000
                if upper0[T1-1,s,c]<=0:
                    upper0[T1-1,s,c]=0
        # if T1<=6:
        #     for s in states:
        #         upper[T1-1,s]=0
        if budget_inf[n-1]==0:
            for s in states:
                S01[T1,s]=S1[T1,s]
                I01[T1,s]=I1[T1,s]
                V01[T1,s]=V1[T1,s]
                R01[T1,s]=R1[T1,s]
        else:
            for s in states:
                S01[T1,s]=(budget_inf[n-1])*S01[T1-1,s]
                I01[T1,s]=(budget_inf[n-1])*I01[T1-1,s]
                V01[T1,s]=(budget_inf[n-1])*V01[T1-1,s]
                R01[T1,s]=(budget_inf[n-1])*R01[T1-1,s]

        vac_alloc(I01,S01,V01,R01,T1,T2,available_at_supplier,a,lambda21)
        print("* \n vac_alloc solved\n ",T1)
        state_alloc(Vac_i,inf,X_of,T1,T2,W1_p,W2_p, W3_p,lambda22,lambda23)
        print(" *\n state_alloc solved\n ")
        for s in states:
            for t in  range(T1-1,T2-1):
                Vac_state[t,s]=Vac_i[t,s]
                opening[t,s]=U_I[t,s]
        for s in states:
            for t in  range(T1-1,T2-1):
                opening_f[t,s]=X_of[t,s]
                supply[t,1,s]=G1_supply[t,1,s]
                supply[t,2,s]=G1_supply[t,2,s]
                supply[t,3,s]=G1_supply[t,3,s]

                for c in counties2[s]:
                    Vac_coun[t,s,c] = Vac_all[t,s,c]
        final_cost+=cost[T1]


        if ((0.95*B[0]<=final_cost) and(final_cost<=B[0])):
            if T1<=T:
                activate=1
                for i in range(T1-3,T):
                    budget_inf[i]=1
            break
        elif (final_cost>B[0]) and ((budget_inf[n-1]==1)and(budget_inf[n-2]==1)):
            budget_inf[n]=1
            final_cost-=cost[T1]
            T1-=1
            T2-=1
            activate=1
            for i in range(T1-3,T):
                budget_inf[i]=1
            break
        elif (final_cost>B[0]):
            budget_inf[n]=1
            final_cost-=cost[T1]
            T1-=1
            T2-=1
            a=(B[0]-final_cost)/((5+19)*(Pi[T1-1,1]+Pi[T1-1,2]+Pi[T1-1,3]+sum(available_at_supplier[T1,i] for i in suppliers)))
        else:
            budget_inf[n]=0
        n+=1
        if n>=14:
            break

    if activate==1:
        upper[T1-2,s]=0
        for s in states:
            S1[T1+1,s]=S01[T1,s]
            I1[T1+1,s]=I01[T1,s]
            V1[T1+1,s]=V01[T1,s]
            R1[T1+1,s]=R01[T1,s]
        for i in suppliers:
            available_at_supplier[T1,i]=0
        while ((T2<T+1)):
            T1+=1
            T2+=1
            for i in suppliers:
                available_at_supplier[T1,i]=0
            for s in states:
                upper[T1-1,s]=0
            for s in states:
                opening_f[T1-1,s]=0
                Vac_state[T1-1,s]=0
                S01[T1,s]=S1[T1,s]
                I01[T1,s]=I1[T1,s]
                V01[T1,s]=V1[T1,s]
                R01[T1,s]=R1[T1,s]
            vac_alloc(I01,S01,V01,R01,T1,T2,available_at_supplier,a,lambda21)

In [ ]:


lambda_list1=[300,200,150,100,50,25,201,6,12,10,1,0]
lambda_list2=[0.01,0.001,0.0005]
lambda_list3=[1,2,5,10]
lambda_list=[0,1,2,6,10,100,100]
# list(pd.MultiIndex.from_product([lambda_list1,lambda_list2,lambda_list3])))
lambda_results=pd.DataFrame(columns=['infections','state_lb','knapsack','county_lb','sol_time'],index =pd.MultiIndex.from_product([lambda_list1,lambda_list2,lambda_list3]))
for li in list(pd.MultiIndex.from_product([lambda_list1,lambda_list2,lambda_list3])):
    sol_time={}
    sol_time0={}
    Vac_avl={}
    for s in states:
        Vac_avl[1,s]=0
    dic={}
    for t in time:
        for s in states:
            dic[t,s]=None
    results1=pd.DataFrame.from_dict(dic, orient='index')
    dic1={}
    for t in time:
        for s in states:
            for c in counties2[s]:
                dic1[t,s]=None
    results2=pd.DataFrame.from_dict(dic1, orient='index')
    # global T1,T2
    Scale=10000
    # global lower_bound
    effec=0.2
    inventory={}
    opening={}
    for s in states:
        #for c in counties2[s]:
        inventory[0,s]=0
    upper={}
    upper0={}
    T=12
    S={}
    V={}
    I={}
    R={}
    Vac={}
    Vac_1={}
    Vac_2={}
    inf={}
    U_I={}
    U_D={}
    reinf={}
    Results={}
    lb={}
    infection_results={}
    state_lower_bound_results={}
    knapsack_results={}
    county_lower_bound_results={}
    Gini_Coef = {}
    G1 = {}
    W1 = {}
    W2 = {}
    G2 = {}
    W3 = {}
    G3 = {}
    Y = {}
    X_o={}
    Y_b = {}
    X_of = {}
    cost = {}
    capacity=10000/10000
    Y_bn=[]
    # results["Vac"]=None
    Vac_all={}
    G1_supply={}
    G2_supply={}
    sup_avl={}
    state_cost={}
    # inventory={}
    lb0={}
    #Retrieval of results
    W1_p={}
    W2_p={}
    W3_p={}
    state_min={}
    tau={}
    for t in range(0,2):
        for s in states:
            W1_p[t,s]=0
            for c in counties2[s]:
                W2_p[t,s,c]=0
                for l in local_centers[s,c]:
                    W3_p[t,s,c,l]=0
    available_at_supplier={}
    for i in suppliers:
        available_at_supplier[1,i]=0
    bud_ava={}
    lam=100
    budget_inf={}
    Vac_coun={}
    Vac_state={}
    S01={}
    I01={}
    V01={}
    R01={}
    opening={}
    opening_f={}
    final_cost=0
    supply={}
    for t in range(-1,T+4):
        budget_inf[t]=0
    n=0
    activate=0
    alg_impl_dec2(li[0],li[1],li[2])
    lambda_results.loc[li]=[sum(infection_results[t] for t in range(2,13)),sum(state_lower_bound_results[t] for t in range(2,12)),sum(knapsack_results[t] for t in range(2,12)),sum(county_lower_bound_results[t] for t in range(2,12)),sum(sol_time[t]+sol_time0[t] for t in range(2,12))]
lambda_results.to_csv('lambda_results.csv',index=True)

In [ ]:
sol_time={}
sol_time0={}
Vac_avl={}
for s in states:
    Vac_avl[1,s]=0
dic={}
for t in time:
    for s in states:
        dic[t,s]=None
results1=pd.DataFrame.from_dict(dic, orient='index')
dic1={}
for t in time:
    for s in states:
        for c in counties2[s]:
            dic1[t,s]=None
results2=pd.DataFrame.from_dict(dic1, orient='index')
global T1,T2
Scale=10000
global lower_bound
effec=0.2
inventory={}
opening={}
for s in states:
    #for c in counties2[s]:
    inventory[0,s]=0
upper={}
upper0={}
T=12
S={}
V={}
I={}
R={}
Vac={}
# S1={}
# V1={}
# I1={}
# R1={}
Vac_1={}
Vac_2={}
inf={}
# X0={}
U_I={}
U_D={}
reinf={}
Results={}
lb={}
u_bar1={}
u2={}
u3={}
Gini_Coef = {}
G1 = {}
W1 = {}
W2 = {}
G2 = {}
W3 = {}
G3 = {}
Y = {}
X_o={}
Y_b = {}
X_of = {}
cost = {}
capacity=10000/10000
Y_bn=[]
# results["Vac"]=None
Vac_all={}
G1_supply={}
G2_supply={}
sup_avl={}
state_cost={}
# inventory={}
lb0={}
#Retrieval of results
W1_p={}
W2_p={}
W3_p={}
state_min={}
tau={}
for t in range(0,2):
    for s in states:
        W1_p[t,s]=0
        for c in counties2[s]:
            W2_p[t,s,c]=0
            for l in local_centers[s,c]:
                W3_p[t,s,c,l]=0
available_at_supplier={}
for i in suppliers:
    available_at_supplier[1,i]=0
bud_ava={}
lam=100
def vac_alloc(I01,S01,V01,R01,T1,T2,available_at_supplier,a,lam):
    global Vac_i,U_I,U_D,Results,inf,U_I0,vac_r,S1,I1,V1,lb,R1
    model0 = gp.Model()

    for t in range(T1-1,T2-1):
        lb[t]= model0.addVar( lb=0, vtype=GRB.CONTINUOUS)
    # global Vac, U_I
    # Define parameters

    for s in states:

        for t in range(T1,T2+1):
            S[t,s] = model0.addVar( lb=0, ub=popplus12_s[s] ,vtype=GRB.CONTINUOUS)
            V[t,s] = model0.addVar( lb=0, ub=popplus12_s[s] , vtype=GRB.CONTINUOUS)
            I[t,s] = model0.addVar( lb=0, ub=popplus12_s[s] ,vtype=GRB.CONTINUOUS)
            R[t,s] = model0.addVar( lb=0, ub=popplus12_s[s] ,vtype=GRB.CONTINUOUS)
            reinf[t,s]= model0.addVar( lb=0,ub=popplus12_s[s] ,vtype=GRB.CONTINUOUS)
            tau[t,s]= model0.addVar( lb=0,ub=popplus12_s[s] ,vtype=GRB.CONTINUOUS)
        for t in range(T1-1,T2-1):
            Vac_1[t,s] = model0.addVar(lb=0, ub=upper[t,s]*popplus12_s[s]/10000,vtype=GRB.CONTINUOUS)
            Vac_2[t,s] = model0.addVar(lb=0, ub=upper[t,s]*popplus12_s[s]/10000,vtype=GRB.CONTINUOUS)
            Vac[t,s] = model0.addVar(lb=upper[t,s]*0, ub=upper[t,s]*popplus12_s[s]/10000,vtype=GRB.CONTINUOUS)
            # Vac[t,s] = model0.addVar(lb=upper[t,s]*opening_f[t-1,s], ub=upper[t,s]*popplus12_s[s]/10000,vtype=GRB.CONTINUOUS)
        for t in range(T1-1,T2-1):
            U_I[t,s] = model0.addVar(vtype=GRB.BINARY)
            U_D[t,s] = model0.addVar(vtype=GRB.BINARY)
    X_o = {(t, s): model0.addVar(lb=0,ub=1-budget_inf[T1-3],vtype=GRB.BINARY)  for t in  range(T1-1,T2-1) for s in states}

    cost={}
    for s in states:
        cost[s] = model0.addVar(lb=0, vtype=GRB.CONTINUOUS)
    for s in states:
        model0.addConstr(S[T1, s] ==S01[T1,s])
        model0.addConstr(V[T1, s] == V01[T1,s])
        model0.addConstr(I[T1, s] == I01[T1,s])
        model0.addConstr(R[T1, s] == R01[T1,s])
        for t in range(T1,T2):
            model0.addConstr(S[t+1,s] == reinf[t,s]+(1-mu)*S[t,s] +mu - 10000*beta[t,s]*I[t,s]*S[t,s]-Vac_1[t-gamma1,s])
            model0.addConstr(V[t+1,s] == V[t,s] +Vac_2[t-gamma1,s]+Vac_1[t-gamma1,s] -effec*10000*beta[t,s]*V[t,s]*I[t,s] -mu*V[t,s])
            model0.addConstr(I[t+1,s] == I[t,s] + 10000*beta[t,s]*S[t,s] *I[t,s]+effec*10000*beta[t,s]*V[t,s]*I[t,s]-gamma*I[t,s]-mu*I[t,s])
            model0.addConstr(R[t+1,s] == R[t,s] + +gamma*I[t,s]-mu*R[t,s]-Vac_2[t-gamma1,s])
            model0.addConstr(Vac[t-gamma1,s]==Vac_1[t-gamma1,s]+Vac_2[t-gamma1,s])
        if t>=(reinf_period+3):
            model0.addConstr(reinf[t,s]==(reinf_rate)*(10000*beta[t-reinf_period-1,s]*S01[t-reinf_period-1,s] *I01[t-reinf_period-1,s]-mu*I01[t-reinf_period-1,s]))
        else:
            model0.addConstr(reinf[t,s]==reinf_rate*underreported[s,t+21-reinf_period]*(states_cases0[s][t+22-reinf_period]-states_cases0[s][t+21-reinf_period])/10000)
        if t>=5:
            model0.addConstr(tau[t,s]==sum(10000*beta[ti,s]*S01[ti,s] *I01[ti,s]+effec*10000*beta[ti,s]*V01[ti,s]*I01[ti,s] for ti in range(t-3,t))/3)
        else:
            model0.addConstr(tau[t,s]==0.5*beta[2,s]*popplus12_s[s]/Scale)
    #                 model0.addConstr((Vac[t,s])<=0.4*sum(county_cap[c] for c in counties2[s]))
    for t in range(T1-1,T2-1):
        for s in states:
            model0.addConstr(Vac[t,s]<=1*est_demand[t,s])
            model0.addConstr(X_o[t,s]<=Vac[t,s])
            model0.addConstr(Vac[t,s]<=sum(county_cap[c] for c in counties2[s])+X_o[t,s])
    for s in states:
        for t in range(T1,T2):
            model0.addConstr(Vac[t-1,s]/sum(popplus12[s,c]for c in counties2[s])*10000>=lb[t-1])


    for s in states:
        for t in range(T1,T2):
            model0.addConstr((tau[t,s] - M*(1-U_I[t-1,s]))<=Scale*beta[t,s] * S[t, s] * I[t, s] + effec*Scale*beta[t,s] *  V[t, s] * I[t, s])
            model0.addConstr((tau[t,s] + M*(U_I[t-1,s]) )>=Scale*beta[t,s] * S[t, s] * I[t, s] + effec*Scale*beta[t,s] *  V[t, s] * I[t, s] )
    for s in states:
        for t in range(T1,T2):
            model0.addConstr((sum(county_cap[c] for c in counties2[s])*(T2-T1) - M*(1-U_D[t-1,s]))<=est_demand[t,s])
            model0.addConstr((sum(county_cap[c] for c in counties2[s])*(T2-T1)+ M*(U_D[t-1,s]) )>=est_demand[t,s] )
    for t in range(T1-1,T2-1):
        model0.addConstr(1/(1-wastage)*(gp.quicksum(Vac[t,s] for s in states))<= (a*(Pi[t-1,1]+Pi[t-1,2]+Pi[t-1,3])+a*(sum(Pi[f,1]+Pi[f,2]+Pi[f,3] for f in range(0,t-1)))-sum(Vac_state[f,s]for s in states for f in range(1,t))))
    for t in range(T1-1,T2-1):
        for s in states:
            model0.addConstr(X_o[t,s]<= U_I[t,s]+U_D[t,s])
            if t>=2:
                model0.addConstr(X_o[t,s]>= (1-budget_inf[T1-3])* (U_I[t,s]))
                model0.addConstr(X_o[t,s]>=(1-budget_inf[T1-3])*U_D[t,s])
    model0.setObjective(gp.quicksum(gp.quicksum((10000*beta[t,s] * S[t, s] * I[t, s] + effec*10000*beta[t,s] *  V[t, s] * I[t, s]) for s in states)for t in range(T1+1, T2+1))-1*sum(1*lb[t-1] for t in range(T1, T2)), GRB.MINIMIZE)
    model0.Params.TIME_LIMIT = 72000
    # model0.Params.MIPGap=0.06
    # model0.setParam(GRB.Param.FlowCoverCuts, 2)
    # model0.setParam(GRB.Param.NetworkCuts, 2)
    model0.optimize()
    # model0.setParam(GRB.Param.Heuristics, 0)
    inf={(t,s): beta[t,s] * S[t, s].x * I[t, s].x + effec*beta[t,s] *  V[t, s].x * I[t, s].x for t in range(T1, T2) for s in states  }

    for t in range(T1-1,T2-1):
        for s in states:
            U_I[t, s]= U_I[t, s].x
            U_D[t, s]= U_D[t, s].x
    Vac_i={(t, s): Vac[t, s].x for t in range(T1-1,T2-1)  for s in states }
    vac_r={}
    for s in states:
        for t in range(T1,T2):
            if sum(Vac[t-1, s].x for s in states)==0:
                vac_r[t-1,s]=0
            else:
                vac_r[t-1,s]=(Vac[t-1, s]).x/sum(Vac[t-1, s].x for s in states)

    S1={(t, s): S[t, s].x for t in range(T1+1,T2+1)  for s in states }
    I1={(t, s): I[t, s].x for t in range(T1+1,T2+1)  for s in states }
    V1={(t, s): V[t, s].x for t in range(T1+1,T2+1) for s in states }
    R1={(t, s): R[t, s].x for t in range(T1+1,T2+1) for s in states }

    Vac_df=pd.DataFrame({"States":states})
    sol_time[T1]=model0.Runtime
    if model0.Status == GRB.OPTIMAL:
        for s in states:
            for t in  range(T1-1,T2-1):
                X_of[t,s] = X_o[t,s].x
    elif model0.Status == GRB.INFEASIBLE:
        for s in states:
            for t in  range(T1-1,T2-1):
               X_of[t,s] = "nan"
    state_min[T1]=lb[T1-1].x
    return Results,Vac_i,inf,U_I,vac_r,S1,I1,V1,lb,U_D,R1, X_of

def state_alloc(Vac_i,inf,X_of,T1,T2,W1_p,W2_p,W3_p,states):
    global inventory
    model1 = gp.Model()
    cost=model1.addVar(lb=0,ub=B[0])
    for s in states:
        Gini_Coef[s] = model1.addVar( lb=0, ub=1, vtype=GRB.CONTINUOUS)
        Y_b[s] = model1.addVar(lb=0, vtype=GRB.BINARY)
        cost[s] = model1.addVar(lb=0, vtype=GRB.CONTINUOUS)
        for t in range(T1-2,T2):
            lb0[t,s]=model1.addVar(lb=0,vtype=GRB.CONTINUOUS)
            W1[t,s] = model1.addVar( lb=0, vtype=GRB.CONTINUOUS)
            for i in [1,2,3]:
                G1[t,i,s] = model1.addVar( lb=0, vtype=GRB.CONTINUOUS)
    W2 = {(t, s, c): model1.addVar(lb=0, vtype=GRB.CONTINUOUS) for t in range(T1-2,T2) for s in states for c in counties2[s]}
    G2 = {(t, s, c): model1.addVar(lb=0, vtype=GRB.CONTINUOUS) for t in range(T1-1,T2) for s in states for c in counties2[s]}
    G3 = {(t, s, c,l): model1.addVar(lb=0, vtype=GRB.CONTINUOUS) for t in range(T1-1,T2) for s in states for c in counties2[s] for l in local_centers[s,c]}
    W3 = {(t, s, c,l): model1.addVar(lb=0, vtype=GRB.CONTINUOUS) for t in range(T1-2,T2) for s in states for c in counties2[s] for l in local_centers[s,c]}
    Vac0 = {(t, s,c): model1.addVar(lb=0, ub=upper0[T1-1,s,c]*popplus12[s,c]/10000, vtype=GRB.CONTINUOUS) for t in range(T1-1,T2-1) for s in states for c in counties2[s]}
    ub=model1.addVar(lb=0,ub=1)
    y=model1.addVar(lb=0,vtype=GRB.CONTINUOUS)
    sup_avl={(t,i): model1.addVar(lb=0, vtype=GRB.CONTINUOUS) for t in range(T1-2,T2) for i in suppliers}
    for i in suppliers:
        model1.addConstr(sup_avl[1,i]==0)
    for t in range(T1-1,T2-1):
        for s in states:
            for c in counties2[s]:
                model1.addConstr(Vac0[t,s,c]<=county_cap[c])

    for t in range(T1-1,T2-1):
        for s in states:
            model1.addConstr(Vac_state[t,s]==(gp.quicksum(Vac0[t,s,c] for c in counties2[s]))+(1-wastage)*X_of[t,s])
    for t in range(T1-1,T2-1):
        for i in suppliers:
            model1.addConstr(gp.quicksum(G1[t,i,s] for s in states)+sup_avl[t,i]==Pi_adjusted[t-1,i]+sup_avl[t-1,i])
    for s in states:
        for t in range(T1-1,T2-1):
            model1.addConstr(gp.quicksum(G1[t,i,s] for i in suppliers)+W1[t-1,s] == gp.quicksum(G2[t,s,c] for c in counties2[s] )+X_of[t,s]*capacity+W1[t,s])
            # model.addConstr(gp.quicksum(G3[t,h_s,j] for h_s in statehubs)*1/21==gp.quicksum(G4[t,j,l]for l in localcenters[j]))
        model1.addConstr(W1[T1-1,s]==0)
    for t in range(T1-1,T2-1):
        for s in states:
            for c in counties2[s]:
                model1.addConstr (G2[t,s,c]  + W2[t-1,s,c]== gp.quicksum(G3[t,s,c,l] for l in local_centers[s,c])+ W2[t,s,c] )
                model1.addConstr(W2[T1-1,s,c]==0)
    for t in range(T1-1,T2-1):
        for s in states:
            for c in counties2[s]:
                model1.addConstr((1-wastage)*gp.quicksum(G3[t,s,c,l]+W3[t-1,s,c,l] for l in local_centers[s,c])==Vac0[t,s,c]+gp.quicksum(W3[t,s,c,l] for l in local_centers[s,c]))
                for l in local_centers[s,c]:
                    model1.addConstr(W3[T1-1,s,c,l]==0)
    # model1.addConstr( sum(sum(cost_param["w"][0]*W1[t,s] for s in states )+sum(cost_param["w"][1]*W2[t,s,c] for s in states for c in counties2[s])+sum(cost_param["w"][1]*W3[t,s,c,l] for s in states for c in counties2[s] for l in local_centers[s,c])+sum(distance[s]/2000*cost_param["g"][1]*G2[t,s,c] for s in states for c in counties2[s] )+sum(cost_param["c"][1]*G1[t,1,s]+cost_param["c"][2]*G1[t,2,s]+cost_param["c"][3]*G1[t,3,s] for s in states )+sum(cost_param["w"][1]*G3[t,s,c,l] for s in states for c in counties2[s] for l in local_centers[s,c])+sum(cost_param["c"][0]*X_of[t,s] for s in states)for t in  range(T1-1,T2-1))==cost)
    model1.setObjective(-cost, GRB.MAXIMIZE)
    model1.Params.TIME_LIMIT = 7200
    # model1.Params.MIPGap=1
    model1.setParam(GRB.Param.FlowCoverCuts, 2)
    model1.setParam(GRB.Param.NetworkCuts, 2)
    model1.Params.ScaleFlag=2
    model1.optimize()
    model1.getObjective()
    if model1.Status == GRB.OPTIMAL:
        for s in states:
            for t in  range(T1-1,T2-1):
                for c in counties2[s]:
                    Vac_all[t,s,c] = Vac0[t,s,c].x
    elif model1.Status == GRB.INFEASIBLE:
        for s in states:
            for t in  range(T1-1,T2-1):
                for c in counties2[s]:
                    Vac_all[t,s,c] = "nan"
    if model1.Status == GRB.OPTIMAL:
        for s in states:
            for t in  range(T1-1,T2-1):
                Vac_avl[t,s] = sum(Vac0[t,s,c].x for c in counties2[s])
    elif model1.Status == GRB.INFEASIBLE:
        for s in states:
            for t in  range(T1-1,T2-1):
                Vac_avl[t,s] = "nan"
    if model1.Status == GRB.INFEASIBLE:
        cost[T1]= "nan"
    #
    else:
        cost[T1]= sum(sum(cost_param["w"][0]*W1[t,s].x for s in states )+sum(cost_param["w"][1]*W2[t,s,c].x for s in states for c in counties2[s])+sum(cost_param["w"][1]*W3[t,s,c,l].x for s in states for c in counties2[s] for l in local_centers[s,c])+sum(distance[s]/2000*cost_param["g"][1]*G2[t,s,c].x for s in states for c in counties2[s] )+sum(cost_param["c"][1]*G1[t,1,s].x+cost_param["c"][2]*G1[t,2,s].x for s in states )+sum(cost_param["w"][1]*G3[t,s,c,l].x for s in states for c in counties2[s] for l in local_centers[s,c])+sum(cost_param["c"][0]*X_of[t,s] for s in states)for t in  range(T1-1,T2-1))
    for i in suppliers:
        available_at_supplier[T1,i]=sup_avl[T1,i].x
    for s in states:
        for t in range(T1-1,T2-1):
            if model1.Status == GRB.OPTIMAL:
                inventory[t,s]=W1[t,s].x
            elif model1.Status == GRB.INFEASIBLE:
                inventory[t,s]="nan"
    #inventory
    if model1.Status == GRB.OPTIMAL:
        for t in range(T1-1,T2-1):
            for s in states:
                W1_p[t,s]=W1[t,s].x
                for c in counties2[s]:
                    W2_p[t,s,c]=W2[t,s,c].x
                    for l in local_centers[s,c]:
                        W3_p[t,s,c,l]=W3[t,s,c,l].x
    for t in range(T1-1,T2-1):
        for s in states:
            if model1.Status == GRB.OPTIMAL:
                G1_supply[t,1,s]= G1[t,1,s].x
                G1_supply[t,2,s]= G1[t,2,s].x
                G1_supply[t,3,s]= G1[t,3,s].x
            elif model1.Status == GRB.INFEASIBLE:
                G1_supply[t,1,s]="nan"
                G1_supply[t,2,s]="nan"
                G1_supply[t,3,s]="nan"
    for t in range(T1-1,T2-1):
        for s in states:
            for c in counties2[s]:
                if model1.Status == GRB.OPTIMAL:
                    G2_supply[t,s,c]= G2[t,s,c].x
                elif model1.Status == GRB.INFEASIBLE:
                    G2_supply[t,s,c]="nan"
    sol_time0[T1]=model1.Runtime
    return inventory,available_at_supplier


budget_inf={}
for t in range(-1,T+4):
    budget_inf[t]=0
final_cost=0
Vac_coun={}
Vac_state={}
S01={}
I01={}
V01={}
R01={}
opening={}
opening_f={}
for s in states:
    for t in  range(-3,1):
        opening_f[t,s]=0
        opening[t,s]=0
for s in states:
    S01[2,s]=S0101[0,s]/10000
    I01[2,s]=I0101[0,s]/10000
    R01[2,s]=0.95*2*states_cases0[s][23]/10000
    V01[2,s]=0
    Vac[1,s]=0
    upper[1,s]=1
T2=3
T1=2
a=1
for s in states:
    upper[T1-1,s]=1
    for c in counties2[s]:
        upper0[T1-1,s,c]=1
vac_alloc(I01,S01,V01,R01,T1,T2,available_at_supplier,a,lam)
for s in states:
    for t in  range(T1-1,T2-1):
        opening[t,s]=U_I[t,s]
        opening_f[t,s]=X_of[t,s]
        Vac_state[t,s]=Vac_i[t,s]
n=0
supply={}
activate=0
while ((T2<T+1)):
    T1+=1
    T2+=1
    for s in states:
        upper[T1-1,s]=1
        if s=='TX':
            upper[T1-1,s]=(popplus12_s[s]/10000-sum(Vac_state[t,s] for t in range(1,T1-1)))/(popplus12_s[s]/10000)
            if upper[T1-1,s]<=0:
                upper[T1-1,s]=0
        else:
            upper[T1-1,s]=(popplus12_s[s]/10000-sum(Vac_state[t,s] for t in range(1,T1-1)))/(popplus12_s[s]/10000)
            if upper[T1-1,s]<=0:
                upper[T1-1,s]=0
    if budget_inf[n-1]==0:
        for s in states:
            S01[T1,s]=S1[T1,s]
            I01[T1,s]=I1[T1,s]
            V01[T1,s]=V1[T1,s]
            R01[T1,s]=R1[T1,s]
    else:
        for s in states:
            S01[T1,s]=(budget_inf[n-1])*S01[T1-1,s]
            I01[T1,s]=(budget_inf[n-1])*I01[T1-1,s]
            V01[T1,s]=(budget_inf[n-1])*V01[T1-1,s]
            R01[T1,s]=(budget_inf[n-1])*R01[T1-1,s]

    vac_alloc(I01,S01,V01,R01,T1,T2,available_at_supplier,a,lam)
    print("* \n vac_alloc solved\n ",T1)
    for s in states:
        for t in  range(T1-1,T2-1):
            Vac_state[t,s]=Vac_i[t,s]
            opening[t,s]=U_I[t,s]
    n+=1
    if n>=14:
        break

if activate==1:
    upper[T1-2,s]=0
    for s in states:
        S1[T1+1,s]=S01[T1,s]
        I1[T1+1,s]=I01[T1,s]
        V1[T1+1,s]=V01[T1,s]
        R1[T1+1,s]=R01[T1,s]
    for i in suppliers:
        available_at_supplier[T1,i]=0
    while ((T2<T+1)):
        T1+=1
        T2+=1
        for i in suppliers:
            available_at_supplier[T1,i]=0
        for s in states:
            upper[T1-1,s]=0
        for s in states:
            opening_f[T1-1,s]=0
            Vac_state[T1-1,s]=0
            S01[T1,s]=S1[T1,s]
            I01[T1,s]=I1[T1,s]
            V01[T1,s]=V1[T1,s]
            R01[T1,s]=R1[T1,s]
        vac_alloc(I01,S01,V01,R01,T1,T2,available_at_supplier,a,lam)
state_cost={}
Pi_adjusted={}
T1=2
T2=12
states_new=[]
for i in [1,2,3]:
    for t in range(2,T+1):
        Pi_adjusted[t-2,i]=Pi[t-2,i]
state_alloc(Vac_state,inf,X_of,2,13,W1_p,W2_p,W3_p,states)
state_cost={}
ub_state={}
sol_time_state={}
def state_alloc_gini(G1_supply,states):
    global inventory
    model1 = gp.Model()
    cost=model1.addVar(lb=0,ub=B[0])
    for s in states:
        Gini_Coef[s] = model1.addVar( lb=0, ub=1, vtype=GRB.CONTINUOUS)
        Y_b[s] = model1.addVar(lb=0, vtype=GRB.BINARY)
        # cost[s] = model1.addVar(lb=0, vtype=GRB.CONTINUOUS)
        # for t in range(2,T+1):
        #     U_I[t,s] = model1.addVar(vtype=GRB.BINARY)
        for t in range(T1-2,T2):
            lb0[t,s]=model1.addVar(lb=0,vtype=GRB.CONTINUOUS)
            W1[t,s] = model1.addVar( lb=0, vtype=GRB.CONTINUOUS)
            # for i in [1,2,3]:
            #     G1[t,i,s] = model1.addVar( lb=0, vtype=GRB.CONTINUOUS)
    W2 = {(t, s, c): model1.addVar(lb=0, vtype=GRB.CONTINUOUS) for t in range(T1-2,T2) for s in states for c in counties2[s]}
    G2 = {(t, s, c): model1.addVar(lb=0, vtype=GRB.CONTINUOUS) for t in range(T1-1,T2) for s in states for c in counties2[s]}
    G3 = {(t, s, c,l): model1.addVar(lb=0, vtype=GRB.CONTINUOUS) for t in range(T1-1,T2) for s in states for c in counties2[s] for l in local_centers[s,c]}
    W3 = {(t, s, c,l): model1.addVar(lb=0, vtype=GRB.CONTINUOUS) for t in range(T1-2,T2) for s in states for c in counties2[s] for l in local_centers[s,c]}
    Vac0 = {(t, s,c): model1.addVar(lb=0, ub=popplus12[s,c]/10000, vtype=GRB.CONTINUOUS) for t in range(T1-1,T2-1) for s in states for c in counties2[s]}
    ub=model1.addVar(lb=0,ub=1)
    y=model1.addVar(lb=0,vtype=GRB.CONTINUOUS)
    sup_avl={(t,i): model1.addVar(lb=0, vtype=GRB.CONTINUOUS) for t in range(T1-2,T2) for i in suppliers}

    for t in range(T1-1,T2-1):
        for s in states:
            for c in counties2[s]:
                model1.addConstr(Vac0[t,s,c]<=county_cap[c])

    for t in range(T1-1,T2-1):
        for s in states:
            model1.addConstr(Vac_state[t,s]==(gp.quicksum(Vac0[t,s,c] for c in counties2[s]))+(1-wastage)*X_of[t,s])
    for s in states:
        for t in range(T1-1,T2-1):
            model1.addConstr(gp.quicksum(G1_supply[t,i,s] for i in suppliers)+W1[t-1,s] == gp.quicksum(G2[t,s,c] for c in counties2[s] )+X_of[t,s]*capacity+W1[t,s])
            # model.addConstr(gp.quicksum(G3[t,h_s,j] for h_s in statehubs)*1/21==gp.quicksum(G4[t,j,l]for l in localcenters[j]))
        model1.addConstr(W1[T1-1,s]==0)
    for t in range(T1-1,T2-1):
        for s in states:
            for c in counties2[s]:
                model1.addConstr (G2[t,s,c]  + W2[t-1,s,c]== gp.quicksum(G3[t,s,c,l] for l in local_centers[s,c])+ W2[t,s,c] )
                model1.addConstr(W2[T1-1,s,c]==0)
    for t in range(T1-1,T2-1):
        for s in states:
            for c in counties2[s]:
                model1.addConstr((1-wastage)*gp.quicksum(G3[t,s,c,l]+W3[t-1,s,c,l] for l in local_centers[s,c])==Vac0[t,s,c]+gp.quicksum(W3[t,s,c,l] for l in local_centers[s,c]))
                for l in local_centers[s,c]:
                    model1.addConstr(W3[T1-1,s,c,l]==0)
    for s in states:
        for c in counties2[s]:
            for cc in counties2[s]:
                u3[s,c,cc] = model1.addVar(lb=0, vtype=GRB.CONTINUOUS)
        for c in counties2[s]:
            u2[s,c] = model1.addVar(lb=0, vtype=GRB.CONTINUOUS)
    for s in states:
        for c in counties2[s]:
            model1.addConstr(u2[s, c] == (sum(Vac0[t,s,c] for t in  range(T1-1,T2-1))/ (popplus12[s,c]/10000)))
        # Constraints to linearize the absolute value
    for s in states:
        for c in counties2[s]:
            for cc in counties2[s]:
                model1.addConstr(u3[s,c,cc] >= u2[s, c] - u2[s,cc])
                model1.addConstr(u3[s, c,cc] >= -(u2[s,c] - u2[s,cc]))
        # Example objective function (minimizing the sum of absolute differences)
        u_bar1[s] = 1/len(counties2[s])*gp.quicksum(u2[s,c] for c in counties2[s])
        # for c in counties[s]:
        model1.addConstr(gp.quicksum(u3[s,c,cc] for c in counties2[s] for cc in counties2[s]) == 2*Gini_Coef[s]*(u_bar1[s])*len(counties2[s])*len(counties2[s]))
        model1.addConstr(Gini_Coef[s] <= ub)
    # Set optimization parameters
    model1.setObjective(-ub, GRB.MAXIMIZE)
    model1.Params.TIME_LIMIT = 7200
    # model1.Params.MIPGap=1
    model1.setParam(GRB.Param.FlowCoverCuts, 2)
    model1.setParam(GRB.Param.NetworkCuts, 2)
    model1.Params.ScaleFlag=2
    model1.optimize()
    model1.getObjective()
    ub_state[s]=ub.x
    state_cost[s]=sum(sum(cost_param["w"][0]*W1[t,s].x for s in states )+sum(cost_param["w"][1]*W2[t,s,c].x for s in states for c in counties2[s])+sum(cost_param["w"][1]*W3[t,s,c,l].x for s in states for c in counties2[s] for l in local_centers[s,c])+sum(distance[s]/2000*cost_param["g"][1]*G2[t,s,c].x for s in states for c in counties2[s] )+sum(cost_param["c"][1]*G1_supply[t,1,s]+cost_param["c"][2]*G1_supply[t,2,s]+cost_param["c"][3]*G1_supply[t,3,s] for s in states )+sum(cost_param["w"][1]*G3[t,s,c,l].x for s in states for c in counties2[s] for l in local_centers[s,c])+sum(cost_param["c"][0]*X_of[t,s] for s in states)for t in  range(T1-1,T2-1))
    sol_time_state[s]=model1.Runtime
T1=2
T2=12
for s in states:
    state_alloc_gini(G1_supply,[s])

In [ ]:

sol_time={}
sol_time0={}
Vac_avl={}
for s in states:
    Vac_avl[1,s]=0
dic={}
for t in time:
    for s in states:
        dic[t,s]=None
results1=pd.DataFrame.from_dict(dic, orient='index')
dic1={}
for t in time:
    for s in states:
        for c in counties2[s]:
            dic1[t,s]=None
results2=pd.DataFrame.from_dict(dic1, orient='index')
global T1,T2
Scale=10000
global lower_bound
effec=0.2
inventory={}
opening={}
for s in states:
    #for c in counties2[s]:
    inventory[0,s]=0
upper={}
upper0={}
T=12
S={}
V={}
I={}
R={}
Vac={}
Vac_1={}
Vac_2={}
inf={}
# X0={}
U_I={}
U_D={}
reinf={}
Results={}
lb={}

Gini_Coef = {}
G1 = {}
W1 = {}
W2 = {}
G2 = {}
W3 = {}
G3 = {}
Y = {}
X_o={}
Y_b = {}
X_of = {}
cost = {}
capacity=10000/10000
Y_bn=[]
# results["Vac"]=None
Vac_all={}
G1_supply={}
G2_supply={}
sup_avl={}
state_cost={}
# inventory={}
lb0={}
#Retrieval of results
W1_p={}
W2_p={}
W3_p={}
tau={}
for t in range(0,2):
    for s in states:
        W1_p[t,s]=0
        for c in counties2[s]:
            W2_p[t,s,c]=0
            for l in local_centers[s,c]:
                W3_p[t,s,c,l]=0
available_at_supplier={}
for i in suppliers:
    available_at_supplier[1,i]=0
bud_ava={}
model0 = gp.Model()

for t in range(T1-1,T2-1):
    lb[t]= model0.addVar( lb=0, vtype=GRB.CONTINUOUS)
# global Vac, U_I
# Define parameters

for s in states:

    for t in range(T1,T2+1):
        S[t,s] = model0.addVar( lb=0, ub=popplus12_s[s]/10000 ,vtype=GRB.CONTINUOUS)
        V[t,s] = model0.addVar( lb=0, ub=popplus12_s[s]/10000 , vtype=GRB.CONTINUOUS)
        I[t,s] = model0.addVar( lb=0, ub=popplus12_s[s]/10000 ,vtype=GRB.CONTINUOUS)
        R[t,s] = model0.addVar( lb=0, ub=popplus12_s[s]/10000 ,vtype=GRB.CONTINUOUS)
        reinf[t,s]= model0.addVar( lb=0,ub=popplus12_s[s]/10000 ,vtype=GRB.CONTINUOUS)
        tau[t,s]= model0.addVar( lb=0,ub=popplus12_s[s] ,vtype=GRB.CONTINUOUS)
    for t in range(T1-1,T2):
        Vac_1[t,s] = model0.addVar(lb=0, ub=popplus12_s[s]/10000,vtype=GRB.CONTINUOUS)
        Vac_2[t,s] = model0.addVar(lb=0, ub=popplus12_s[s]/10000,vtype=GRB.CONTINUOUS)
        Vac[t,s] = model0.addVar(lb=0, ub=popplus12_s[s]/10000,vtype=GRB.CONTINUOUS)
        # Vac[t,s] = model0.addVar(lb=upper[t,s]*opening_f[t-1,s], ub=upper[t,s]*popplus12_s[s]/10000,vtype=GRB.CONTINUOUS)
    for t in range(T1-1,T2):
        U_I[t,s] = model0.addVar(vtype=GRB.BINARY)
        U_D[t,s] = model0.addVar(vtype=GRB.BINARY)
X_o = {(t, s): model0.addVar(lb=0, vtype=GRB.BINARY)  for t in  range(T1-1,T2) for s in states}
for s in states:
    Gini_Coef[s] = model0.addVar( lb=0, ub=1, vtype=GRB.CONTINUOUS)
    Y_b[s] = model0.addVar(lb=0, vtype=GRB.BINARY)
    # cost[s] = model0.addVar(lb=0, vtype=GRB.CONTINUOUS)
    # for t in range(2,T+1):
    #     U_I[t,s] = model0.addVar(vtype=GRB.BINARY)
    for t in range(T1-2,T2):
        lb0[t,s]=model0.addVar(lb=0,vtype=GRB.CONTINUOUS)
        W1[t,s] = model0.addVar( lb=0, vtype=GRB.CONTINUOUS)
        for i in [1,2,3]:
            G1[t,i,s] = model0.addVar( lb=0, vtype=GRB.CONTINUOUS)
W2 = {(t, s, c): model0.addVar(lb=0, vtype=GRB.CONTINUOUS) for t in range(T1-2,T2) for s in states for c in counties2[s]}
G2 = {(t, s, c): model0.addVar(lb=0, vtype=GRB.CONTINUOUS) for t in range(T1-1,T2) for s in states for c in counties2[s]}
G3 = {(t, s, c,l): model0.addVar(lb=0, vtype=GRB.CONTINUOUS) for t in range(T1-1,T2) for s in states for c in counties2[s] for l in local_centers[s,c]}
W3 = {(t, s, c,l): model0.addVar(lb=0, vtype=GRB.CONTINUOUS) for t in range(T1-2,T2) for s in states for c in counties2[s] for l in local_centers[s,c]}
Vac0 = {(t, s,c): model0.addVar(lb=0, ub=popplus12[s,c]/10000, vtype=GRB.CONTINUOUS) for t in range(T1-1,T2) for s in states for c in counties2[s]}
ub=model0.addVar(lb=0,ub=1)
y=model0.addVar(lb=0,vtype=GRB.CONTINUOUS)
sup_avl={(t,i): model0.addVar(lb=0, ub=Pi[t-1,i], vtype=GRB.CONTINUOUS) for t in range(T1-1,T2) for i in suppliers}
C=model0.addVar(lb=0)
cost={}
for s in states:
    cost[s] = model0.addVar(lb=0, vtype=GRB.CONTINUOUS)
for s in states:
    model0.addConstr(S[2, s] ==S0101[0,s]/10000)
    model0.addConstr(V[2, s] == 0)
    model0.addConstr(I[2, s] == I0101[0,s]/10000)
    model0.addConstr(R[2, s] == 0.95*2*states_cases0[s][23]/10000)
    for t in range(T1,T2):
        model0.addConstr(S[t+1,s] == reinf[t,s]+(1-mu)*S[t,s] +mu - 10000*beta[t,s]*I[t,s]*S[t,s]-Vac_1[t-gamma1,s])
        model0.addConstr(V[t+1,s] == V[t,s] +Vac_2[t-gamma1,s]+Vac_1[t-gamma1,s] -effec*10000*beta[t,s]*V[t,s]*I[t,s] -mu*V[t,s])
        model0.addConstr(I[t+1,s] == I[t,s] + 10000*beta[t,s]*S[t,s] *I[t,s]+effec*10000*beta[t,s]*V[t,s]*I[t,s]-gamma*I[t,s]-mu*I[t,s])
        model0.addConstr(R[t+1,s] == R[t,s] + +gamma*I[t,s]-mu*R[t,s]-Vac_2[t-gamma1,s])
        model0.addConstr(Vac[t-gamma1,s]==Vac_1[t-gamma1,s]+Vac_2[t-gamma1,s])
    if t>=(reinf_period+3):
        model0.addConstr(reinf[t,s]==(reinf_rate)*(10000*beta[t-reinf_period-1,s]*S[t-reinf_period-1,s] *I[t-reinf_period-1,s]-mu*I[t-reinf_period-1,s]))
    else:
        model0.addConstr(reinf[t,s]==reinf_rate*underreported[s,t+21-reinf_period]*(states_cases0[s][t+22-reinf_period]-states_cases0[s][t+21-reinf_period])/10000)
    if t>=5:
        model0.addConstr(tau[t,s]==sum(10000*beta[ti,s]*S[ti,s] *I[ti,s]+effec*10000*beta[ti,s]*V[ti,s]*I[ti,s] for ti in range(t-3,t))/3)
    else:
        model0.addConstr(tau[t,s]==0.5*beta[2,s]*popplus12_s[s]/Scale)
#                 model0.addConstr((Vac[t,s])<=0.4*sum(county_cap[c] for c in counties2[s]))
for t in range(T1-1,T2):
    for s in states:
        model0.addConstr(Vac[t,s]<=1*est_demand[t,s])
        model0.addConstr(Vac[t,s]<=sum(county_cap[c] for c in counties2[s])+X_o[t,s])
for s in states:
    for t in range(T1,T2):
        model0.addConstr(Vac[t-1,s]/sum(popplus12[s,c]for c in counties2[s])*10000>=lb[t-1])
for s in states:
    for t in range(T1,T2):
        model0.addConstr((tau[t,s] - M*(1-U_I[t-1,s]))<=Scale*beta[t,s] * S[t, s] * I[t, s] + effec*Scale*beta[t,s] *  V[t, s] * I[t, s])
        model0.addConstr((tau[t,s] + M*(U_I[t-1,s]) )>=Scale*beta[t,s] * S[t, s] * I[t, s] + effec*Scale*beta[t,s] *  V[t, s] * I[t, s] )
for s in states:
    for t in range(T1,T2):
        model0.addConstr((sum(county_cap[c] for c in counties2[s])*(T2-T1) - M*(1-U_D[t-1,s]))<=est_demand[t,s])
        model0.addConstr((sum(county_cap[c] for c in counties2[s])*(T2-T1))+ M*(U_D[t-1,s]) >=est_demand[t,s] )
for t in range(T1-1,T2):
    for s in states:
        model0.addConstr(X_o[t,s]<= U_I[t,s]+U_D[t,s])
        model0.addConstr(X_o[t,s]>= (U_I[t,s])-(C)/B[0])
        model0.addConstr(X_o[t,s]>= (U_D[t,s])-(C)/B[0])
for t in range(T1-1,T2):
    for s in states:
        for c in counties2[s]:
            model0.addConstr(Vac0[t,s,c]<=county_cap[c])

for t in range(T1-1,T2):
    for s in states:
        model0.addConstr(Vac[t,s]==(gp.quicksum(Vac0[t,s,c] for c in counties2[s]))+(1-wastage)*X_o[t,s])
for t in range(T1-1,T2):
    for i in suppliers:
        model0.addConstr(gp.quicksum(G1[t,i,s] for s in states)<=Pi[t-1,i])
# for s in states:
#     for t in range(T1-1,T2-1):
#         model0.addConstr(gp.quicksum(G1[t,i,s] for i in suppliers)+W1[t-1,s] == Vac_i[t,s])

for s in states:
    for t in range(T1-1,T2):
        model0.addConstr(gp.quicksum(G1[t,i,s] for i in suppliers)+W1[t-1,s] == gp.quicksum(G2[t,s,c] for c in counties2[s] )+X_o[t,s]*capacity+W1[t,s])
        # model.addConstr(gp.quicksum(G3[t,h_s,j] for h_s in statehubs)*1/21==gp.quicksum(G4[t,j,l]for l in localcenters[j]))
    model0.addConstr(W1[0,s]==0)
for t in range(T1-1,T2):
    for s in states:
        for c in counties2[s]:
            model0.addConstr (G2[t,s,c]  + W2[t-1,s,c]== gp.quicksum(G3[t,s,c,l] for l in local_centers[s,c])+ W2[t,s,c] )
            model0.addConstr(W2[0,s,c]==0)
for t in range(T1-1,T2):
    for s in states:
        for c in counties2[s]:
            model0.addConstr((1-wastage)*gp.quicksum(G3[t,s,c,l]+W3[t-1,s,c,l] for l in local_centers[s,c])==Vac0[t,s,c]+gp.quicksum(W3[t,s,c,l] for l in local_centers[s,c]))
            for l in local_centers[s,c]:
                model0.addConstr(W3[0,s,c,l]==0)
                # model0.addConstr(W3[-1,s,c,l]==0)
for t in range(T1-1,T2):
    for s in states:
        for c in counties2[s]:
            # model0.addConstr(Vac0[t,s,c] <=county_cap[c])
            model0.addConstr(Vac0[t,s,c]/popplus12[s,c]*10000 >= lb0[t,s])
# if T1>=3:
#     for t in range(T1-1,T2-1):
#         for s in states:
model0.addConstr(sum(sum(cost_param["w"][0]*W1[t,s] for s in states )+sum(cost_param["w"][1]*W2[t,s,c] for s in states for c in counties2[s])+sum(cost_param["w"][2]*W3[t,s,c,l] for s in states for c in counties2[s] for l in local_centers[s,c])+sum(distance[s]/2000*cost_param["g"][1]*G2[t,s,c] for s in states for c in counties2[s] )+sum(cost_param["c"][1]*G1[t,1,s]+cost_param["c"][2]*G1[t,2,s] +cost_param["c"][3]*G1[t,3,s] for s in states )+sum(cost_param["w"][3]*G3[t,s,c,l] for s in states for c in counties2[s] for l in local_centers[s,c])+sum(cost_param["c"][0]*X_o[t,s] for s in states)for t in  range(T1-1,T2-1))==C)

model0.addConstr(C<=B[0])
model0.setObjective(-1*gp.quicksum(access[s]*(sum(weights[s,c]*(Vac0[t,s,c])+lb0[t,s]for c in counties2[s]))+ 10*X_o[t,s] for s in states for t in range(T1-1,12))+50*gp.quicksum(gp.quicksum((10000*beta[t,s] * S[t, s] * I[t, s] + effec*10000*beta[t,s] *  V[t, s] * I[t, s]) for s in states)for t in range(2, 12))-300*sum(1*lb[t-1] for t in range(T1, T2)), GRB.MINIMIZE)
# model0.setObjective(sum(I[T2,s] for s in states) , GRB.MINIMIZE)
# Set optimization parameters
#
model0.Params.TIME_LIMIT = 3600*5

model0.Params.ScaleFlag=2
model0.optimize()
model0.getObjective()

if model0.Status == GRB.OPTIMAL:
    for s in states:
        for t in  range(T1-1,T2-1):
            for c in counties2[s]:
                Vac_all[t,s,c] = Vac0[t,s,c].x
elif model0.Status == GRB.INFEASIBLE:
    for s in states:
        for t in  range(T1-1,T2-1):
            for c in counties2[s]:
                Vac_all[t,s,c] = "nan"
if model0.Status == GRB.OPTIMAL:
    for s in states:
        for t in  range(T1-1,T2-1):
            Vac_avl[t,s] = sum(Vac0[t,s,c].x for c in counties2[s])
elif model0.Status == GRB.INFEASIBLE:
    for s in states:
        for t in  range(T1-1,T2-1):
            Vac_avl[t,s] = "nan"
if model0.Status == GRB.INFEASIBLE:
    cost[T1]= "nan"

else:
    cost[T1]= sum(sum(cost_param["w"][0]*W1[t,s].x for s in states )+sum(cost_param["w"][1]*W2[t,s,c].x for s in states for c in counties2[s])+sum(cost_param["w"][1]*W3[t,s,c,l].x for s in states for c in counties2[s] for l in local_centers[s,c])+sum(distance[s]/2000*cost_param["g"][1]*G2[t,s,c].x for s in states for c in counties2[s] )+sum(cost_param["c"][1]*G1[t,1,s].x+cost_param["c"][2]*G1[t,2,s].x+cost_param["c"][3]*G1[t,3,s].x for s in states )+sum(cost_param["w"][1]*G3[t,s,c,l].x for s in states for c in counties2[s] for l in local_centers[s,c])+sum(cost_param["c"][0]*X_o[t,s].x for s in states)for t in  range(T1-1,T2-1))
# for i in suppliers:
#     available_at_supplier[T1,i]=sup_avl[T1-1,i].x
for s in states:
    for t in range(T1-1,T2-1):
        #for c in counties2[s]:
        if model0.Status == GRB.OPTIMAL:
            #inventory[t,s]= sum(W2[t,s,c].x for c in counties2[s])
            inventory[t,s]=W1[t,s].x
        elif model0.Status == GRB.INFEASIBLE:
            inventory[t,s]="nan"
#inventory
if model0.Status == GRB.OPTIMAL:
    for t in range(T1-1,T2-1):
        for s in states:
            W1_p[t,s]=W1[t,s].x
            for c in counties2[s]:
                W2_p[t,s,c]=W2[t,s,c].x
                for l in local_centers[s,c]:
                    W3_p[t,s,c,l]=W3[t,s,c,l].x


#Vaccination
# for t in range(T1-1,T2-1):
#     if model0.Status == GRB.OPTIMAL:
#         results1[f"t/s"]=Vac[t,s]
#     elif model0.Status == GRB.INFEASIBLE:
#         results1[f"t/s"]="nan"
#Supply from suppliers
for t in range(T1-1,T2):
    for s in states:
        if model0.Status == GRB.OPTIMAL:
            G1_supply[t,1,s]= G1[t,1,s].x
            G1_supply[t,2,s]= G1[t,2,s].x
            G1_supply[t,3,s]= G1[t,3,s].x
        elif model0.Status == GRB.INFEASIBLE:
            G1_supply[t,1,s]="nan"
            G1_supply[t,2,s]="nan"
            G1_supply[t,3,s]="nan"
for t in range(T1-1,T2-1):
    for s in states:
        for c in counties2[s]:
            if model0.Status == GRB.OPTIMAL:
                G2_supply[t,s,c]= G2[t,s,c].x
            elif model0.Status == GRB.INFEASIBLE:
                G2_supply[t,s,c]="nan"

sol_time0[T1]=model0.Runtime

model0.setParam(GRB.Param.Heuristics, 0)
inf={(t,s): beta[t,s] * S[t, s].x * I[t, s].x + effec*beta[t,s] *  V[t, s].x * I[t, s].x for t in range(T1, T2) for s in states  }
U_I={(t, s): U_I[t, s].x for t in range(T1-1,T2-1)  for s in states  }
U_D={(t, s): U_D[t, s].x for t in range(T1-1,T2-1)  for s in states  }
# Vac_r={}    # for s in states:
#     vac_r[T1,s]=Vac[t, s].x/sum(Vac[T1, s].x for s in states)
#     for t in range(T1+1,T2):
#         vac_r[t,s]=0
Vac_i={(t, s): Vac[t, s].x for t in range(T1-1,T2-1)  for s in states }
vac_r={}
for s in states:
    for t in range(T1,T2):
        if sum(Vac[t-1, s].x for s in states)==0:
            vac_r[t-1,s]=0
        else:
            vac_r[t-1,s]=(Vac[t-1, s]).x/sum(Vac[t-1, s].x for s in states)
S1={(t, s): S[t, s].x for t in range(T1+1,T2+1)  for s in states }
I1={(t, s): I[t, s].x for t in range(T1+1,T2+1)  for s in states }
V1={(t, s): V[t, s].x for t in range(T1+1,T2+1) for s in states }
R1={(t, s): R[t, s].x for t in range(T1+1,T2+1) for s in states }
Vac_df=pd.DataFrame({"States":states})

sol_time[T1]=model0.Runtime
if model0.Status == GRB.OPTIMAL:
    for s in states:
        for t in  range(T1-1,T2-1):
            X_of[t,s] = X_o[t,s].x
elif model0.Status == GRB.INFEASIBLE:
    for s in states:
        for t in  range(T1-1,T2-1):
           X_of[t,s] = "nan"
print(sum(X_o[t,s].x for s in states for t in range(1,12)))
for t in range(1,11):
    print(s,t,lb[t].x)
for s in states:
    for t in range(2,12):
        print(10000*(beta[t,s]*S[t,s].x*I[t,s].x+effec*beta[t,s]*V[t,s].x*I[t,s].x ),tau[t,s])

In [ ]:

sol_time={}
sol_time0={}
Vac_avl={}
for s in states:
    Vac_avl[1,s]=0
dic={}
for t in time:
    for s in states:
        dic[t,s]=None
results1=pd.DataFrame.from_dict(dic, orient='index')
dic1={}
for t in time:
    for s in states:
        for c in counties2[s]:
            dic1[t,s]=None
results2=pd.DataFrame.from_dict(dic1, orient='index')
global T1,T2
Scale=10000
global lower_bound
effec=0.2
inventory={}
opening={}
for s in states:
    #for c in counties2[s]:
    inventory[0,s]=0
upper={}
upper0={}
T=12
S={}
V={}
I={}
R={}
Vac={}
# S1={}
# V1={}
# I1={}
# R1={}
Vac_1={}
Vac_2={}
inf={}
# X0={}
U_I={}
U_D={}
reinf={}
Results={}
lb={}

Gini_Coef = {}
G1 = {}
W1 = {}
W2 = {}
G2 = {}
W3 = {}
G3 = {}
Y = {}
X_o={}
Y_b = {}
X_of = {}
cost = {}
capacity=10000/10000
Y_bn=[]
# results["Vac"]=None
Vac_all={}
G1_supply={}
G2_supply={}
sup_avl={}
state_cost={}
# inventory={}
lb0={}
#Retrieval of results
W1_p={}
W2_p={}
W3_p={}
tau={}
for t in range(0,2):
    for s in states:
        W1_p[t,s]=0
        for c in counties2[s]:
            W2_p[t,s,c]=0
            for l in local_centers[s,c]:
                W3_p[t,s,c,l]=0
available_at_supplier={}
for i in suppliers:
    available_at_supplier[1,i]=0
bud_ava={}
model0 = gp.Model()

for t in range(T1-1,T2-1):
    lb[t]= model0.addVar( lb=0, vtype=GRB.CONTINUOUS)
# global Vac, U_I
# Define parameters

for s in states:

    for t in range(T1,T2+1):
        S[t,s] = model0.addVar( lb=0, ub=popplus12_s[s]/10000 ,vtype=GRB.CONTINUOUS)
        V[t,s] = model0.addVar( lb=0, ub=popplus12_s[s]/10000 , vtype=GRB.CONTINUOUS)
        I[t,s] = model0.addVar( lb=0, ub=popplus12_s[s]/10000 ,vtype=GRB.CONTINUOUS)
        R[t,s] = model0.addVar( lb=0, ub=popplus12_s[s]/10000 ,vtype=GRB.CONTINUOUS)
        reinf[t,s]= model0.addVar( lb=0,ub=popplus12_s[s]/10000 ,vtype=GRB.CONTINUOUS)
        tau[t,s]= model0.addVar( lb=0,ub=popplus12_s[s] ,vtype=GRB.CONTINUOUS)
    for t in range(T1-1,T2):
        Vac_1[t,s] = model0.addVar(lb=0, ub=popplus12_s[s]/10000,vtype=GRB.CONTINUOUS)
        Vac_2[t,s] = model0.addVar(lb=0, ub=popplus12_s[s]/10000,vtype=GRB.CONTINUOUS)
        Vac[t,s] = model0.addVar(lb=0, ub=popplus12_s[s]/10000,vtype=GRB.CONTINUOUS)
        # Vac[t,s] = model0.addVar(lb=upper[t,s]*opening_f[t-1,s], ub=upper[t,s]*popplus12_s[s]/10000,vtype=GRB.CONTINUOUS)
    for t in range(T1-1,T2):
        U_I[t,s] = model0.addVar(vtype=GRB.BINARY)
        U_D[t,s] = model0.addVar(vtype=GRB.BINARY)
X_o = {(t, s): model0.addVar(lb=0, vtype=GRB.BINARY)  for t in  range(T1-1,T2) for s in states}
for s in states:
    Gini_Coef[s] = model0.addVar( lb=0, ub=1, vtype=GRB.CONTINUOUS)
    Y_b[s] = model0.addVar(lb=0, vtype=GRB.BINARY)
    # cost[s] = model0.addVar(lb=0, vtype=GRB.CONTINUOUS)
    # for t in range(2,T+1):
    #     U_I[t,s] = model0.addVar(vtype=GRB.BINARY)
    for t in range(T1-2,T2):
        lb0[t,s]=model0.addVar(lb=0,vtype=GRB.CONTINUOUS)
        W1[t,s] = model0.addVar( lb=0, vtype=GRB.CONTINUOUS)
        for i in [1,2,3]:
            G1[t,i,s] = model0.addVar( lb=0, vtype=GRB.CONTINUOUS)
W2 = {(t, s, c): model0.addVar(lb=0, vtype=GRB.CONTINUOUS) for t in range(T1-2,T2) for s in states for c in counties2[s]}
G2 = {(t, s, c): model0.addVar(lb=0, vtype=GRB.CONTINUOUS) for t in range(T1-1,T2) for s in states for c in counties2[s]}
G3 = {(t, s, c,l): model0.addVar(lb=0, vtype=GRB.CONTINUOUS) for t in range(T1-1,T2) for s in states for c in counties2[s] for l in local_centers[s,c]}
W3 = {(t, s, c,l): model0.addVar(lb=0, vtype=GRB.CONTINUOUS) for t in range(T1-2,T2) for s in states for c in counties2[s] for l in local_centers[s,c]}
Vac0 = {(t, s,c): model0.addVar(lb=0, ub=popplus12[s,c]/10000, vtype=GRB.CONTINUOUS) for t in range(T1-1,T2) for s in states for c in counties2[s]}
ub=model0.addVar(lb=0,ub=1)
y=model0.addVar(lb=0,vtype=GRB.CONTINUOUS)
sup_avl={(t,i): model0.addVar(lb=0, ub=Pi[t-1,i], vtype=GRB.CONTINUOUS) for t in range(T1-1,T2) for i in suppliers}
C=model0.addVar(lb=0)

cost={}
for s in states:
    cost[s] = model0.addVar(lb=0, vtype=GRB.CONTINUOUS)
for s in states:
    model0.addConstr(S[2, s] ==S0101[0,s]/10000)
    model0.addConstr(V[2, s] == 0)
    model0.addConstr(I[2, s] == I0101[0,s]/10000)
    model0.addConstr(R[2, s] == 0.95*2*states_cases0[s][23]/10000)
    for t in range(T1,T2):
        model0.addConstr(S[t+1,s] == reinf[t,s]+(1-mu)*S[t,s] +mu - 10000*beta[t,s]*I[t,s]*S[t,s]-Vac_1[t-gamma1,s])
        model0.addConstr(V[t+1,s] == V[t,s] +Vac_2[t-gamma1,s]+Vac_1[t-gamma1,s] -effec*10000*beta[t,s]*V[t,s]*I[t,s] -mu*V[t,s])
        model0.addConstr(I[t+1,s] == I[t,s] + 10000*beta[t,s]*S[t,s] *I[t,s]+effec*10000*beta[t,s]*V[t,s]*I[t,s]-gamma*I[t,s]-mu*I[t,s])
        model0.addConstr(R[t+1,s] == R[t,s] + +gamma*I[t,s]-mu*R[t,s]-Vac_2[t-gamma1,s])
        model0.addConstr(Vac[t-gamma1,s]==Vac_1[t-gamma1,s]+Vac_2[t-gamma1,s])
    if t>=(reinf_period+3):
        model0.addConstr(reinf[t,s]==(reinf_rate)*(10000*beta[t-reinf_period-1,s]*S[t-reinf_period-1,s] *I[t-reinf_period-1,s]-mu*I[t-reinf_period-1,s]))
    else:
        model0.addConstr(reinf[t,s]==reinf_rate*underreported[s,t+21-reinf_period]*(states_cases0[s][t+22-reinf_period]-states_cases0[s][t+21-reinf_period])/10000)
    if t>=5:
        model0.addConstr(tau[t,s]==sum(10000*beta[ti,s]*S[ti,s] *I[ti,s]+effec*10000*beta[ti,s]*V[ti,s]*I[ti,s] for ti in range(t-3,t))/3)
    else:
        model0.addConstr(tau[t,s]==0.5*beta[2,s]*popplus12_s[s]/Scale)
#                 model0.addConstr((Vac[t,s])<=0.4*sum(county_cap[c] for c in counties2[s]))
for t in range(T1-1,T2):
    for s in states:
        model0.addConstr(Vac[t,s]<=1*est_demand[t,s])
        model0.addConstr(Vac[t,s]<=sum(county_cap[c] for c in counties2[s])+X_o[t,s])
for s in states:
    for t in range(T1,T2):
        model0.addConstr(Vac[t-1,s]/sum(popplus12[s,c]for c in counties2[s])*10000>=lb[t-1])
for s in states:
    for t in range(T1,T2):
        model0.addConstr((tau[t,s] - M*(1-U_I[t-1,s]))<=Scale*beta[t,s] * S[t, s] * I[t, s] + effec*Scale*beta[t,s] *  V[t, s] * I[t, s])
        model0.addConstr((tau[t,s] + M*(U_I[t-1,s]) )>=Scale*beta[t,s] * S[t, s] * I[t, s] + effec*Scale*beta[t,s] *  V[t, s] * I[t, s] )
for s in states:
    for t in range(T1,T2):
        model0.addConstr((sum(county_cap[c] for c in counties2[s])*(T2-T1) - M*(1-U_D[t-1,s]))<=est_demand[t,s])
        model0.addConstr((sum(county_cap[c] for c in counties2[s])*(T2-T1))+ M*(U_D[t-1,s]) >=est_demand[t,s] )
for t in range(T1-1,T2):
    for s in states:
        model0.addConstr(X_o[t,s]<= U_I[t,s]+U_D[t,s])
        model0.addConstr(X_o[t,s]>= (U_I[t,s]+U_D[t,s])*(B[0]-C)/100/B[0])

for t in range(T1-1,T2):
    for s in states:
        for c in counties2[s]:
            model0.addConstr(Vac0[t,s,c]<=county_cap[c])

for t in range(T1-1,T2):
    for s in states:
        model0.addConstr(Vac[t,s]==(gp.quicksum(Vac0[t,s,c] for c in counties2[s]))+(1-wastage)*X_o[t,s])
for t in range(T1-1,T2):
    for i in suppliers:
        model0.addConstr(gp.quicksum(G1[t,i,s] for s in states)<=Pi[t-1,i])
# for s in states:
#     for t in range(T1-1,T2-1):
#         model0.addConstr(gp.quicksum(G1[t,i,s] for i in suppliers)+W1[t-1,s] == Vac_i[t,s])

for s in states:
    for t in range(T1-1,T2):
        model0.addConstr(gp.quicksum(G1[t,i,s] for i in suppliers)+W1[t-1,s] == gp.quicksum(G2[t,s,c] for c in counties2[s] )+X_o[t,s]*capacity+W1[t,s])
        # model.addConstr(gp.quicksum(G3[t,h_s,j] for h_s in statehubs)*1/21==gp.quicksum(G4[t,j,l]for l in localcenters[j]))
    model0.addConstr(W1[0,s]==0)
for t in range(T1-1,T2):
    for s in states:
        for c in counties2[s]:
            model0.addConstr (G2[t,s,c]  + W2[t-1,s,c]== gp.quicksum(G3[t,s,c,l] for l in local_centers[s,c])+ W2[t,s,c] )
            model0.addConstr(W2[0,s,c]==0)
for t in range(T1-1,T2):
    for s in states:
        for c in counties2[s]:
            model0.addConstr((1-wastage)*gp.quicksum(G3[t,s,c,l]+W3[t-1,s,c,l] for l in local_centers[s,c])==Vac0[t,s,c]+gp.quicksum(W3[t,s,c,l] for l in local_centers[s,c]))
            for l in local_centers[s,c]:
                model0.addConstr(W3[0,s,c,l]==0)
                # model0.addConstr(W3[-1,s,c,l]==0)
for t in range(T1-1,T2):
    for s in states:
        for c in counties2[s]:
            # model0.addConstr(Vac0[t,s,c] <=county_cap[c])
            model0.addConstr(Vac0[t,s,c]/popplus12[s,c]*10000 >= lb0[t,s])
# if T1>=3:
#     for t in range(T1-1,T2-1):
#         for s in states:
model0.addConstr(sum(sum(cost_param["w"][0]*W1[t,s] for s in states )+sum(cost_param["w"][1]*W2[t,s,c] for s in states for c in counties2[s])+sum(cost_param["w"][1]*W3[t,s,c,l] for s in states for c in counties2[s] for l in local_centers[s,c])+sum(distance[s]/2000*cost_param["g"][1]*G2[t,s,c] for s in states for c in counties2[s] )+sum(cost_param["c"][1]*G1[t,1,s]+cost_param["c"][2]*G1[t,2,s]+cost_param["c"][3]*G1[t,3,s] for s in states )+sum(cost_param["w"][1]*G3[t,s,c,l] for s in states for c in counties2[s] for l in local_centers[s,c])+sum(cost_param["c"][0]*X_o[t,s] for s in states)for t in  range(T1-1,T2-1))==C)

model0.addConstr(C<=B[0])
model0.setObjective(-gp.quicksum(access[s]*(sum(weights[s,c]*(Vac0[t,s,c])for c in counties2[s]))+lb0[t,s]+X_o[t,s] for s in states for t in range(T1-1,12))/(sum(Pi[t,2]+Pi[t,1]+Pi[t,3]for t in range(0,T-1)))+1*gp.quicksum(gp.quicksum((10000*beta[t,s] * S[t, s] * I[t, s] + effec*10000*beta[t,s] *  V[t, s] * I[t, s]) for s in states)for t in range(2, 12))/(1.5*sum(states_cases0[s][34]-1.5*states_cases0[s][23] for s in states)/10000)-sum(1*lb[t-1] for t in range(T1, T2))/(sum(Pi[t,2]+Pi[t,1]+Pi[t,3]for t in range(0,T-1))), GRB.MINIMIZE)
model0.Params.TIME_LIMIT = 3600*3
model0.Params.ScaleFlag=2
model0.optimize()
model0.getObjective()
if model0.Status == GRB.OPTIMAL:
    for s in states:
        for t in  range(T1-1,T2-1):
            for c in counties2[s]:
                Vac_all[t,s,c] = Vac0[t,s,c].x
elif model0.Status == GRB.INFEASIBLE:
    for s in states:
        for t in  range(T1-1,T2-1):
            for c in counties2[s]:
                Vac_all[t,s,c] = "nan"
if model0.Status == GRB.OPTIMAL:
    for s in states:
        for t in  range(T1-1,T2-1):
            Vac_avl[t,s] = sum(Vac0[t,s,c].x for c in counties2[s])
elif model0.Status == GRB.INFEASIBLE:
    for s in states:
        for t in  range(T1-1,T2-1):
            Vac_avl[t,s] = "nan"
if model0.Status == GRB.INFEASIBLE:
    cost[T1]= "nan"

else:
    cost[T1]= sum(sum(cost_param["w"][0]*W1[t,s].x for s in states )+sum(cost_param["w"][1]*W2[t,s,c].x for s in states for c in counties2[s])+sum(cost_param["w"][1]*W3[t,s,c,l].x for s in states for c in counties2[s] for l in local_centers[s,c])+sum(distance[s]/2000*cost_param["g"][1]*G2[t,s,c].x for s in states for c in counties2[s] )+sum(cost_param["c"][1]*G1[t,1,s].x+cost_param["c"][2]*G1[t,2,s].x+cost_param["c"][3]*G1[t,3,s].x for s in states )+sum(cost_param["w"][1]*G3[t,s,c,l].x for s in states for c in counties2[s] for l in local_centers[s,c])+sum(cost_param["c"][0]*X_o[t,s].x for s in states)for t in  range(T1-1,T2-1))
# for i in suppliers:
#     available_at_supplier[T1,i]=sup_avl[T1-1,i].x
for s in states:
    for t in range(T1-1,T2-1):
        #for c in counties2[s]:
        if model0.Status == GRB.OPTIMAL:
            #inventory[t,s]= sum(W2[t,s,c].x for c in counties2[s])
            inventory[t,s]=W1[t,s].x
        elif model0.Status == GRB.INFEASIBLE:
            inventory[t,s]="nan"
#inventory
if model0.Status == GRB.OPTIMAL:
    for t in range(T1-1,T2-1):
        for s in states:
            W1_p[t,s]=W1[t,s].x
            for c in counties2[s]:
                W2_p[t,s,c]=W2[t,s,c].x
                for l in local_centers[s,c]:
                    W3_p[t,s,c,l]=W3[t,s,c,l].x
for t in range(T1-1,T2):
    for s in states:
        if model0.Status == GRB.OPTIMAL:
            G1_supply[t,1,s]= G1[t,1,s].x
            G1_supply[t,2,s]= G1[t,2,s].x
            G1_supply[t,3,s]= G1[t,3,s].x
        elif model0.Status == GRB.INFEASIBLE:
            G1_supply[t,1,s]="nan"
            G1_supply[t,2,s]="nan"
            G1_supply[t,3,s]="nan"
for t in range(T1-1,T2-1):
    for s in states:
        for c in counties2[s]:
            if model0.Status == GRB.OPTIMAL:
                G2_supply[t,s,c]= G2[t,s,c].x
            elif model0.Status == GRB.INFEASIBLE:
                G2_supply[t,s,c]="nan"
sol_time0[T1]=model0.Runtime
model0.setParam(GRB.Param.Heuristics, 0)
inf={(t,s): beta[t,s] * S[t, s].x * I[t, s].x + effec*beta[t,s] *  V[t, s].x * I[t, s].x for t in range(T1, T2) for s in states  }

U_I={(t, s): U_I[t, s].x for t in range(T1-1,T2-1)  for s in states  }
U_D={(t, s): U_D[t, s].x for t in range(T1-1,T2-1)  for s in states  }
Vac_i={(t, s): Vac[t, s].x for t in range(T1-1,T2-1)  for s in states }
vac_r={}
for s in states:
    for t in range(T1,T2):
        if sum(Vac[t-1, s].x for s in states)==0:
            vac_r[t-1,s]=0
        else:
            vac_r[t-1,s]=(Vac[t-1, s]).x/sum(Vac[t-1, s].x for s in states)
S1={(t, s): S[t, s].x for t in range(T1+1,T2+1)  for s in states }
I1={(t, s): I[t, s].x for t in range(T1+1,T2+1)  for s in states }
V1={(t, s): V[t, s].x for t in range(T1+1,T2+1) for s in states }
R1={(t, s): R[t, s].x for t in range(T1+1,T2+1) for s in states }
Vac_df=pd.DataFrame({"States":states})
sol_time[T1]=model0.Runtime
if model0.Status == GRB.OPTIMAL:
    for s in states:
        for t in  range(T1-1,T2-1):
            X_of[t,s] = X_o[t,s].x
elif model0.Status == GRB.INFEASIBLE:
    for s in states:
        for t in  range(T1-1,T2-1):
           X_of[t,s] = "nan"
u_bar1={}
u2={}
u3={}
gini_Coef={}
for s in states:
    for t in  range(2,T):
        for c in counties2[s]:
            u2[t, s, c] = sum(Vac0[t,s,c].x for t in range(2,12))/ (popplus12[s,c]/10000)
    # Constraints to linearize the absolute value
for s in states:
    for c in counties2[s]:
        for cc in counties2[s]:
            u3[t, s,c,cc] =abs( u2[t,s, c] - u2[t, s,cc])
for s in states:
    u_bar1[s] = 1/len(counties2[s])*sum(u2[t ,s,c] for c in counties2[s])
    gini_Coef[s]=sum(u3[t,  s,c,cc] for c in counties2[s] for cc in counties2[s])/( 2*(u_bar1[s])*len(counties2[s])*len(counties2[s]))
data01={}
for sol_index in range(model0.SolCount):
    model0.setParam('SolutionNumber', sol_index)
    data01[sol_index]=[]
    for s in states:
        for t in range(2,12):
    # print(sum(beta[t,s]*S[t,s].xn*I[t,s].xn+effec*beta[t,s]*V[t,s].xn*I[t,s].xn for s in states for t in range(2,T+1))*10000*10000)
            data01[sol_index].append(X_o[t,s].xn*10+data[t,s])
            # print(X_o[t,s].xn,data[t,s])
for s in states:
    print(s,sum(G1[t,i,s].x for i in suppliers for t in range(2,12)),sum(Vac[t,s].x for t in range(2,12))*10000,state_vaccine[10,s])
print(sum(beta[t,s]*S[t,s].x*I[t,s].x+effec*beta[t,s]*V[t,s].x*I[t,s].x for s in states for t in range(2,T+1))*10000*10000)

In [ ]:
# states=[]

# states=states2
# states.extend(Middle_Atlantic)
# states.extend(New_England)
#Decomposition 2 edited on 11.12.24
#STATE Level
#Solve Sub-problem
T1=2
T2=12
T=12
vac_hes={}
est_demand={}
Scale=10000
lam=100
states=states2
for s in states:
    for t in range(1,13):
        vac_hes[t,s]=0
        # est_demand[t,s]=1/13*(1-sum(hesit[c]for c in counties2[s])/len(counties2[s]))*popplus12_s[s]/Scale
        if s in West:
            est_demand[t,s]=1/9*(1-sum(hesit[c]for c in counties2[s])/len(counties2[s]))*popplus12_s[s]/Scale
        if (s in New_England):
            est_demand[t,s]=1/8*(1-sum(hesit[c]for c in counties2[s])/len(counties2[s]))*popplus12_s[s]/Scale
        if (s in Middle_Atlantic):
            est_demand[t,s]=1/10*(1-sum(hesit[c]for c in counties2[s])/len(counties2[s]))*popplus12_s[s]/Scale
        if s in Midwest:
            est_demand[t,s]=1/10*(1-sum(hesit[c]for c in counties2[s])/len(counties2[s]))*popplus12_s[s]/Scale
        if s in South:
            est_demand[t,s]=1/11*(1-sum(hesit[c]for c in counties2[s])/len(counties2[s]))*popplus12_s[s]/Scale
        if s in Southwest:
            est_demand[t,s]=1/10*(1-sum(hesit[c]for c in counties2[s])/len(counties2[s]))*popplus12_s[s]/Scale

local_centers={}
for s in states:
    for c in counties2[s]:
        local_centers[s,c]=range(int(np.floor(3*popplus12[s,c]/10000)))

Pi={}
Pi[0,1]=0
Pi[0,2]=0
Pi[0,3]=0
wastage=0.00
pfizer_sub=[]
moderna_sub=[]
johnson_sub=[]
for t in range(0,21):
    pfizer_sub.append(0)
    moderna_sub.append(0)
    johnson_sub.append(0)
    #
    pfizer_sub[t]+=Pf_M[t]
    moderna_sub[t]+=Mo_M[t]
    johnson_sub[t]+=Jo_M[t]
    pfizer_sub[t]+=Pf_N[t]
    moderna_sub[t]+=Mo_N[t]
    johnson_sub[t]+=Jo_N[t]
    #
    # pfizer_sub[t]+=Pf_S[t]
    # moderna_sub[t]+=Mo_S[t]
    # johnson_sub[t]+=Jo_S[t]
    #
    # pfizer_sub[t]+=Pf_SW[t]
    # moderna_sub[t]+=Mo_SW[t]
    # johnson_sub[t]+=Jo_SW[t]
    #
    # pfizer_sub[t]+=Pf_MW[t]
    # moderna_sub[t]+=Mo_MW[t]
    # johnson_sub[t]+=Jo_MW[t]
    #
    # pfizer_sub[t]+=Pf_W[t]
    # moderna_sub[t]+=Mo_W[t]
    # johnson_sub[t]+=Jo_W[t]
# wastage=wastage_S
# # states.extend(South)
# pfizer_sub.extend(Pf_S)
# moderna_sub.extend(Mo_S)
# johnson_sub.extend(Jo_S)
# pfizer_sub.extend(Pf_M)
# moderna_sub.extend(Mo_M)
# johnson_sub.extend(Jo_M)
# pfizer_sub.extend(Pf_SW)
# moderna_sub.extend(Mo_SW)
# johnson_sub.extend(Jo_SW)
# pfizer_sub.extend(Pf_MW)
# moderna_sub.extend(Mo_MW)
# johnson_sub.extend(Jo_MW)
# pfizer_sub.extend(Pf_N)
# moderna_sub.extend(Mo_N)
# johnson_sub.extend(Jo_N)
wastage=0.0

states=[]
#

# states=states2
states.extend(Middle_Atlantic)
states.extend(New_England)
# pfizer_sub=Pf_MANE
# moderna_sub=Mo_MANE
# johnson_sub=Jo_MANE
# pfizer_sub.insert(0,0)
# moderna_sub.insert(0,0)
# johnson_sub.insert(0,0)
# states.extend(West)
# pfizer_sub=Pf_W
# moderna_sub=Mo_W
# johnson_sub=Jo_W
#
# states.extend(Midwest)
# pfizer_sub=Pf_MW
# moderna_sub=Mo_MW
# johnson_sub=Jo_MW

# states.extend(Southwest)
# pfizer_sub=Pf_SW
# moderna_sub=Mo_SW
# johnson_sub=Jo_SW




# for t in range(0,)


for t in range(1,T-1):
    t1=2*t-2
    t2=2*t-1
    Pi[t,1]=pfizer_sub[2*t-2]+pfizer_sub[2*t-1]
    Pi[t,2]=moderna_sub[2*t-2]+moderna_sub[2*t-1]
    Pi[t,3]=johnson_sub[2*t-2]+johnson_sub[2*t-1]
Pi[10,1]+=1*(pfizer_sub[19])
Pi[10,2]+=moderna_sub[19]
Pi[10,3]+=johnson_sub[19]
tau={}
for s in states:
    i=0
    tau[s]=0.01*(popplus12_s[s])/10000
    # budget[s]=df2["budget"][i]/10000
    i+=1

# N = sum(N_initial)/10000
M = 1000000000/10000
supply=1000000/10000
cost_param=pd.DataFrame({"w":[1,1,1,1],
                "g":[1,1,1,1],
                "c":[20,20,15,5]})
B=[500000,100000,1200000]
suppliers = [1,2,3]



mu=14/75/365/10000
gamma = 1
gamma1=1
# results["Y_bn"]=1
# Y_bn=[]
# Y_bn=list(results["Y_bn"])
# results["U_I"]=1
# U_I={}
# for s in states:
#     for t in time:
#         U_I[t,s]=1
# results=pd.DataFrame({"State":states})
sol_time={}
sol_time0={}
Vac_avl={}
for s in states:
    Vac_avl[1,s]=0
dic={}
for t in time:
    for s in states:
        dic[t,s]=None
results1=pd.DataFrame.from_dict(dic, orient='index')
dic1={}
for t in time:
    for s in states:
        for c in counties2[s]:
            dic1[t,s]=None
results2=pd.DataFrame.from_dict(dic1, orient='index')
global T1,T2
Scale=10000
global lower_bound
effec=0.2
inventory={}
opening={}
for s in states:
    #for c in counties2[s]:
    inventory[0,s]=0
upper={}
upper0={}
T=12
u2={}
u3={}
u_bar1={}
S={}
V={}
I={}
R={}
Vac={}
# S1={}
# V1={}
# I1={}
# R1={}
Vac_1={}
Vac_2={}
inf={}
# X0={}
U_I={}
U_D={}
reinf={}
Results={}
lb={}

Gini_Coef = {}
G1 = {}
W1 = {}
W2 = {}
G2 = {}
W3 = {}
G3 = {}
Y = {}
X_o={}
Y_b = {}
X_of = {}
cost = {}
capacity=10000/10000
Y_bn=[]
# results["Vac"]=None
Vac_all={}
G1_supply={}
G2_supply={}
sup_avl={}
state_cost={}
# inventory={}
lb0={}
#Retrieval of results
W1_p={}
W2_p={}
W3_p={}
tau={}
for t in range(0,2):
    for s in states:
        W1_p[t,s]=0
        for c in counties2[s]:
            W2_p[t,s,c]=0
            for l in local_centers[s,c]:
                W3_p[t,s,c,l]=0
available_at_supplier={}
for i in suppliers:
    available_at_supplier[1,i]=0
bud_ava={}
model0 = gp.Model()

for t in range(T1-1,T2-1):
    lb[t]= model0.addVar( lb=0, vtype=GRB.CONTINUOUS)
# global Vac, U_I
# Define parameters

for s in states:

    for t in range(T1,T2+1):
        S[t,s] = model0.addVar( lb=0, ub=popplus12_s[s]/10000 ,vtype=GRB.CONTINUOUS)
        V[t,s] = model0.addVar( lb=0, ub=popplus12_s[s]/10000 , vtype=GRB.CONTINUOUS)
        I[t,s] = model0.addVar( lb=0, ub=popplus12_s[s]/10000 ,vtype=GRB.CONTINUOUS)
        R[t,s] = model0.addVar( lb=0, ub=popplus12_s[s]/10000 ,vtype=GRB.CONTINUOUS)
        reinf[t,s]= model0.addVar( lb=0,ub=popplus12_s[s]/10000 ,vtype=GRB.CONTINUOUS)
        tau[t,s]= model0.addVar( lb=0,ub=popplus12_s[s] ,vtype=GRB.CONTINUOUS)
    for t in range(T1-1,T2):
        Vac_1[t,s] = model0.addVar(lb=0, ub=popplus12_s[s]/10000,vtype=GRB.CONTINUOUS)
        Vac_2[t,s] = model0.addVar(lb=0, ub=popplus12_s[s]/10000,vtype=GRB.CONTINUOUS)
        Vac[t,s] = model0.addVar(lb=0, ub=popplus12_s[s]/10000,vtype=GRB.CONTINUOUS)
        # Vac[t,s] = model0.addVar(lb=upper[t,s]*opening_f[t-1,s], ub=upper[t,s]*popplus12_s[s]/10000,vtype=GRB.CONTINUOUS)
    for t in range(T1-1,T2):
        U_I[t,s] = model0.addVar(vtype=GRB.BINARY)
        U_D[t,s] = model0.addVar(vtype=GRB.BINARY)
X_o = {(t, s): model0.addVar(lb=0, vtype=GRB.BINARY)  for t in  range(T1-1,T2) for s in states}
for s in states:
    Gini_Coef[s] = model0.addVar( lb=0, ub=1, vtype=GRB.CONTINUOUS)
    Y_b[s] = model0.addVar(lb=0, vtype=GRB.BINARY)
    # cost[s] = model0.addVar(lb=0, vtype=GRB.CONTINUOUS)
    # for t in range(2,T+1):
    #     U_I[t,s] = model0.addVar(vtype=GRB.BINARY)
    for t in range(T1-2,T2):
        lb0[t,s]=model0.addVar(lb=0,vtype=GRB.CONTINUOUS)
        W1[t,s] = model0.addVar( lb=0, vtype=GRB.CONTINUOUS)
        for i in [1,2,3]:
            G1[t,i,s] = model0.addVar( lb=0, vtype=GRB.CONTINUOUS)
W2 = {(t, s, c): model0.addVar(lb=0, vtype=GRB.CONTINUOUS) for t in range(T1-2,T2) for s in states for c in counties2[s]}
G2 = {(t, s, c): model0.addVar(lb=0, vtype=GRB.CONTINUOUS) for t in range(T1-1,T2) for s in states for c in counties2[s]}
G3 = {(t, s, c,l): model0.addVar(lb=0, vtype=GRB.CONTINUOUS) for t in range(T1-1,T2) for s in states for c in counties2[s] for l in local_centers[s,c]}
W3 = {(t, s, c,l): model0.addVar(lb=0, vtype=GRB.CONTINUOUS) for t in range(T1-2,T2) for s in states for c in counties2[s] for l in local_centers[s,c]}
Vac0 = {(t, s,c): model0.addVar(lb=0, ub=popplus12[s,c]/10000, vtype=GRB.CONTINUOUS) for t in range(T1-1,T2) for s in states for c in counties2[s]}
ub=model0.addVar(lb=0,ub=1)
y=model0.addVar(lb=0,vtype=GRB.CONTINUOUS)
sup_avl={(t,i): model0.addVar(lb=0, ub=Pi[t-1,i], vtype=GRB.CONTINUOUS) for t in range(T1-1,T2) for i in suppliers}
C=model0.addVar(lb=0)
        # if s=='TX':
        # Vac[t,s] = model0.addVar(lb=0,ub=Vac_TX[t-1]/10000, vtype=GRB.CONTINUOUS)
        # else:
        # if T1==2:
        #     Vac[t,s] = model0.addVar(lb=(state_vaccine[t-1,s]-state_vaccine_unk[t-1,s]-0)/10000,ub=(state_vaccine[t-1,s]-state_vaccine_unk[t-1,s]-0)/10000, vtype=GRB.CONTINUOUS)
        # else:
        #     Vac[t,s] = model0.addVar(lb=(state_vaccine[t-1,s]-state_vaccine[t-2,s]-state_vaccine_unk[t-1,s]+state_vaccine_unk[t-2,s])/10000,ub=(state_vaccine[t-1,s]-state_vaccine[t-2,s]-state_vaccine_unk[t-1,s]+state_vaccine_unk[t-2,s])/10000, vtype=GRB.CONTINUOUS)
# #-state_vaccine_unk[t-1,s]+state_vaccine_unk[t-2,s]
#         if T1==2:
#             model0.addConstr(Vac[T1-1,s]==0)
        # X0[t,s] = model0.addVar(vtype=GRB.BINARY)

cost={}
for s in states:
    cost[s] = model0.addVar(lb=0, vtype=GRB.CONTINUOUS)
for s in states:
    model0.addConstr(S[2, s] ==S0101[0,s]/10000)
    model0.addConstr(V[2, s] == 0)
    model0.addConstr(I[2, s] == I0101[0,s]/10000)
    model0.addConstr(R[2, s] == 0.95*2*states_cases0[s][23]/10000)
    # if t>=10:
    #     model0.addConstr(S[T1, s] ==S01[T1,s]+0.2*(sum(beta[t,s]*I01[t,s]*S01[t,s]+effec*10000*beta[t,s]*V01[t,s]*I01[t,s] for t in range(2,T1))+15*I01[2,s]))
    # else:
    #model0.addConstr(S[T1, s] ==S01[T1,s])
    # if (s=='TX') & (T1<=6):
    #     model0.addConstr(Vac[T1-1, s] == 0)
    # model0.addConstr(Vac[T1-1, s] == Vac_avl[T1-1,s])
    for t in range(T1,T2):
        # model0.addConstr((10000*beta[t,s] * S[t,s]* I[t,s]+10000*0.5*beta[t,s]*V[t,s]*I[t,s]) >= tau[s] - M*(1-U_I[t,s]))
        # model0.addConstr((10000*beta[t,s] * S[t,s]* I[t,s]+10000*0.5*beta[t,s]*V[t,s]*I[t,s]) <= tau[s] + M*(U_I[t,s]) )
        model0.addConstr(S[t+1,s] == reinf[t,s]+(1-mu)*S[t,s] +mu - 10000*beta[t,s]*I[t,s]*S[t,s]-Vac_1[t-gamma1,s])
        model0.addConstr(V[t+1,s] == V[t,s] +Vac_2[t-gamma1,s]+Vac_1[t-gamma1,s] -effec*10000*beta[t,s]*V[t,s]*I[t,s] -mu*V[t,s])
        model0.addConstr(I[t+1,s] == I[t,s] + 10000*beta[t,s]*S[t,s] *I[t,s]+effec*10000*beta[t,s]*V[t,s]*I[t,s]-gamma*I[t,s]-mu*I[t,s])
        model0.addConstr(R[t+1,s] == R[t,s] + +gamma*I[t,s]-mu*R[t,s]-Vac_2[t-gamma1,s])
        model0.addConstr(Vac[t-gamma1,s]==Vac_1[t-gamma1,s]+Vac_2[t-gamma1,s])
    if t>=(reinf_period+3):
        model0.addConstr(reinf[t,s]==(reinf_rate)*(10000*beta[t-reinf_period-1,s]*S[t-reinf_period-1,s] *I[t-reinf_period-1,s]-mu*I[t-reinf_period-1,s]))
    else:
        model0.addConstr(reinf[t,s]==reinf_rate*underreported[s,t+21-reinf_period]*(states_cases0[s][t+22-reinf_period]-states_cases0[s][t+21-reinf_period])/10000)
    if t>=5:
        model0.addConstr(tau[t,s]==sum(10000*beta[ti,s]*S[ti,s] *I[ti,s]+effec*10000*beta[ti,s]*V[ti,s]*I[ti,s] for ti in range(t-3,t))/3)
    else:
        model0.addConstr(tau[t,s]==0.5*beta[2,s]*popplus12_s[s]/Scale)
#                 model0.addConstr((Vac[t,s])<=0.4*sum(county_cap[c] for c in counties2[s]))
for t in range(T1-1,T2):
    for s in states:
        model0.addConstr(Vac[t,s]<=1*est_demand[t,s])
        model0.addConstr(Vac[t,s]<=sum(county_cap[c] for c in counties2[s])+X_o[t,s])
for s in states:
    for t in range(T1,T2):
        model0.addConstr(Vac[t-1,s]/sum(popplus12[s,c]for c in counties2[s])*10000>=lb[t-1])
for s in states:
    for t in range(T1,T2):
        model0.addConstr((tau[t,s] - M*(1-U_I[t-1,s]))<=Scale*beta[t,s] * S[t, s] * I[t, s] + effec*Scale*beta[t,s] *  V[t, s] * I[t, s])
        model0.addConstr((tau[t,s] + M*(U_I[t-1,s]) )>=Scale*beta[t,s] * S[t, s] * I[t, s] + effec*Scale*beta[t,s] *  V[t, s] * I[t, s] )
for s in states:
    for t in range(T1,T2):
        model0.addConstr((sum(county_cap[c] for c in counties2[s])*(T2-T1) - M*(1-U_D[t-1,s]))<=est_demand[t,s])
        model0.addConstr((sum(county_cap[c] for c in counties2[s])*(T2-T1))+ M*(U_D[t-1,s]) >=est_demand[t,s] )
# for t in range(T1-1,T2-1):
    # model0.addConstr(1/(1-wastage)*(gp.quicksum(Vac[t,s] for s in states))<= (a*(Pi[t-1,1]+Pi[t-1,2]+Pi[t-1,3])+a*(sum(Pi[f,1]+Pi[f,2]+Pi[f,3] for f in range(0,t-1)))-sum(Vac_state[f,s]for s in states for f in range(1,t))))
    #new09262024
    # model0.addConstr(1/(1-wastage)*(gp.quicksum(Vac[t,s] for s in states))<= (a*(Pi[t-1,1]+Pi[t-1,2]+Pi[t-1,3])+sum(inventory[t-1,s]for s in states)))
    # model0.addConstr(1/(1-wastage)*(gp.quicksum(Vac[t,s] for s in states))<= a*(Pi[t-1,1]+Pi[t-1,2]+Pi[t-1,3])+a*sum(available_at_supplier[T1-1,i] for i in suppliers))
    # model0.addConstr(1/(1-wastage)*(gp.quicksum(Vac[t,s] for s in states))>=0.9*(Pi[t,1]+Pi[t,2]))
for t in range(T1-1,T2):
    for s in states:
        model0.addConstr(X_o[t,s]<= U_I[t,s]+U_D[t,s])
        model0.addConstr(X_o[t,s]>= (U_I[t,s]+U_D[t,s])*(B[0]-C)/100/B[0])

for t in range(T1-1,T2):
    for s in states:
        for c in counties2[s]:
            model0.addConstr(Vac0[t,s,c]<=county_cap[c])

for t in range(T1-1,T2):
    for s in states:
        model0.addConstr(Vac[t,s]==(gp.quicksum(Vac0[t,s,c] for c in counties2[s]))+(1-wastage)*X_o[t,s])
for t in range(T1-1,T2):
    for i in suppliers:
        model0.addConstr(gp.quicksum(G1[t,i,s] for s in states)<=Pi[t-1,i])
# for s in states:
#     for t in range(T1-1,T2-1):
#         model0.addConstr(gp.quicksum(G1[t,i,s] for i in suppliers)+W1[t-1,s] == Vac_i[t,s])

for s in states:
    for t in range(T1-1,T2):
        model0.addConstr(gp.quicksum(G1[t,i,s] for i in suppliers)+W1[t-1,s] == gp.quicksum(G2[t,s,c] for c in counties2[s] )+X_o[t,s]*capacity+W1[t,s])
        # model.addConstr(gp.quicksum(G3[t,h_s,j] for h_s in statehubs)*1/21==gp.quicksum(G4[t,j,l]for l in localcenters[j]))
    model0.addConstr(W1[0,s]==0)
for t in range(T1-1,T2):
    for s in states:
        for c in counties2[s]:
            model0.addConstr (G2[t,s,c]  + W2[t-1,s,c]== gp.quicksum(G3[t,s,c,l] for l in local_centers[s,c])+ W2[t,s,c] )
            model0.addConstr(W2[0,s,c]==0)
for t in range(T1-1,T2):
    for s in states:
        for c in counties2[s]:
            model0.addConstr((1-wastage)*gp.quicksum(G3[t,s,c,l]+W3[t-1,s,c,l] for l in local_centers[s,c])==Vac0[t,s,c]+gp.quicksum(W3[t,s,c,l] for l in local_centers[s,c]))
            for l in local_centers[s,c]:
                model0.addConstr(W3[0,s,c,l]==0)
                # model0.addConstr(W3[-1,s,c,l]==0)
# for t in range(T1-1,T2):
    # for s in states:
    #     for c in counties2[s]:
    #         # model0.addConstr(Vac0[t,s,c] <=county_cap[c])
    #         model0.addConstr(Vac0[t,s,c]/popplus12[s,c]*10000 >= lb0[t,s])

for s in states:
    for c in counties2[s]:
        for cc in counties2[s]:
            u3[s,c,cc] = model0.addVar(lb=0, vtype=GRB.CONTINUOUS)
    for c in counties2[s]:
        u2[s,c] = model0.addVar(lb=0, vtype=GRB.CONTINUOUS)
for s in states:
    for c in counties2[s]:
        model0.addConstr(u2[s, c] == (sum(Vac0[t,s,c] for t in  range(T1-1,T2-1))/ (popplus12[s,c]/10000)))
    # Constraints to linearize the absolute value
for s in states:
    for c in counties2[s]:
        for cc in counties2[s]:
            model0.addConstr(u3[s,c,cc] >= u2[s, c] - u2[s,cc])
            model0.addConstr(u3[s, c,cc] >= -(u2[s,c] - u2[s,cc]))
    # Example objective function (minimizing the sum of absolute differences)
    u_bar1[s] = 1/len(counties2[s])*gp.quicksum(u2[s,c] for c in counties2[s])
    # for c in counties[s]:
    model0.addConstr(gp.quicksum(u3[s,c,cc] for c in counties2[s] for cc in counties2[s]) <= 2*Gini_Coef[s]*(u_bar1[s])*len(counties2[s])*len(counties2[s]))
    model0.addConstr(Gini_Coef[s] <= ub)
# if T1>=3:
#     for t in range(T1-1,T2-1):
#         for s in states:
model0.addConstr(sum(sum(cost_param["w"][0]*W1[t,s] for s in states )+sum(cost_param["w"][1]*W2[t,s,c] for s in states for c in counties2[s])+sum(cost_param["w"][1]*W3[t,s,c,l] for s in states for c in counties2[s] for l in local_centers[s,c])+sum(distance[s]/2000*cost_param["g"][1]*G2[t,s,c] for s in states for c in counties2[s] )+sum(cost_param["c"][1]*G1[t,1,s]+cost_param["c"][2]*G1[t,2,s]+cost_param["c"][3]*G1[t,3,s] for s in states )+sum(cost_param["w"][1]*G3[t,s,c,l] for s in states for c in counties2[s] for l in local_centers[s,c])+sum(cost_param["c"][0]*X_o[t,s] for s in states)for t in  range(T1-1,T2-1))==C)

model0.addConstr(C<=B[0])

model0.setObjective(ub+1*gp.quicksum(gp.quicksum((10000*beta[t,s] * S[t, s] * I[t, s] + effec*10000*beta[t,s] *  V[t, s] * I[t, s]) for s in states)for t in range(2, 12))/(1.5*sum(states_cases0[s][34]-1.5*states_cases0[s][23] for s in states)/10000)-sum(1*lb[t-1] for t in range(T1, T2))/(sum(Pi[t,2]+Pi[t,1]+Pi[t,3]for t in range(0,T-1))), GRB.MINIMIZE)
model0.Params.TIME_LIMIT = 3600*12
model0.Params.ScaleFlag=2
model0.optimize()
model0.getObjective()
if model0.Status == GRB.OPTIMAL:
    for s in states:
        for t in  range(T1-1,T2-1):
            for c in counties2[s]:
                Vac_all[t,s,c] = Vac0[t,s,c].x
elif model0.Status == GRB.INFEASIBLE:
    for s in states:
        for t in  range(T1-1,T2-1):
            for c in counties2[s]:
                Vac_all[t,s,c] = "nan"
if model0.Status == GRB.OPTIMAL:
    for s in states:
        for t in  range(T1-1,T2-1):
            Vac_avl[t,s] = sum(Vac0[t,s,c].x for c in counties2[s])
elif model0.Status == GRB.INFEASIBLE:
    for s in states:
        for t in  range(T1-1,T2-1):
            Vac_avl[t,s] = "nan"
if model0.Status == GRB.INFEASIBLE:
    cost[T1]= "nan"

else:
    cost[T1]= sum(sum(cost_param["w"][0]*W1[t,s].x for s in states )+sum(cost_param["w"][1]*W2[t,s,c].x for s in states for c in counties2[s])+sum(cost_param["w"][1]*W3[t,s,c,l].x for s in states for c in counties2[s] for l in local_centers[s,c])+sum(distance[s]/2000*cost_param["g"][1]*G2[t,s,c].x for s in states for c in counties2[s] )+sum(cost_param["c"][1]*G1[t,1,s].x+cost_param["c"][2]*G1[t,2,s].x+cost_param["c"][3]*G1[t,3,s].x for s in states )+sum(cost_param["w"][1]*G3[t,s,c,l].x for s in states for c in counties2[s] for l in local_centers[s,c])+sum(cost_param["c"][0]*X_o[t,s].x for s in states)for t in  range(T1-1,T2-1))
# for i in suppliers:
#     available_at_supplier[T1,i]=sup_avl[T1-1,i].x
for s in states:
    for t in range(T1-1,T2-1):
        #for c in counties2[s]:
        if model0.Status == GRB.OPTIMAL:
            #inventory[t,s]= sum(W2[t,s,c].x for c in counties2[s])
            inventory[t,s]=W1[t,s].x
        elif model0.Status == GRB.INFEASIBLE:
            inventory[t,s]="nan"
#inventory
if model0.Status == GRB.OPTIMAL:
    for t in range(T1-1,T2-1):
        for s in states:
            W1_p[t,s]=W1[t,s].x
            for c in counties2[s]:
                W2_p[t,s,c]=W2[t,s,c].x
                for l in local_centers[s,c]:
                    W3_p[t,s,c,l]=W3[t,s,c,l].x
for t in range(T1-1,T2):
    for s in states:
        if model0.Status == GRB.OPTIMAL:
            G1_supply[t,1,s]= G1[t,1,s].x
            G1_supply[t,2,s]= G1[t,2,s].x
            G1_supply[t,3,s]= G1[t,3,s].x
        elif model0.Status == GRB.INFEASIBLE:
            G1_supply[t,1,s]="nan"
            G1_supply[t,2,s]="nan"
            G1_supply[t,3,s]="nan"
for t in range(T1-1,T2-1):
    for s in states:
        for c in counties2[s]:
            if model0.Status == GRB.OPTIMAL:
                G2_supply[t,s,c]= G2[t,s,c].x
            elif model0.Status == GRB.INFEASIBLE:
                G2_supply[t,s,c]="nan"
sol_time0[T1]=model0.Runtime
model0.setParam(GRB.Param.Heuristics, 0)
inf={(t,s): beta[t,s] * S[t, s].x * I[t, s].x + effec*beta[t,s] *  V[t, s].x * I[t, s].x for t in range(T1, T2) for s in states  }
U_I={(t, s): U_I[t, s].x for t in range(T1-1,T2-1)  for s in states  }
U_D={(t, s): U_D[t, s].x for t in range(T1-1,T2-1)  for s in states  }
Vac_i={(t, s): Vac[t, s].x for t in range(T1-1,T2-1)  for s in states }
vac_r={}
for s in states:
    for t in range(T1,T2):
        if sum(Vac[t-1, s].x for s in states)==0:
            vac_r[t-1,s]=0
        else:
            vac_r[t-1,s]=(Vac[t-1, s]).x/sum(Vac[t-1, s].x for s in states)
S1={(t, s): S[t, s].x for t in range(T1+1,T2+1)  for s in states }
I1={(t, s): I[t, s].x for t in range(T1+1,T2+1)  for s in states }
V1={(t, s): V[t, s].x for t in range(T1+1,T2+1) for s in states }
R1={(t, s): R[t, s].x for t in range(T1+1,T2+1) for s in states }
Vac_df=pd.DataFrame({"States":states})
sol_time[T1]=model0.Runtime
if model0.Status == GRB.OPTIMAL:
    for s in states:
        for t in range(T1-1,T2-1):
            X_of[t,s] = X_o[t,s].x
elif model0.Status == GRB.INFEASIBLE:
    for s in states:
        for t in  range(T1-1,T2-1):
           X_of[t,s] = "nan"
print(sum(beta[t,s]*S[t,s].x*I[t,s].x+effec*beta[t,s]*V[t,s].x*I[t,s].x for s in states for t in range(2,T+1))*10000*10000)
print(sum(X_o[t,s].x for s in states for t in range(1,12)))

## Data Visualization

In [ ]:
# Define the data in a dictionary
data = {
    "Baseline": [4226684, 9165662, 12782983, 15052723, 16376487, 17429352, 18392975, 19502808, 20700332, 21674524, 22420740],
    "Budget_Medium": [4226684, 9165662, 12782983, 15052723, 16376487, 17429352, 18392975, 19625289, 21172611, 22832831, 24662261],
    "Budget_Limited": [4226684, 9165662, 12444184, 14710741, 16220021, 17657022, 19265285, 21659001, 25138282, 29174633, 33950924],
    "Supply_Higher": [4226684, 9165662, 12754246, 14968594, 16228144, 17201105, 18058977, 19016163, 20000056, 20744498, 21493128],
    "Supply_Lower": [4226684, 9165662, 12814079, 15142222, 16530809, 17666001, 18750058, 20067632, 21580188, 22883180, 23961021],
    "InfectionRate_Higher": [8453367, 27835007, 53718818, 79906628, 101167415, 121724706, 141461521, 154481125, 160176038, 161949330, 162427851],
    "InfectionRate_Lower": [2113342, 3359783, 3828730, 3982055, 4029130, 4049086, 4059005, 4065386, 4069278, 4071081, 4071849],
    "VaccineEffectiveness_Higher": [4226684, 9165662, 12750662, 14965877, 16230531, 17209263, 18081375, 19045976, 20027648, 20765136, 21273105],
    "VaccineEffectiveness_Lower": [4226684, 9165662, 12847630, 15228990, 16678231, 17898735, 19090284, 20596960, 22411344, 24090872, 25625525],
    "Capacity_Higher": [4226684, 9165662, 12782983, 15052723, 16376487, 17429352, 18392975, 19502808, 20700332, 21674524, 22420740],
    "Capacity_Lower": [4226684, 9165662, 12803358, 15085412, 16410776, 17464124, 18423799, 19579376, 20894919, 22061728, 23068356],
    "Demand_Higher": [4226684, 9165662, 12775467, 15047187, 16371614, 17427678, 18404221, 19533710, 20734212, 21676669, 22370836],
    "Demand_Lower": [4226684, 9165662, 12812027, 15115071, 16469414, 17563685, 18572398, 19784171, 21151153, 22323897, 23290716],
}

# Create a DataFrame from the dictionary
df = pd.DataFrame(data)

# Create a figure with a specific size
fig, axes = plt.subplots(2, 3, figsize=(24, 14))

# Titles and data for each subplot
categories = [
    ("Supply", "Baseline", "Supply_Higher", "Supply_Lower"),
    ("Budget", "Baseline", "Budget_Medium", "Budget_Limited"),
    ("Infection Rate", "Baseline", "InfectionRate_Higher", "InfectionRate_Lower"),
    ("Vaccine Effectiveness", "Baseline", "VaccineEffectiveness_Higher", "VaccineEffectiveness_Lower"),
    ("Capacity", "Baseline", "Capacity_Higher", "Capacity_Lower"),
    ("Demand", "Baseline", "Demand_Higher", "Demand_Lower"),
]

time = [f" {i+1}" for i in range(len(df.index))]  # Time in months

# Iterate over the categories and plot on respective axes
for ax, (category, baseline, higher, lower) in zip(axes.flatten(), categories):
    ax.plot(time, df[baseline], label="Baseline", marker='o')
    ax.plot(time, df[higher], label=f"Higher{category}", marker='x')
    ax.plot(time, df[lower], label=f"Lower{category} ", marker='s')
    ax.set_title(f"Sensitivity Analysis of {category} \n (Based on Cumulative Infections)",fontsize=18)
    ax.set_xlabel("Time",fontsize=02)
    ax.set_ylabel("Cumulative Infections",fontsize=10)
    ax.axhline(color='black', linewidth=0.8)
    ax.axvline(x=0, color='black', linewidth=0.8)
    ax.legend(loc='upper left',bbox_to_anchor=(0.1, 1))
    ax.grid(True)

# Adjust layout to prevent overlap
plt.tight_layout()

# Show the multiplot figure
plt.show()


In [ ]:
#SOLVING THE ORIGINAL PROBLEM
colors = [
    'Aqua', 'Aquamarine', 'Blue', 'BlueViolet', 'Brown', 'CadetBlue', 'Chartreuse', 'Chocolate',
    'Coral', 'Crimson', 'Cyan', 'DarkBlue', 'DarkCyan', 'DarkGoldenRod', 'DarkGray', 'DarkGreen',
    'DarkKhaki', 'DarkMagenta', 'DarkOliveGreen', 'DarkOrange', 'DarkOrchid', 'DarkRed',
    'DarkSalmon', 'DarkSeaGreen', 'DarkSlateBlue', 'DarkSlateGray', 'DarkTurquoise', 'DarkViolet',
    'DeepPink', 'DeepSkyBlue', 'DimGray', 'DodgerBlue', 'FireBrick', 'ForestGreen', 'Fuchsia',
    'Gold', 'GoldenRod', 'Gray', 'IndianRed', 'Indigo', 'LawnGreen', 'LightCoral', 'Lime',
    'Magenta', 'Maroon', 'MediumBlue', 'MediumOrchid', 'MediumPurple', 'MediumSeaGreen',
    'MediumSlateBlue', 'MediumSpringGreen', 'MediumTurquoise'
]
x1 = ['Dec','Jan','Jan','Feb','Feb','Mar','Mar','Apr','Apr','May']
x=range(0,34)
fig, ax = plt.subplots(figsize=(12, 10),facecolor='none')
y={}
txt=0
for s in states:
    y[s]=[]
    for t in range(1,4):
        y[s].append(5000*beta01[t,s])
    for t in range(4,23):
        y[s].append(50000*beta01[t,s])
    for t in range(23,35):
        y[s].append(50000*beta[t-21,s])
    print(s,y[s])
    ax.plot(x, y[s], linewidth=2.0, label=s,color=colors[txt])

    # plt.text(9.2,0.04*txt,s,fontsize=5, font='Helvetica 18 bold')
    txt+=1
# plot
plt.grid(False)
# x=[21]
# y=range(0,1)

ax.spines['bottom'].set_color('black')  # Bottom x-axis
ax.spines['left'].set_color('black')    # Left y-axis
ax.spines['top'].set_color('black')     # Top axis (optional)
ax.spines['right'].set_color('black')
plt.gcf().set_facecolor("none")
ax.set(xlim=(0, 33), xlabel="Time",
       ylim=(0,1), yticks=[0,1],ylabel="Spatio-temporal Effective Infection Rate")
ax.set_xlabel("Time", fontsize=24)  # Increased font size
ax.set_ylabel("Spatio-temporal Effective Infection Rate", fontsize=24)
# plt.xlabel(fontsize=12)
# plt.ylabel(fontsize=12)
plt.legend(fontsize=14,ncol=2,loc="upper left", bbox_to_anchor=(1, 1))
# ax[0][0]=ax
plt.xticks(ticks=[0,21],labels=['1/29/2020','12.16.2020'],fontsize=14)
plt.yticks(fontsize=14)
plt.axvline(x=21,color='black')
plt.savefig('Beta.png')
plt.show()

In [ ]:
# Dictionary of state capitals with their latitudes and longitudes
state_capitals_0 = {
    "Alabama": [32.377, -86.300],
    "Alaska": [47.037, -125.900],
    "Arizona": [33.448, -112.074],
    "Arkansas": [34.746, -92.290],
    "California": [38.576, -121.494],
    "Colorado": [39.739, -104.990],
    "Connecticut": [41.764, -72.682],
    "Delaware": [39.158, -75.524],
    "Florida": [30.438, -84.280],
    "Georgia": [33.749, -84.388],
    "Hawaii": [30.438, -125.900],
    "Idaho": [43.617, -116.199],
    "Illinois": [39.798, -89.654],
    "Indiana": [39.768, -86.162],
    "Iowa": [41.591, -93.603],
    "Kansas": [39.048, -95.677],
    "Kentucky": [38.197, -84.863],
    "Louisiana": [30.457, -91.187],
    "Maine": [44.307, -69.781],
    "Maryland": [38.978, -76.492],
    "Massachusetts": [42.360, -71.058],
    "Michigan": [42.732, -84.555],
    "Minnesota": [44.953, -93.090],
    "Mississippi": [32.303, -90.182],
    "Missouri": [38.576, -92.173],
    "Montana": [46.589, -112.021],
    "Nebraska": [40.813, -96.702],
    "Nevada": [39.163, -118.766],
    "New Hampshire": [43.208, -71.537],
    "New Jersey": [40.220, -74.772],
    "New Mexico": [35.687, -105.938],
    "New York": [42.652, -73.757],
    "North Carolina": [35.780, -78.639],
    "North Dakota": [46.808, -100.783],
    "Ohio": [39.961, -82.999],
    "Oklahoma": [35.467, -97.516],
    "Oregon": [44.942, -123.035],
    "Pennsylvania": [40.264, -76.883],
    "Rhode Island": [41.823, -71.412],
    "South Carolina": [34.000, -81.035],
    "South Dakota": [44.367, -100.346],
    "Tennessee": [36.162, -86.781],
    "Texas": [30.267, -97.743],
    "Utah": [40.760, -111.891],
    "Vermont": [44.263, -72.575],
    "Virginia": [37.540, -77.436],
    "Washington": [47.037, -122.900],
    "West Virginia": [38.349, -81.633],
    "Wisconsin": [43.073, -89.401],
    "Wyoming": [41.140, -104.820],
    "Washington, D.C.": [38.9072, -77.0369]
}
# State abbreviations
state_abbreviations = {
    "Alabama": "AL", "Alaska": "AK", "Arizona": "AZ", "Arkansas": "AR", "California": "CA", "Colorado": "CO",
    "Connecticut": "CT", "Delaware": "DE", "Florida": "FL", "Georgia": "GA", "Hawaii": "HI", "Idaho": "ID",
    "Illinois": "IL", "Indiana": "IN", "Iowa": "IA", "Kansas": "KS", "Kentucky": "KY", "Louisiana": "LA",
    "Maine": "ME", "Maryland": "MD", "Massachusetts": "MA", "Michigan": "MI", "Minnesota": "MN",
    "Mississippi": "MS", "Missouri": "MO", "Montana": "MT", "Nebraska": "NE", "Nevada": "NV",
    "New Hampshire": "NH", "New Jersey": "NJ", "New Mexico": "NM", "New York": "NY", "North Carolina": "NC",
    "North Dakota": "ND", "Ohio": "OH", "Oklahoma": "OK", "Oregon": "OR", "Pennsylvania": "PA",
    "Rhode Island": "RI", "South Carolina": "SC", "South Dakota": "SD", "Tennessee": "TN", "Texas": "TX",
    "Utah": "UT", "Vermont": "VT", "Virginia": "VA", "Washington": "WA", "West Virginia": "WV",
    "Wisconsin": "WI", "Wyoming": "WY", "Washington, D.C.": "DC"
}
InfectionDemandScore=[]
for s in states:
    InfectionDemandScore.append(beta[2,s]/max(beta[2,s]for s in states)**2+((sum((1-hesit[c])*popplus12[s,c] for c in counties2[s]))/popplus12_s[s])**2)
states_capitals={}
for a in states:
    for s in state_abbreviations.keys():
        if state_abbreviations[s]==a:
            state_capitals[a]=state_capitals_0[s]
state_capitals={}
regions=[]
for a in states:
    for s in state_abbreviations.keys():
        if state_abbreviations[s]==a:
            state_capitals[a]=state_capitals_0[s]
            regions.append(s)
# Create DataFrame with random data
results_data = pd.DataFrame({
    "Region": regions,
    "Latitude": [coords[0] for coords in state_capitals.values()],
    "Longitude": [coords[1] for coords in state_capitals.values()],
    "Abbreviation": states,
    "Actual": [(sum(allocation.loc[allocation["State"]==s]['1st Dose Allocations Pfizer'])+sum(allocation.loc[allocation["State"]==s]['1st Dose Allocations Moderna'])+sum(allocation.loc[allocation["State"]==s]['1st Dose Allocations Janssen']))/10000 for s in states],
    "Optimized": [sum(supply[t,i,s] for i in [1,2,3] for t in range(2,12))
                  for s in states],
    "InfectionDemandScore": [((beta[2,s]/max(beta[2,s]for s in states))**2+((sum((1-hesit[c])*popplus12[s,c] for c in counties2[s]))/popplus12_s[s])**2)/max((beta[2,s]/max(beta[2,s]for s in states))**2+((sum((1-hesit[c])*popplus12[s,c] for c in counties2[s]))/popplus12_s[s])**2 for s in states) for s in states]
})

# Initialize the map with a minimal background
us_map = folium.Map(location=[37.8, -96], zoom_start=5, tiles=None)
folium.TileLayer("cartodbpositron", name="Light Background").add_to(us_map)

# Define a colormap for infection-demand score
colormap = linear.Oranges_09.scale(results_data["InfectionDemandScore"].min(), results_data["InfectionDemandScore"].max())

# Use a public GeoJSON URL for US states
geojson_url = "https://raw.githubusercontent.com/PublicaMundi/MappingAPI/master/data/geojson/us-states.json"

# Add shaded regions based on Infection-Demand Score
folium.Choropleth(
    geo_data=geojson_url,
    name="choropleth",
    data=results_data,
    columns=["Region", "InfectionDemandScore"],
    key_on="feature.properties.name",
    fill_color="Oranges",
    fill_opacity=0.3,
    line_opacity=0.2,
    legend_name="Infection Demand Score"
).add_to(us_map)

# Add a heatmap for infection demand scores
# heat_data = [
#     [row["Latitude"], row["Longitude"], row["InfectionDemandScore"]] for _, row in results_data.iterrows()
# ]
# HeatMap(heat_data, radius=25, blur=10, max_zoom=5).add_to(us_map)

# Add circles and abbreviations for actual and optimized values

for _, row in results_data.iterrows():
    # Actual data circles
    folium.Circle(
        location=[row["Latitude"], row["Longitude"]],
        radius=row["Actual"] * 100,
        color="blue",
        fill=True,
        fill_opacity=0.6,
        tooltip=f"Actual: {row['Actual']}"
    ).add_to(us_map)

    # Optimized data circles
    folium.Circle(
        location=[row["Latitude"], row["Longitude"]],
        radius=row["Optimized"] * 100,
        color="red",
        fill=True,
        fill_opacity=0.4,
        tooltip=f"Optimized: {row['Optimized']}"
    ).add_to(us_map)

    # Add state abbreviation
    folium.Marker(
        location=[row["Latitude"] + 0.5, row["Longitude"]],
        icon=folium.DivIcon(
            html=f"<div style='font-size: 12px; color: black;'><b>{row['Abbreviation']}</b></div>"
        )
    ).add_to(us_map)

# Add a legend for the circles
legend_html = '''
<div style="position: fixed;
            bottom: 50px; left: 50px; width: 200px; height: 90px;
            background-color: white; z-index:9999; font-size:14px;
            border:2px solid grey; padding: 10px;">
    <b>Legend</b><br>
    <i style="background: blue; width: 10px; height: 10px; display: inline-block;"></i> Actual<br>
    <i style="background: red; width: 10px; height: 10px; display: inline-block;"></i> Optimized
</div>
'''
us_map.get_root().html.add_child(folium.Element(legend_html))

# Save and display the map
output_path = "us_map_infection_demand.html"
us_map.save(output_path)
print("Map saved as 'us_map_infection_demand.html'")
